In [ ]:
import os
import shutil
import json
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any, Literal, Union
import numpy as np
import pandas as pd
from numpy.linalg import norm
import math
import random

# Get all player matches


In [ ]:
os.makedirs("rift-rewind/", exist_ok=True)
os.makedirs("rift-rewind/players", exist_ok=True)
os.makedirs("rift-rewind/raw", exist_ok=True)
os.makedirs("rift-rewind/processed", exist_ok=True)
os.makedirs("rift-rewind/application", exist_ok=True)

In [ ]:
# for match in os.listdir("crazepic_matches"):
#     os.makedirs(f"rift-rewind/raw/{match.split(".")[0]}", exist_ok=True)
#     shutil.copyfile(f"crazepic_matches/{match}", f"rift-rewind/raw/{match.split(".")[0]}/{match}")
#     shutil.copyfile(f"crazepic_timelines/{match}", f"rift-rewind/raw/{match.split('.')[0]}/{match.split('.')[0]}_timeline.json")


In [ ]:
bucket_root = "rift-rewind"

In [ ]:
user_puuid = ""

In [ ]:
with open(f"{bucket_root}/players/{user_puuid}/info.json", "r", encoding="utf-8") as f:
    account_data = json.load(f)
    user_matches = account_data["matches"]

# Models


In [ ]:
class PlayerIdentity(BaseModel):
    puuid: str
    riotIdGameName: str
    riotIdTagline: str
    summonerName: str
    summonerId: str

    profileIcon: int
    summonerLevel: int

    participantId: int
    teamId: int
    individualPosition: str
    teamPosition: str
    role: str
    lane: str

    championName: str
    championId: int
    champLevel: int
    champExperience: int


class CombatStats(BaseModel):
    kills: int
    deaths: int
    assists: int

    firstBloodAssist: bool
    firstBloodKill: bool

    doubleKills: int
    tripleKills: int
    quadraKills: int
    pentaKills: int
    unrealKills: int

    killingSprees: int
    largestKillingSpree: int
    largestMultiKill: int

    longestTimeSpentLiving: int
    totalTimeSpentDead: int
    timePlayed: int

    totalDamageDealt: int
    totalDamageDealtToChampions: int
    physicalDamageDealt: int
    physicalDamageDealtToChampions: int
    magicDamageDealt: int
    magicDamageDealtToChampions: int
    trueDamageDealt: int
    trueDamageDealtToChampions: int

    totalDamageTaken: int
    physicalDamageTaken: int
    magicDamageTaken: int
    trueDamageTaken: int
    damageSelfMitigated: int

    totalHeal: int
    totalHealsOnTeammates: int
    totalDamageShieldedOnTeammates: int

    timeCCingOthers: int
    totalTimeCCDealt: int


class ObjectiveStats(BaseModel):
    turretKills: int
    turretTakedowns: int
    turretsLost: int
    inhibitorKills: int
    inhibitorTakedowns: int
    inhibitorsLost: int

    firstTowerAssist: bool
    firstTowerKill: bool

    baronKills: int
    dragonKills: int
    # riftHeraldKills: int
    # elderDragonKills: int
    # atakhanKills: int

    damageDealtToBuildings: int
    damageDealtToTurrets: int
    damageDealtToObjectives: int

    objectivesStolen: int
    objectivesStolenAssists: int

    nexusKills: int
    nexusTakedowns: int
    nexusLost: int


class EconomyStats(BaseModel):
    goldEarned: int
    goldSpent: int

    itemsPurchased: int
    consumablesPurchased: int
    item0: int
    item1: int
    item2: int
    item3: int
    item4: int
    item5: int
    item6: int


class VisionStats(BaseModel):
    visionScore: int
    wardsPlaced: int
    wardsKilled: int
    visionWardsBoughtInGame: int


class PerkSelection(BaseModel):
    perk: int
    var1: int
    var2: int
    var3: int


class PerkStyle(BaseModel):
    description: str
    selections: List[PerkSelection] = []
    style: int


class Perks(BaseModel):
    statPerks: Dict[str, int]
    styles: List[PerkStyle] = []


class AbilityStats(BaseModel):
    spell1Casts: int
    spell2Casts: int
    spell3Casts: int
    spell4Casts: int

    summoner1Id: int
    summoner2Id: int
    summoner1Casts: int
    summoner2Casts: int

    perks: Perks


class PingStats(BaseModel):
    allInPings: int
    assistMePings: int
    basicPings: int
    commandPings: int
    dangerPings: int
    getBackPings: int
    holdPings: int
    onMyWayPings: int
    pushPings: int
    retreatPings: int
    needVisionPings: int
    enemyMissingPings: int
    enemyVisionPings: int
    visionClearedPings: int


class FarmingStats(BaseModel):
    totalMinionsKilled: int
    neutralMinionsKilled: int
    totalAllyJungleMinionsKilled: int
    totalEnemyJungleMinionsKilled: int


class PlayerData(BaseModel):
    identity: PlayerIdentity
    combat: CombatStats
    objectives: ObjectiveStats
    economy: EconomyStats
    vision: VisionStats
    abilityUsage: AbilityStats
    pings: PingStats
    farming: FarmingStats


class TeamInfo(BaseModel):
    teamId: int
    win: bool


class GameContext(BaseModel):
    matchId: str
    platformId: str
    gameId: int
    queueId: int
    mapId: int
    gameMode: str
    gameType: str
    gameVersion: str
    gameName: str
    gameCreation: int
    gameStartTimestamp: int
    gameEndTimestamp: int
    gameDuration: int
    teams: List[TeamInfo]
    participants: List[str]
    gameEndedInSurrender: bool
    endOfGameResult: str


class MatchData(BaseModel):
    game: GameContext
    players: List[PlayerData]

In [ ]:
class Position(BaseModel):
    x: int
    y: int


class PlayerFrame(BaseModel):
    playerId: int
    position: Position
    currentGold: int
    totalGold: int
    level: int
    xp: int
    minionsKilled: int
    jungleMinionsKilled: int

    totalDamageDone: int
    totalDamageDoneToChampions: int
    totalDamageTaken: int
    physicalDamageDone: int
    physicalDamageDoneToChampions: int
    physicalDamageTaken: int
    magicDamageDone: int
    magicDamageDoneToChampions: int
    magicDamageTaken: int
    trueDamageDone: int
    trueDamageDoneToChampions: int
    trueDamageTaken: int


class PlayersFrame(BaseModel):
    playerFrames: Dict[str, PlayerFrame]  # Define PlayerFrame as needed


class VictimDamage(BaseModel):
    basic: bool
    physicalDamage: int
    magicDamage: int
    trueDamage: int
    name: str
    participantId: int
    spellName: str
    spellSlot: int
    type: str


class EventBase(BaseModel):
    type: str
    timestamp: int


class PauseEnd(EventBase):
    type: Literal["PAUSE_END"]
    realTimestamp: int


# class ItemPurchased(EventBase):
#     type: Literal["ITEM_PURCHASED"]
#     participantId: int
#     itemId: int


# class ItemUndo(EventBase):
#     type: Literal["ITEM_UNDO"]
#     participantId: int
#     beforeId: int
#     afterId: int
#     goldGain: int


# class SkillLevelUp(EventBase):
#     type: Literal["SKILL_LEVEL_UP"]
#     participantId: int
#     skillSlot: int
#     levelUpType: Optional[str]


# class ItemDestroyed(EventBase):
#     type: Literal["ITEM_DESTROYED"]
#     participantId: int
#     itemId: int


class WardPlaced(EventBase):
    type: Literal["WARD_PLACED"]
    creatorId: int
    wardType: str


class WardKill(EventBase):
    type: Literal["WARD_KILL"]
    killerId: int
    wardType: str


class LevelUp(EventBase):
    type: Literal["LEVEL_UP"]
    participantId: int
    level: int


class ChampionKill(EventBase):
    type: Literal["CHAMPION_KILL"]
    killerId: int
    victimId: int
    bounty: int
    shutdownBounty: int
    killStreakLength: int
    position: Position
    assistingParticipantIds: List[int] = []
    victimDamageDealt: List[VictimDamage] = []
    victimDamageReceived: List[VictimDamage] = []


class ChampionSpecialKill(EventBase):
    type: Literal["CHAMPION_SPECIAL_KILL"]
    killType: str
    killerId: int
    # position: Optional[Position]


class ChampionSpecialKillFirstBlood(ChampionSpecialKill):
    killType: Literal["KILL_FIRST_BLOOD"]


class ChampionSpecialKillMultiKill(ChampionSpecialKill):
    killType: Literal["KILL_MULTI"]
    multiKillLength: int


# class TurretPlateDestroyed(EventBase):
#     type: Literal["TURRET_PLATE_DESTROYED"]
#     killerId: int
#     laneType: str
#     teamId: int
#     position: Optional[Position]


class EliteMonsterKill(EventBase):
    type: Literal["ELITE_MONSTER_KILL"]
    killerId: int
    killerTeamId: int
    # bounty: Optional[int]
    monsterType: Literal["HORDE", "ATAKHAN", "DRAGON", "RIFTHERALD", "BARON_NASHOR"]
    # position: Optional[Position]
    assistingParticipantIds: List[int] = []


class DragonMonsterKill(EliteMonsterKill):
    monsterType: Literal["DRAGON"]
    monsterSubType: Literal["AIR_DRAGON", "WATER_DRAGON", "FIRE_DRAGON", "EARTH_DRAGON", "CHEMTECH_DRAGON", "HEXTECH_DRAGON", "ELDER_DRAGON", "UNKNOWN"]


class BuildingKill(EventBase):
    type: Literal["BUILDING_KILL"]
    killerId: int
    # bounty: Optional[int]
    buildingType: str
    laneType: Literal["MID_LANE", "TOP_LANE", "BOT_LANE"]
    # position: Optional[Position]
    buildingTeamId: int
    assistingParticipantIds: List[int] = []


class TowerKill(BuildingKill):
    buildingType: Literal["TOWER_BUILDING"]
    towerType: Literal["OUTER_TURRET", "INNER_TURRET", "BASE_TURRET", "NEXUS_TURRET"]


class InhibitorKill(BuildingKill):
    buildingType: Literal["INHIBITOR_BUILDING"]


# class ItemSold(EventBase):
#     type: Literal["ITEM_SOLD"]
#     participantId: int
#     itemId: int


# class ObjectiveBountyPrestart(EventBase):
#     type: Literal["OBJECTIVE_BOUNTY_PRESTART"]
#     teamId: int
#     actualStartTime: int


# class ObjectiveBountyFinish(EventBase):
#     type: Literal["OBJECTIVE_BOUNTY_FINISH"]
#     teamId: int


# class DragonSoulGiven(EventBase):
#     type: Literal["DRAGON_SOUL_GIVEN"]
#     teamId: int
#     name: str


class GameEnd(EventBase):
    type: Literal["GAME_END"]
    gameId: int
    realTimestamp: int
    winningTeam: int


# class ChampionTransform(EventBase):
#     type: Literal["CHAMPION_TRANSFORM"]
#     participantId: int
#     transformType: str


# class FeatUpdate(EventBase):
#     type: Literal["FEAT_UPDATE"]
#     teamId: int
#     featType: str
#     featValue: int


class FrameData(BaseModel):
    playersFrames: List[PlayersFrame]  # Define ParticipantFrame as needed
    events: List[
        Union[
            PauseEnd,
            # ItemPurchased,
            # ItemUndo,
            # SkillLevelUp,
            WardPlaced,
            WardKill,
            LevelUp,
            # ItemDestroyed,
            ChampionKill,
            ChampionSpecialKill,
            # TurretPlateDestroyed,
            EliteMonsterKill,
            BuildingKill,
            # ItemSold,
            # ObjectiveBountyPrestart,
            # ObjectiveBountyFinish,
            # DragonSoulGiven,
            GameEnd,
            # ChampionTransform,
            # FeatUpdate,
        ]
    ]  # Define EventData as needed


class TimelinePlayerData(BaseModel):
    playerId: int
    puuid: str


class TimelineMetadata(BaseModel):
    dataVersion: str
    matchId: str
    gameId: int
    endOfGameResult: str
    players: List[TimelinePlayerData]


class MatchTimelineData(BaseModel):
    metadata: TimelineMetadata
    frameInterval: int
    frames: List[FrameData]  # Define FrameData as needed

# Processed Data


In [ ]:
def process_match(match_id: str):
    with open(f"{bucket_root}/raw/{match_id}/{match_id + '.json'}", "r", encoding="utf-8") as f:
        data = json.load(f)
        match = MatchData(
            game=GameContext(
                matchId=data["metadata"]["matchId"],
                platformId=data["info"]["platformId"],
                gameId=data["info"]["gameId"],
                queueId=data["info"]["queueId"],
                mapId=data["info"]["mapId"],
                gameMode=data["info"]["gameMode"],
                gameType=data["info"]["gameType"],
                gameVersion=data["info"]["gameVersion"],
                gameName=data["info"]["gameName"],
                gameCreation=data["info"]["gameCreation"],
                gameStartTimestamp=data["info"]["gameStartTimestamp"],
                gameEndTimestamp=data["info"]["gameEndTimestamp"],
                gameDuration=data["info"]["gameDuration"],
                teams=[TeamInfo(teamId=team["teamId"], win=team["win"]) for team in data["info"]["teams"]],
                participants=[participant["puuid"] for participant in data["info"]["participants"]],
                gameEndedInSurrender=any([player["gameEndedInSurrender"] for player in data["info"]["participants"]]),
                endOfGameResult=data["info"]["endOfGameResult"],
            ),
            players=[
                PlayerData(
                    identity=PlayerIdentity(
                        puuid=player["puuid"],
                        riotIdGameName=player["riotIdGameName"],
                        riotIdTagline=player["riotIdTagline"],
                        summonerName=player["summonerName"],
                        summonerId=player["summonerId"],
                        profileIcon=player["profileIcon"],
                        summonerLevel=player["summonerLevel"],
                        participantId=player["participantId"],
                        teamId=player["teamId"],
                        individualPosition=player["individualPosition"],
                        teamPosition=player["teamPosition"],
                        role=player["role"],
                        lane=player["lane"],
                        championName=player["championName"],
                        championId=player["championId"],
                        champLevel=player["champLevel"],
                        champExperience=player["champExperience"],
                    ),
                    combat=CombatStats(
                        kills=player["kills"],
                        deaths=player["deaths"],
                        assists=player["assists"],
                        firstBloodAssist=player["firstBloodAssist"],
                        firstBloodKill=player["firstBloodKill"],
                        doubleKills=player["doubleKills"],
                        tripleKills=player["tripleKills"],
                        quadraKills=player["quadraKills"],
                        pentaKills=player["pentaKills"],
                        unrealKills=player["unrealKills"],
                        killingSprees=player["killingSprees"],
                        largestKillingSpree=player["largestKillingSpree"],
                        largestMultiKill=player["largestMultiKill"],
                        longestTimeSpentLiving=player["longestTimeSpentLiving"],
                        totalTimeSpentDead=player["totalTimeSpentDead"],
                        timePlayed=player["timePlayed"],
                        totalDamageDealt=player["totalDamageDealt"],
                        totalDamageDealtToChampions=player["totalDamageDealtToChampions"],
                        physicalDamageDealt=player["physicalDamageDealt"],
                        physicalDamageDealtToChampions=player["physicalDamageDealtToChampions"],
                        magicDamageDealt=player["magicDamageDealt"],
                        magicDamageDealtToChampions=player["magicDamageDealtToChampions"],
                        trueDamageDealt=player["trueDamageDealt"],
                        trueDamageDealtToChampions=player["trueDamageDealtToChampions"],
                        totalDamageTaken=player["totalDamageTaken"],
                        physicalDamageTaken=player["physicalDamageTaken"],
                        magicDamageTaken=player["magicDamageTaken"],
                        trueDamageTaken=player["trueDamageTaken"],
                        damageSelfMitigated=player["damageSelfMitigated"],
                        totalHeal=player["totalHeal"],
                        totalHealsOnTeammates=player["totalHealsOnTeammates"],
                        totalDamageShieldedOnTeammates=player["totalDamageShieldedOnTeammates"],
                        timeCCingOthers=player["timeCCingOthers"],
                        totalTimeCCDealt=player["totalTimeCCDealt"],
                    ),
                    objectives=ObjectiveStats(
                        turretKills=player["turretKills"],
                        turretTakedowns=player["turretTakedowns"],
                        turretsLost=player["turretsLost"],
                        inhibitorKills=player["inhibitorKills"],
                        inhibitorTakedowns=player["inhibitorTakedowns"],
                        inhibitorsLost=player["inhibitorsLost"],
                        firstTowerAssist=player["firstTowerAssist"],
                        firstTowerKill=player["firstTowerKill"],
                        baronKills=player["baronKills"],
                        dragonKills=player["dragonKills"],
                        # riftHeraldKills=player["riftHeraldKills"],
                        # elderDragonKills=player["elderDragonKills"],
                        # atakhanKills=player["atakhanKills"],
                        damageDealtToBuildings=player["damageDealtToBuildings"],
                        damageDealtToTurrets=player["damageDealtToTurrets"],
                        damageDealtToObjectives=player["damageDealtToObjectives"],
                        objectivesStolen=player["objectivesStolen"],
                        objectivesStolenAssists=player["objectivesStolenAssists"],
                        nexusKills=player["nexusKills"],
                        nexusTakedowns=player["nexusTakedowns"],
                        nexusLost=player["nexusLost"],
                    ),
                    economy=EconomyStats(
                        goldEarned=player["goldEarned"],
                        goldSpent=player["goldSpent"],
                        itemsPurchased=player["itemsPurchased"],
                        consumablesPurchased=player["consumablesPurchased"],
                        item0=player["item0"],
                        item1=player["item1"],
                        item2=player["item2"],
                        item3=player["item3"],
                        item4=player["item4"],
                        item5=player["item5"],
                        item6=player["item6"],
                    ),
                    vision=VisionStats(
                        visionScore=player["visionScore"],
                        wardsPlaced=player["wardsPlaced"],
                        wardsKilled=player["wardsKilled"],
                        visionWardsBoughtInGame=player["visionWardsBoughtInGame"],
                    ),
                    abilityUsage=AbilityStats(
                        spell1Casts=player["spell1Casts"],
                        spell2Casts=player["spell2Casts"],
                        spell3Casts=player["spell3Casts"],
                        spell4Casts=player["spell4Casts"],
                        summoner1Id=player["summoner1Id"],
                        summoner2Id=player["summoner2Id"],
                        summoner1Casts=player["summoner1Casts"],
                        summoner2Casts=player["summoner2Casts"],
                        perks=Perks(
                            statPerks=player["perks"]["statPerks"],
                            styles=[
                                PerkStyle(
                                    description=style["description"],
                                    selections=[
                                        PerkSelection(
                                            perk=selection["perk"],
                                            var1=selection["var1"],
                                            var2=selection["var2"],
                                            var3=selection["var3"],
                                        )
                                        for selection in style["selections"]
                                    ],
                                    style=style["style"],
                                )
                                for style in player["perks"]["styles"]
                            ],
                        ),
                    ),
                    pings=PingStats(
                        allInPings=player["allInPings"],
                        assistMePings=player["assistMePings"],
                        basicPings=player["basicPings"],
                        commandPings=player["commandPings"],
                        dangerPings=player["dangerPings"],
                        getBackPings=player["getBackPings"],
                        holdPings=player["holdPings"],
                        onMyWayPings=player["onMyWayPings"],
                        pushPings=player["pushPings"],
                        retreatPings=player["retreatPings"],
                        needVisionPings=player["needVisionPings"],
                        enemyMissingPings=player["enemyMissingPings"],
                        enemyVisionPings=player["enemyVisionPings"],
                        visionClearedPings=player["visionClearedPings"],
                    ),
                    farming=FarmingStats(
                        totalMinionsKilled=player["totalMinionsKilled"],
                        neutralMinionsKilled=player["neutralMinionsKilled"],
                        totalAllyJungleMinionsKilled=player["totalAllyJungleMinionsKilled"],
                        totalEnemyJungleMinionsKilled=player["totalEnemyJungleMinionsKilled"],
                    ),
                )
                for player in data["info"]["participants"]
            ],
        )
        return match

In [ ]:
def parse_event(event: dict):
    match event["type"]:
        # --- Basic events ---
        case "PAUSE_END":
            return PauseEnd(**event)
        case "WARD_PLACED":
            return WardPlaced(**event)
        case "WARD_KILL":
            return WardKill(**event)
        case "LEVEL_UP":
            return LevelUp(**event)
        case "CHAMPION_KILL":
            return ChampionKill(**event)
        case "GAME_END":
            return GameEnd(**event)

        # --- CHAMPION_SPECIAL_KILL subtypes ---
        case "CHAMPION_SPECIAL_KILL":
            kill_type = event.get("killType")
            if kill_type == "KILL_FIRST_BLOOD":
                return ChampionSpecialKillFirstBlood(**event)
            elif kill_type == "KILL_MULTI":
                return ChampionSpecialKillMultiKill(**event)
            else:
                return ChampionSpecialKill(**event)

        # --- ELITE_MONSTER_KILL subtypes ---
        case "ELITE_MONSTER_KILL":
            if event.get("monsterType") == "DRAGON":
                return DragonMonsterKill(**event)
            else:
                return EliteMonsterKill(**event)

        # --- BUILDING_KILL subtypes ---
        case "BUILDING_KILL":
            building_type = event.get("buildingType")
            if building_type == "TOWER_BUILDING":
                return TowerKill(**event, buildingTeamId=event["teamId"])
            elif building_type == "INHIBITOR_BUILDING":
                return InhibitorKill(**event, buildingTeamId=event["teamId"])
            else:
                return BuildingKill(**event, buildingTeamId=event["teamId"])

        case _:
            return None  # Ignore unsupported or unknown types

In [ ]:



def process_timeline(match_id: str):
    with open(f"{bucket_root}/raw/{match_id}/{match_id}_timeline.json", "r", encoding="utf-8") as f:
        data = json.load(f)
        timeline = MatchTimelineData(
            metadata=TimelineMetadata(
                dataVersion=data["metadata"]["dataVersion"],
                matchId=data["metadata"]["matchId"],
                gameId=data["info"]["gameId"],
                endOfGameResult=data["info"]["endOfGameResult"],
                players=[
                    TimelinePlayerData(
                        playerId=player["participantId"],
                        puuid=player["puuid"],
                    )
                    for player in data["info"]["participants"]
                ],
            ),
            frameInterval=data["info"]["frameInterval"],
            frames=[
                FrameData(
                    playersFrames=[
                        PlayersFrame(
                            playerFrames={
                                player_id: PlayerFrame(
                                    playerId=player_frame["participantId"],
                                    position=Position(
                                        x=player_frame["position"]["x"],
                                        y=player_frame["position"]["y"],
                                    ),
                                    currentGold=player_frame["currentGold"],
                                    totalGold=player_frame["totalGold"],
                                    level=player_frame["level"],
                                    xp=player_frame["xp"],
                                    minionsKilled=player_frame["minionsKilled"],
                                    jungleMinionsKilled=player_frame["jungleMinionsKilled"],
                                    totalDamageDone=player_frame["damageStats"]["totalDamageDone"],
                                    totalDamageDoneToChampions=player_frame["damageStats"]["totalDamageDoneToChampions"],
                                    totalDamageTaken=player_frame["damageStats"]["totalDamageTaken"],
                                    physicalDamageDone=player_frame["damageStats"]["physicalDamageDone"],
                                    physicalDamageDoneToChampions=player_frame["damageStats"]["physicalDamageDoneToChampions"],
                                    physicalDamageTaken=player_frame["damageStats"]["physicalDamageTaken"],
                                    magicDamageDone=player_frame["damageStats"]["magicDamageDone"],
                                    magicDamageDoneToChampions=player_frame["damageStats"]["magicDamageDoneToChampions"],
                                    magicDamageTaken=player_frame["damageStats"]["magicDamageTaken"],
                                    trueDamageDone=player_frame["damageStats"]["trueDamageDone"],
                                    trueDamageDoneToChampions=player_frame["damageStats"]["trueDamageDoneToChampions"],
                                    trueDamageTaken=player_frame["damageStats"]["trueDamageTaken"],
                                )
                                for player_id, player_frame in frame["participantFrames"].items()
                            }
                        )
                    ],
                    events=[ev for event in frame.get("events", []) if (ev := parse_event(event)) is not None],
                )
                for frame in data["info"]["frames"]
            ],
        )
        return timeline

In [ ]:
for match_id in user_matches:
    try:
        os.makedirs(f"rift-rewind/processed/{match_id}", exist_ok=True)
        match = process_match(match_id)
        timeline = process_timeline(match_id)
        with open(f"rift-rewind/processed/{match_id}/{match_id}_processed.json", "w", encoding="utf-8") as out_f:
            out_f.write(match.model_dump_json(indent=4))
        with open(f"rift-rewind/processed/{match_id}/{match_id}_timeline_processed.json", "w", encoding="utf-8") as out_f:
            out_f.write(timeline.model_dump_json(indent=4))
    except Exception as e:
        print(f"Error processing match {match_id}: {e}")
        if os.path.exists(f"rift-rewind/processed/{match_id}/{match_id}_processed.json"):
            os.remove(f"rift-rewind/processed/{match_id}/{match_id}_processed.json")
        if os.path.exists(f"rift-rewind/processed/{match_id}/{match_id}_timeline_processed.json"):
            os.remove(f"rift-rewind/processed/{match_id}/{match_id}_timeline_processed.json")
        os.rmdir(f"rift-rewind/processed/{match_id}")

# Application Data


In [ ]:
sampled_players = {}
for rank in os.listdir("Player_Data"):
    players_for_rank_count = {}
    for temp_match in os.listdir(f"Player_Data/{rank}/data/raw/match_data"):
        with open(f"Player_Data/{rank}/data/raw/match_data/{temp_match}", "r", encoding="utf-8") as f:
            match_data = json.load(f)
            for player in match_data["info"]["participants"]:
                if player["puuid"] not in players_for_rank_count:
                    players_for_rank_count[player["puuid"]] = 0
                players_for_rank_count[player["puuid"]] += 1
    print(sorted(players_for_rank_count.items(), key=lambda x: x[1], reverse=True)[:50])
    actual_players = filter(lambda x: players_for_rank_count[x] >= 30, players_for_rank_count)
    sampled_players[rank] = list(actual_players)
print(sampled_players)

In [ ]:
with open("sampled_players.json", "w", encoding="utf-8") as f:
    json.dump(sampled_players, f, indent=4)

In [ ]:
for rank in os.listdir("Player_Data"):
    print(f"\nRank: {rank} " + str(len(os.listdir(f"Player_Data/{rank}/data/raw/match_data"))))

In [ ]:
for rank in sampled_players:
    print(f"Rank: {rank}, Sampled Players: {len(sampled_players[rank])}")

In [ ]:
# if player has summoner rift games, generate AI insights w/ recommendations
# else, just stat bragging

In [ ]:
# Combat Effectiveness (kills, KDA, multi-kills, first bloods, streaks, shutdowns)
# Map Influence (roaming, objective participation, kill assists)
# Team Contribution (assists, healing/shielding, CC, objective assists)
# Resource Efficiency (gold/CS per minute, damage per gold, objective efficiency)
# Consistency (variance/stability across metrics)
# Vision & Awareness (wards, vision score, pings, objective vision)

In [ ]:
size = 0
for path, dirs, files in os.walk("rift-rewind/raw/NA1_5196186524"):
    for f in files:
        fp = os.path.join(path, f)
        size += os.stat(fp).st_size

# display size
print("Folder size: " + str(size / (1024 * 1024)) + " MB")

## Ionia

### Raw

In [ ]:
def ranked_cdf_score(
    value,
    sample_array,
    alpha=0.7,
    top_extrapolate=True,
    q_cutoff=0.995,
    k=5.0,
    eps=1e-6,
):
    sample_sorted = np.sort(sample_array)
    N = len(sample_sorted)
    if N == 0:
        return 0.0

    percentiles = np.arange(1, N + 1) / N
    score = np.interp(value, sample_sorted, percentiles, left=0.0, right=1.0)

    if top_extrapolate:
        q_cutoff = min(max(q_cutoff, 0.0), 1.0)
        threshold = np.quantile(sample_sorted, q_cutoff)
        base = np.interp(threshold, sample_sorted, percentiles, left=0.0, right=1.0)
        top_tail = max(0.0, value - threshold)
        scale = max(threshold, eps)
        tail = (1.0 - base) * (1.0 - np.exp(-k * top_tail / scale))
        score = min(score, base + tail)  # ensures percentile dominates for values below threshold

    score = score**alpha
    return float(min(max(score, 0.0), 1.0))

In [ ]:
def consistency_score_logistic(mean, std, a=10, b=0.3, eps=1e-6):
    cv = std / (abs(mean) + eps)
    return 1 / (1 + math.exp(a * (cv - b)))

#### Extract Ionia Metrics

In [ ]:
def extract_combat_metrics(player: PlayerData, match_data: MatchData, timeline_data: MatchTimelineData):
    team_players = [p for p in match_data.players if p.identity.teamId == player.identity.teamId]

    team_kills = sum(p.combat.kills for p in team_players)
    total_team_damage = sum(p.combat.totalDamageDealtToChampions for p in team_players)

    kill_participation = (player.combat.kills + player.combat.assists) / max(1, team_kills)
    kda = (player.combat.kills + player.combat.assists) / max(1, player.combat.deaths)
    damage_per_min = player.combat.totalDamageDealtToChampions / max(1, match_data.game.gameDuration / 60)
    damage_share = player.combat.totalDamageDealtToChampions / max(1, total_team_damage)

    multi_kills = {
        "double": player.combat.doubleKills,
        "triple": player.combat.tripleKills,
        "quadra": player.combat.quadraKills,
        "penta": player.combat.pentaKills,
    }
    first_blood_kill = player.combat.firstBloodKill
    first_blood_assist = player.combat.firstBloodAssist

    # TODO: kill streaks and shutdowns
    # TODO: put survival calculations here

    return {
        "kill_participation": kill_participation,
        "kda": kda,
        "damage_per_min": damage_per_min,
        "damage_share": damage_share,
        "double_kills": multi_kills["double"],
        "triple_kills": multi_kills["triple"],
        "quadra_kills": multi_kills["quadra"],
        "penta_kills": multi_kills["penta"],
        "first_blood_kill": first_blood_kill,
        "first_blood_assist": first_blood_assist,
    }


def extract_team_contribution_metrics(player: PlayerData, match_data: MatchData, timeline_data: MatchTimelineData):
    time_played_minutes = match_data.game.gameDuration / 60
    team_players = [p for p in match_data.players if p.identity.teamId == player.identity.teamId]

    team_kills = sum(p.combat.kills for p in team_players)

    kill_participation = (player.combat.kills + player.combat.assists) / max(1, team_kills)

    heal_shield_per_min = (player.combat.totalHealsOnTeammates + player.combat.totalDamageShieldedOnTeammates) / time_played_minutes

    total_team_objectives_taken = 0
    team_objectives_participated = 0
    for frame in timeline_data.frames:
        for event in frame.events:
            if event.type == "ELITE_MONSTER_KILL":
                if hasattr(event, "killerTeamId") and event.killerTeamId == player.identity.teamId:
                    total_team_objectives_taken += 1
                    if player.identity.participantId in event.assistingParticipantIds:
                        team_objectives_participated += 1
            # elif event.type == "BUILDING_KILL":
            #     if hasattr(event, "buildingTeamId") and event.buildingTeamId != player.identity.teamId:
            #         total_team_objectives_taken += 1
            #         if player.identity.participantId in event.assistingParticipantIds:
            #             team_objectives_participated += 1
    objective_participation_ratio = team_objectives_participated / max(1, total_team_objectives_taken)

    # roaming / lane presence
    time_outside_lane = 0
    # TODO: LOOK AT EVENT DATA TO SEE IF THEY PARTICIPATED IN OBJECTIVES OUTSIDE THEIR LANE, IF SO INCREMENT time_outside_lane
    total_time = match_data.game.gameDuration / timeline_data.frameInterval
    lane = player.identity.lane
    playerId = timeline_data.metadata.players[next(i for i, p in enumerate(timeline_data.metadata.players) if p.puuid == player.identity.puuid)].playerId
    for frame in timeline_data.frames:
        pf = next((p for p in frame.playersFrames if p.playerFrames[str(playerId)]), None)
        if pf:
            position = pf.playerFrames[str(playerId)].position
            # map is 15000 x 15000
            BOT_BLUE_SIDE_LINE = ((0, 1000), (15000, 1000))
            TOP_BLUE_SIDE_LINE = ((1000, 0), (1000, 15000))
            BOT_RED_SIDE_LINE = ((14000, 15000), (14000, 0))
            TOP_RED_SIDE_LINE = ((15000, 14000), (0, 14000))
            MID_LINE = ((0, 0), (15000, 15000))
            distance_threshold = 1000

            def point_line_distance(point, line_start, line_end):
                point = np.array(point)
                p1 = np.array(line_start)
                p2 = np.array(line_end)
                d = norm(np.cross(p2 - p1, p1 - point)) / norm(p2 - p1)
                return d

            if lane == "TOP":
                dist1 = point_line_distance((position.x, position.y), TOP_BLUE_SIDE_LINE[0], TOP_BLUE_SIDE_LINE[1])
                dist2 = point_line_distance((position.x, position.y), TOP_RED_SIDE_LINE[0], TOP_RED_SIDE_LINE[1])
                if min(dist1, dist2) > distance_threshold:
                    time_outside_lane += 1
            elif lane == "MID":
                dist = point_line_distance((position.x, position.y), MID_LINE[0], MID_LINE[1])
                if dist > distance_threshold:
                    time_outside_lane += 1
            elif lane == "BOT":
                dist1 = point_line_distance((position.x, position.y), BOT_BLUE_SIDE_LINE[0], BOT_BLUE_SIDE_LINE[1])
                dist2 = point_line_distance((position.x, position.y), BOT_RED_SIDE_LINE[0], BOT_RED_SIDE_LINE[1])
                if min(dist1, dist2) > distance_threshold:
                    time_outside_lane += 1
            elif lane == "JUNGLE":
                distance_to_top = min(
                    point_line_distance((position.x, position.y), TOP_BLUE_SIDE_LINE[0], TOP_BLUE_SIDE_LINE[1]),
                    point_line_distance((position.x, position.y), TOP_RED_SIDE_LINE[0], TOP_RED_SIDE_LINE[1]),
                )
                distance_to_bot = min(
                    point_line_distance((position.x, position.y), BOT_BLUE_SIDE_LINE[0], BOT_BLUE_SIDE_LINE[1]),
                    point_line_distance((position.x, position.y), BOT_RED_SIDE_LINE[0], BOT_RED_SIDE_LINE[1]),
                )
                distance_to_mid = point_line_distance((position.x, position.y), MID_LINE[0], MID_LINE[1])
                if min(distance_to_top, distance_to_bot, distance_to_mid) < distance_threshold:
                    time_outside_lane += 1
            total_time += 1

    roaming_ratio = time_outside_lane / max(1, total_time)

    return {
        "kill_participation": kill_participation,
        "heal_shield_per_min": heal_shield_per_min,
        "objective_participation_ratio": objective_participation_ratio,
        "roaming_ratio": roaming_ratio,
    }


def extract_resource_efficiency_metrics(player: PlayerData, match_data: MatchData, timeline_data: MatchTimelineData):
    time_played_minutes = match_data.game.gameDuration / 60

    gold_per_min = player.economy.goldEarned / max(1, time_played_minutes)
    cs_per_min = (player.farming.totalMinionsKilled + player.farming.neutralMinionsKilled + player.farming.totalAllyJungleMinionsKilled + player.farming.totalEnemyJungleMinionsKilled) / max(
        1, time_played_minutes
    )

    return {
        "gold_per_min": gold_per_min,
        "cs_per_min": cs_per_min,
    }


def extract_vision_awareness_metrics(player: PlayerData, match_data: MatchData, timeline_data: MatchTimelineData):
    time_played_minutes = match_data.game.gameDuration / 60

    vision_score_per_min = player.vision.visionScore / max(1, time_played_minutes)

    return {
        "vision_score_per_min": vision_score_per_min,
    }

#### Raw

In [ ]:
def get_raw_ionia_metrics_for_match(puuid: str, match_data: MatchData, timeline_data: MatchTimelineData):
    player_data = next((player for player in match_data.players if player.identity.puuid == puuid), None)
    if player_data is None:
        return None

    return {
        "combat_metrics": extract_combat_metrics(player_data, match_data, timeline_data),
        "team_contribution_metrics": extract_team_contribution_metrics(player_data, match_data, timeline_data),
        "resource_efficiency_metrics": extract_resource_efficiency_metrics(player_data, match_data, timeline_data),
        "vision_awareness_metrics": extract_vision_awareness_metrics(player_data, match_data, timeline_data),
    }

### Info

#### Calculate Baselines for Ionia Stats

Sample 20 players from each rank and 30 of their SR games each (given they have 30+ SR games)

Directory structure is
- /Player_Data/{rank}_sr/data/raw/match_data
- /Player_Data/{rank}_sr/data/raw/timeline

In [ ]:
valid_players = {}
for rank in os.listdir("Player_Data"):
    players_for_rank_count = {}
    for temp_match in os.listdir(f"Player_Data/{rank}/data/raw/match_data"):
        with open(f"Player_Data/{rank}/data/raw/match_data/{temp_match}", "r", encoding="utf-8") as f:
            match_data = json.load(f)
            for player in match_data["info"]["participants"]:
                if player["puuid"] not in players_for_rank_count:
                    players_for_rank_count[player["puuid"]] = 0
                players_for_rank_count[player["puuid"]] += 1
    # 30+ games played
    actual_players = filter(lambda x: players_for_rank_count[x] >= 30, players_for_rank_count)
    valid_players[rank] = list(actual_players)

In [ ]:
for rank in valid_players:
    print(f"Rank: {rank}, Valid Players: {len(valid_players[rank])}")
    assert len(valid_players[rank]) >= 20

In [ ]:
sampled_players = {}
for rank in valid_players:
    sampled_players[rank] = random.sample(valid_players[rank], 20)

In [ ]:
sampled_players

In [ ]:
player_matches = {
    rank: {player: [] for player in sampled_players[rank]} for rank in sampled_players
}
for rank in os.listdir("Player_Data"):
    players_in_rank = sampled_players[rank]
    for temp_match in os.listdir(f"Player_Data/{rank}/data/raw/match_data"):
        with open(f"Player_Data/{rank}/data/raw/match_data/{temp_match}", "r", encoding="utf-8") as f:
            match_data = json.load(f)
            for player in match_data["info"]["participants"]:
                if player["puuid"] in players_in_rank:
                    player_matches[rank][player["puuid"]].append(match_data["metadata"]["matchId"])
for rank in player_matches:
    for player in player_matches[rank]:
        # print(f"Rank: {rank}, Player: {player}, Matches: {len(player_matches[rank][player])}")
        player_matches[rank][player] = random.sample(player_matches[rank][player], 30)
for rank in player_matches:
    print(f"Rank: {rank}")
    for player in player_matches[rank]:
        print(f"  Player: {player}, Matches: {len(player_matches[rank][player])}")

In [ ]:
with open("sampled_player_matches.json", "w", encoding="utf-8") as f:
    json.dump(player_matches, f, indent=4)

In [ ]:
# I FORGOT TO PROCESS RAW MATCHES

In [ ]:
for rank in os.listdir("Player_Data"):
    os.makedirs(f"Player_Data/{rank}/data/processed/match_data", exist_ok=True)
    os.makedirs(f"Player_Data/{rank}/data/processed/timeline_data", exist_ok=True)
    for temp_match in os.listdir(f"Player_Data/{rank}/data/raw/match_data"):
        try:
            with open(f"Player_Data/{rank}/data/raw/match_data/{temp_match}", "r", encoding="utf-8") as f:
                data = json.load(f)
                match = MatchData(
                    game=GameContext(
                        matchId=data["metadata"]["matchId"],
                        platformId=data["info"]["platformId"],
                        gameId=data["info"]["gameId"],
                        queueId=data["info"]["queueId"],
                        mapId=data["info"]["mapId"],
                        gameMode=data["info"]["gameMode"],
                        gameType=data["info"]["gameType"],
                        gameVersion=data["info"]["gameVersion"],
                        gameName=data["info"]["gameName"],
                        gameCreation=data["info"]["gameCreation"],
                        gameStartTimestamp=data["info"]["gameStartTimestamp"],
                        gameEndTimestamp=data["info"]["gameEndTimestamp"],
                        gameDuration=data["info"]["gameDuration"],
                        teams=[TeamInfo(teamId=team["teamId"], win=team["win"]) for team in data["info"]["teams"]],
                        participants=[participant["puuid"] for participant in data["info"]["participants"]],
                        gameEndedInSurrender=any([player["gameEndedInSurrender"] for player in data["info"]["participants"]]),
                        endOfGameResult=data["info"]["endOfGameResult"],
                    ),
                    players=[
                        PlayerData(
                            identity=PlayerIdentity(
                                puuid=player["puuid"],
                                riotIdGameName=player["riotIdGameName"],
                                riotIdTagline=player["riotIdTagline"],
                                summonerName=player["summonerName"],
                                summonerId=player["summonerId"],
                                profileIcon=player["profileIcon"],
                                summonerLevel=player["summonerLevel"],
                                participantId=player["participantId"],
                                teamId=player["teamId"],
                                individualPosition=player["individualPosition"],
                                teamPosition=player["teamPosition"],
                                role=player["role"],
                                lane=player["lane"],
                                championName=player["championName"],
                                championId=player["championId"],
                                champLevel=player["champLevel"],
                                champExperience=player["champExperience"],
                            ),
                            combat=CombatStats(
                                kills=player["kills"],
                                deaths=player["deaths"],
                                assists=player["assists"],
                                firstBloodAssist=player["firstBloodAssist"],
                                firstBloodKill=player["firstBloodKill"],
                                doubleKills=player["doubleKills"],
                                tripleKills=player["tripleKills"],
                                quadraKills=player["quadraKills"],
                                pentaKills=player["pentaKills"],
                                unrealKills=player["unrealKills"],
                                killingSprees=player["killingSprees"],
                                largestKillingSpree=player["largestKillingSpree"],
                                largestMultiKill=player["largestMultiKill"],
                                longestTimeSpentLiving=player["longestTimeSpentLiving"],
                                totalTimeSpentDead=player["totalTimeSpentDead"],
                                timePlayed=player["timePlayed"],
                                totalDamageDealt=player["totalDamageDealt"],
                                totalDamageDealtToChampions=player["totalDamageDealtToChampions"],
                                physicalDamageDealt=player["physicalDamageDealt"],
                                physicalDamageDealtToChampions=player["physicalDamageDealtToChampions"],
                                magicDamageDealt=player["magicDamageDealt"],
                                magicDamageDealtToChampions=player["magicDamageDealtToChampions"],
                                trueDamageDealt=player["trueDamageDealt"],
                                trueDamageDealtToChampions=player["trueDamageDealtToChampions"],
                                totalDamageTaken=player["totalDamageTaken"],
                                physicalDamageTaken=player["physicalDamageTaken"],
                                magicDamageTaken=player["magicDamageTaken"],
                                trueDamageTaken=player["trueDamageTaken"],
                                damageSelfMitigated=player["damageSelfMitigated"],
                                totalHeal=player["totalHeal"],
                                totalHealsOnTeammates=player["totalHealsOnTeammates"],
                                totalDamageShieldedOnTeammates=player["totalDamageShieldedOnTeammates"],
                                timeCCingOthers=player["timeCCingOthers"],
                                totalTimeCCDealt=player["totalTimeCCDealt"],
                            ),
                            objectives=ObjectiveStats(
                                turretKills=player["turretKills"],
                                turretTakedowns=player["turretTakedowns"],
                                turretsLost=player["turretsLost"],
                                inhibitorKills=player["inhibitorKills"],
                                inhibitorTakedowns=player["inhibitorTakedowns"],
                                inhibitorsLost=player["inhibitorsLost"],
                                firstTowerAssist=player["firstTowerAssist"],
                                firstTowerKill=player["firstTowerKill"],
                                baronKills=player["baronKills"],
                                dragonKills=player["dragonKills"],
                                # riftHeraldKills=player["riftHeraldKills"],
                                # elderDragonKills=player["elderDragonKills"],
                                # atakhanKills=player["atakhanKills"],
                                damageDealtToBuildings=player["damageDealtToBuildings"],
                                damageDealtToTurrets=player["damageDealtToTurrets"],
                                damageDealtToObjectives=player["damageDealtToObjectives"],
                                objectivesStolen=player["objectivesStolen"],
                                objectivesStolenAssists=player["objectivesStolenAssists"],
                                nexusKills=player["nexusKills"],
                                nexusTakedowns=player["nexusTakedowns"],
                                nexusLost=player["nexusLost"],
                            ),
                            economy=EconomyStats(
                                goldEarned=player["goldEarned"],
                                goldSpent=player["goldSpent"],
                                itemsPurchased=player["itemsPurchased"],
                                consumablesPurchased=player["consumablesPurchased"],
                                item0=player["item0"],
                                item1=player["item1"],
                                item2=player["item2"],
                                item3=player["item3"],
                                item4=player["item4"],
                                item5=player["item5"],
                                item6=player["item6"],
                            ),
                            vision=VisionStats(
                                visionScore=player["visionScore"],
                                wardsPlaced=player["wardsPlaced"],
                                wardsKilled=player["wardsKilled"],
                                visionWardsBoughtInGame=player["visionWardsBoughtInGame"],
                            ),
                            abilityUsage=AbilityStats(
                                spell1Casts=player["spell1Casts"],
                                spell2Casts=player["spell2Casts"],
                                spell3Casts=player["spell3Casts"],
                                spell4Casts=player["spell4Casts"],
                                summoner1Id=player["summoner1Id"],
                                summoner2Id=player["summoner2Id"],
                                summoner1Casts=player["summoner1Casts"],
                                summoner2Casts=player["summoner2Casts"],
                                perks=Perks(
                                    statPerks=player["perks"]["statPerks"],
                                    styles=[
                                        PerkStyle(
                                            description=style["description"],
                                            selections=[
                                                PerkSelection(
                                                    perk=selection["perk"],
                                                    var1=selection["var1"],
                                                    var2=selection["var2"],
                                                    var3=selection["var3"],
                                                )
                                                for selection in style["selections"]
                                            ],
                                            style=style["style"],
                                        )
                                        for style in player["perks"]["styles"]
                                    ],
                                ),
                            ),
                            pings=PingStats(
                                allInPings=player["allInPings"],
                                assistMePings=player["assistMePings"],
                                basicPings=player["basicPings"],
                                commandPings=player["commandPings"],
                                dangerPings=player["dangerPings"],
                                getBackPings=player["getBackPings"],
                                holdPings=player["holdPings"],
                                onMyWayPings=player["onMyWayPings"],
                                pushPings=player["pushPings"],
                                retreatPings=player["retreatPings"],
                                needVisionPings=player["needVisionPings"],
                                enemyMissingPings=player["enemyMissingPings"],
                                enemyVisionPings=player["enemyVisionPings"],
                                visionClearedPings=player["visionClearedPings"],
                            ),
                            farming=FarmingStats(
                                totalMinionsKilled=player["totalMinionsKilled"],
                                neutralMinionsKilled=player["neutralMinionsKilled"],
                                totalAllyJungleMinionsKilled=player["totalAllyJungleMinionsKilled"],
                                totalEnemyJungleMinionsKilled=player["totalEnemyJungleMinionsKilled"],
                            ),
                        )
                        for player in data["info"]["participants"]
                    ],
                )
                with open(f"Player_Data/{rank}/data/processed/match_data/{temp_match[:-5]}_processed.json", "w", encoding="utf-8") as out_f:
                    out_f.write(match.model_dump_json(indent=4))
            with open(f"Player_Data/{rank}/data/raw/timeline/{temp_match}", "r", encoding="utf-8") as f:
                data = json.load(f)
                timeline = MatchTimelineData(
                    metadata=TimelineMetadata(
                        dataVersion=data["metadata"]["dataVersion"],
                        matchId=data["metadata"]["matchId"],
                        gameId=data["info"]["gameId"],
                        endOfGameResult=data["info"]["endOfGameResult"],
                        players=[
                            TimelinePlayerData(
                                playerId=player["participantId"],
                                puuid=player["puuid"],
                            )
                            for player in data["info"]["participants"]
                        ],
                    ),
                    frameInterval=data["info"]["frameInterval"],
                    frames=[
                        FrameData(
                            playersFrames=[
                                PlayersFrame(
                                    playerFrames={
                                        player_id: PlayerFrame(
                                            playerId=player_frame["participantId"],
                                            position=Position(
                                                x=player_frame["position"]["x"],
                                                y=player_frame["position"]["y"],
                                            ),
                                            currentGold=player_frame["currentGold"],
                                            totalGold=player_frame["totalGold"],
                                            level=player_frame["level"],
                                            xp=player_frame["xp"],
                                            minionsKilled=player_frame["minionsKilled"],
                                            jungleMinionsKilled=player_frame["jungleMinionsKilled"],
                                            totalDamageDone=player_frame["damageStats"]["totalDamageDone"],
                                            totalDamageDoneToChampions=player_frame["damageStats"]["totalDamageDoneToChampions"],
                                            totalDamageTaken=player_frame["damageStats"]["totalDamageTaken"],
                                            physicalDamageDone=player_frame["damageStats"]["physicalDamageDone"],
                                            physicalDamageDoneToChampions=player_frame["damageStats"]["physicalDamageDoneToChampions"],
                                            physicalDamageTaken=player_frame["damageStats"]["physicalDamageTaken"],
                                            magicDamageDone=player_frame["damageStats"]["magicDamageDone"],
                                            magicDamageDoneToChampions=player_frame["damageStats"]["magicDamageDoneToChampions"],
                                            magicDamageTaken=player_frame["damageStats"]["magicDamageTaken"],
                                            trueDamageDone=player_frame["damageStats"]["trueDamageDone"],
                                            trueDamageDoneToChampions=player_frame["damageStats"]["trueDamageDoneToChampions"],
                                            trueDamageTaken=player_frame["damageStats"]["trueDamageTaken"],
                                        )
                                        for player_id, player_frame in frame["participantFrames"].items()
                                    }
                                )
                            ],
                            events=[ev for event in frame.get("events", []) if (ev := parse_event(event)) is not None],
                        )
                        for frame in data["info"]["frames"]
                    ],
                )
                with open(f"Player_Data/{rank}/data/processed/timeline_data/{temp_match[:-5]}_timeline_processed.json", "w", encoding="utf-8") as out_f:
                    out_f.write(timeline.model_dump_json(indent=4))
        except Exception as e:
            print(f"Error processing match {temp_match}: {e}")
            if os.path.exists(f"Player_Data/{rank}/data/processed/match_data/{temp_match[:-5]}_processed.json"):
                os.remove(f"Player_Data/{rank}/data/processed/match_data/{temp_match[:-5]}_processed.json")
            if os.path.exists(f"Player_Data/{rank}/data/processed/timeline_data/{temp_match[:-5]}_timeline_processed.json"):
                os.remove(f"Player_Data/{rank}/data/processed/timeline_data/{temp_match[:-5]}_timeline_processed.json")


In [ ]:
rank_player_metrics = {}
for rank in player_matches:
    rank_player_metrics[rank] = {}
    for player in player_matches[rank]:
        rank_player_metrics[rank][player] = []
        player_matches_data = player_matches[rank][player]
        for match_id in player_matches_data:
            try:
                with open(f"Player_Data/{rank}/data/processed/match_data/{match_id}_processed.json", "r", encoding="utf-8") as f:
                    match_data_json = json.load(f)
                    match_data = MatchData.model_validate(match_data_json)
                with open(f"Player_Data/{rank}/data/processed/timeline_data/{match_id}_timeline_processed.json", "r", encoding="utf-8") as f:
                    timeline_data_json = json.load(f)
                    timeline_data = MatchTimelineData.model_validate(timeline_data_json)
            except Exception as e:
                print(f"Error processing match {match_id}: {e}")
                continue
            metrics = get_raw_ionia_metrics_for_match(player, match_data, timeline_data)
            rank_player_metrics[rank][player].append(metrics)

In [ ]:
rank_player_metrics_avg = {}
for rank in rank_player_metrics:
    rank_player_metrics_avg[rank] = {}
    for player in rank_player_metrics[rank]:
        metrics_list = rank_player_metrics[rank][player]
        avg_metrics = {
            "combat": {
                "kill_participation": 0,
                "kda": 0,
                "damage_per_min": 0,
                "damage_share": 0,
                "double_kills": 0,
                "triple_kills": 0,
                "quadra_kills": 0,
                "penta_kills": 0,
                "first_blood_kill": 0,
                "first_blood_assist": 0,
            },
            "team_contribution": {
                "kill_participation": 0,
                "heal_shield_per_min": 0,
                "objective_participation_ratio": 0,
                "roaming_ratio": 0,
            },
            "resource_efficiency": {
                "gold_per_min": 0,
                "cs_per_min": 0,
            },
            "vision_awareness": {
                "vision_score_per_min": 0,
            },
        }
        num_matches = len(metrics_list)
        for metrics in metrics_list:
            for category in avg_metrics:
                for metric in avg_metrics[category]:
                    avg_metrics[category][metric] += metrics[f"{category}_metrics"][metric]
        for category in avg_metrics:
            for metric in avg_metrics[category]:
                avg_metrics[category][metric] /= max(1, num_matches)
        rank_player_metrics_avg[rank][player] = avg_metrics

In [ ]:
with open("rank_player_metrics_avg.json", "w", encoding="utf-8") as f:
    json.dump(rank_player_metrics_avg, f, indent=4)

In [ ]:
stat_baselines_before_consistency = {
    "combat": {
        "kill_participation": [],
        "kda": [],
        "damage_per_min": [],
        "damage_share": [],
        "double_kills": [],
        "triple_kills": [],
        "quadra_kills": [],
        "penta_kills": [],
        "first_blood_kill": [],
        "first_blood_assist": [],
    },
    "team_contribution": {
        "kill_participation": [],
        "heal_shield_per_min": [],
        "objective_participation_ratio": [],
        "roaming_ratio": [],
    },
    "resource_efficiency": {
        "gold_per_min": [],
        "cs_per_min": [],
    },
    "vision_awareness": {
        "vision_score_per_min": [],
    },
}
for rank in rank_player_metrics_avg:
    for player in rank_player_metrics_avg[rank]:
        avg_metrics = rank_player_metrics_avg[rank][player]
        for category in avg_metrics:
            for metric in avg_metrics[category]:
                stat_baselines_before_consistency[category][metric].append(avg_metrics[category][metric])
for category in stat_baselines_before_consistency:
    for metric in stat_baselines_before_consistency[category]:
        stat_baselines_before_consistency[category][metric] = sorted(stat_baselines_before_consistency[category][metric])

Calculate consistency based on the other four category metric scores

In [ ]:
rank_player_metrics_consistency = {}


def normalize_raw_metric_before_consistency(value, metric_name, category):
    sample_array = stat_baselines_before_consistency[category][metric_name]
    return ranked_cdf_score(value, sample_array)


for rank in rank_player_metrics:
    rank_player_metrics_consistency[rank] = {}
    for player in rank_player_metrics[rank]:
        metrics_list = rank_player_metrics[rank][player]
        combat_scores = []
        team_contribution_scores = []
        resource_efficiency_scores = []
        vision_awareness_scores = []
        for m in metrics_list:
            combat_scores.append(sum([normalize_raw_metric_before_consistency(m["combat_metrics"][metric], metric, "combat") * weight for metric, weight in combat_weighting.items()]))
            team_contribution_scores.append(
                sum([normalize_raw_metric_before_consistency(m["team_contribution_metrics"][metric], metric, "team_contribution") * weight for metric, weight in team_contribution_weighting.items()])
            )
            resource_efficiency_scores.append(
                sum(
                    [
                        normalize_raw_metric_before_consistency(m["resource_efficiency_metrics"][metric], metric, "resource_efficiency") * weight
                        for metric, weight in resource_efficiency_weighting.items()
                    ]
                )
            )
            vision_awareness_scores.append(
                sum([normalize_raw_metric_before_consistency(m["vision_awareness_metrics"][metric], metric, "vision_awareness") * weight for metric, weight in vision_awareness_weighting.items()])
            )

        statistics = {
            "combat": {
                "avg": np.mean(combat_scores).item(),
                "std": np.std(combat_scores).item(),
            },
            "team_contribution": {
                "avg": np.mean(team_contribution_scores).item(),
                "std": np.std(team_contribution_scores).item(),
            },
            "resource_efficiency": {
                "avg": np.mean(resource_efficiency_scores).item(),
                "std": np.std(resource_efficiency_scores).item(),
            },
            "vision_awareness": {
                "avg": np.mean(vision_awareness_scores).item(),
                "std": np.std(vision_awareness_scores).item(),
            },
        }
        rank_player_metrics_consistency[rank][player] = {
            "combat": consistency_score_logistic(statistics["combat"]["avg"], statistics["combat"]["std"]),
            "team_contribution": consistency_score_logistic(statistics["team_contribution"]["avg"], statistics["team_contribution"]["std"]),
            "resource_efficiency": consistency_score_logistic(statistics["resource_efficiency"]["avg"], statistics["resource_efficiency"]["std"]),
            "vision_awareness": consistency_score_logistic(statistics["vision_awareness"]["avg"], statistics["vision_awareness"]["std"]),
        }

In [ ]:
with open("rank_player_metrics_consistency.json", "w", encoding="utf-8") as f:
    json.dump(rank_player_metrics_consistency, f, indent=4)

In [ ]:
stat_baselines = {
    "combat": {
        "kill_participation": stat_baselines_before_consistency["combat"]["kill_participation"],
        "kda": stat_baselines_before_consistency["combat"]["kda"],
        "damage_per_min": stat_baselines_before_consistency["combat"]["damage_per_min"],
        "damage_share": stat_baselines_before_consistency["combat"]["damage_share"],
        "double_kills": stat_baselines_before_consistency["combat"]["double_kills"],
        "triple_kills": stat_baselines_before_consistency["combat"]["triple_kills"],
        "quadra_kills": stat_baselines_before_consistency["combat"]["quadra_kills"],
        "penta_kills": stat_baselines_before_consistency["combat"]["penta_kills"],
        "first_blood_kill": stat_baselines_before_consistency["combat"]["first_blood_kill"],
        "first_blood_assist": stat_baselines_before_consistency["combat"]["first_blood_assist"],
    },
    "team_contribution": {
        "kill_participation": stat_baselines_before_consistency["team_contribution"]["kill_participation"],
        "heal_shield_per_min": stat_baselines_before_consistency["team_contribution"]["heal_shield_per_min"],
        "objective_participation_ratio": stat_baselines_before_consistency["team_contribution"]["objective_participation_ratio"],
        "roaming_ratio": stat_baselines_before_consistency["team_contribution"]["roaming_ratio"],
    },
    "resource_efficiency": {
        "gold_per_min": stat_baselines_before_consistency["resource_efficiency"]["gold_per_min"],
        "cs_per_min": stat_baselines_before_consistency["resource_efficiency"]["cs_per_min"],
    },
    "vision_awareness": {
        "vision_score_per_min": stat_baselines_before_consistency["vision_awareness"]["vision_score_per_min"],
    },
    "consistency": {
        "combat": sorted([rank_player_metrics_consistency[rank][player]["combat"] for rank in rank_player_metrics_consistency for player in rank_player_metrics_consistency[rank]]),
        "team_contribution": sorted([rank_player_metrics_consistency[rank][player]["team_contribution"] for rank in rank_player_metrics_consistency for player in rank_player_metrics_consistency[rank]]),
        "resource_efficiency": sorted([rank_player_metrics_consistency[rank][player]["resource_efficiency"] for rank in rank_player_metrics_consistency for player in rank_player_metrics_consistency[rank]]),
        "vision_awareness": sorted([rank_player_metrics_consistency[rank][player]["vision_awareness"] for rank in rank_player_metrics_consistency for player in rank_player_metrics_consistency[rank]]),
    }
}

In [ ]:
with open("stat_baselines.json", "w", encoding="utf-8") as f:
    json.dump(stat_baselines, f, indent=4)

#### Baselines

In [ ]:
sample_baselines = {
    "combat": {
        "kill_participation": [
            0.1969486093279247,
            0.2774818085123838,
            0.2995440123294645,
            0.313436843501546,
            0.31845280042851964,
            0.318607839571442,
            0.3224990218830479,
            0.3282838994551443,
            0.3353577218289778,
            0.337158172229984,
            0.34471717903981447,
            0.3469265426359487,
            0.35064364284046595,
            0.36462484240538134,
            0.3669992757382211,
            0.3672842805034401,
            0.36874738746192787,
            0.369913845376115,
            0.3701852225968662,
            0.3739250791972593,
            0.37753574028749065,
            0.379615651413058,
            0.38079677897195535,
            0.38123298242114084,
            0.3818659755056755,
            0.3833264196950938,
            0.38379590267204866,
            0.3874737929149619,
            0.3877575436806211,
            0.39074669410446833,
            0.39208716031178353,
            0.3944727849477151,
            0.4070100712904291,
            0.40779311681429803,
            0.4092586917963396,
            0.40987188145900927,
            0.4109741343458643,
            0.4114519083736396,
            0.4127790025058854,
            0.4129039628330616,
            0.4164643885877991,
            0.4173571207506232,
            0.4183814771242191,
            0.4202419954442069,
            0.42154715394657527,
            0.4217955838130217,
            0.4222257947234955,
            0.42225224515460114,
            0.42407359089741964,
            0.4241144963273898,
            0.4241566056279524,
            0.42547444120429134,
            0.4256022104261528,
            0.42652668379377817,
            0.4265270889833183,
            0.4299327761070992,
            0.4320259160583636,
            0.43237846852035317,
            0.43859800542268484,
            0.4395517422349982,
            0.4409859106086401,
            0.4415536764844707,
            0.442929722692382,
            0.4430680754080809,
            0.44328515020249776,
            0.44423516285811615,
            0.4449708601422407,
            0.4453310202947719,
            0.4467908984375721,
            0.44770679181247336,
            0.44851681296150947,
            0.4493253996694262,
            0.4496307768964231,
            0.44994899372585817,
            0.4503377788378216,
            0.45220809529869194,
            0.45269926297210233,
            0.4538583207722677,
            0.4551200145970929,
            0.4586094595398254,
            0.45968361152306597,
            0.4628015663175343,
            0.4628250839788912,
            0.46411154308024094,
            0.46490386645691084,
            0.4652685691525765,
            0.4652921610506836,
            0.46529333847783666,
            0.4659669781123333,
            0.46713308782652374,
            0.4680908388426006,
            0.46854914540367004,
            0.46875385637311123,
            0.4695519130962913,
            0.4698912278567893,
            0.47125542415005645,
            0.47141955715956846,
            0.4716348117143056,
            0.47286792951179174,
            0.47353902649172447,
            0.47364569679058427,
            0.47452941922812963,
            0.4757335508341053,
            0.4758714099225129,
            0.47619983694767815,
            0.4769421242548153,
            0.47696299831086864,
            0.47846233666565535,
            0.4786284826804351,
            0.48001030796341976,
            0.4821183518004515,
            0.4822071564193452,
            0.48264646112255954,
            0.4827835363157473,
            0.4830093391939423,
            0.4838853554943573,
            0.48517833184729275,
            0.4861403142788382,
            0.4861872048948692,
            0.4871654954518732,
            0.48795867968171674,
            0.48818777383174783,
            0.4902527558377103,
            0.4905651276341469,
            0.49173541404742843,
            0.4923902312834149,
            0.4934145678250712,
            0.49748399775918245,
            0.49872771668230076,
            0.49880693259024034,
            0.49893818033410287,
            0.49904277937416874,
            0.5010459759955294,
            0.5020678618082985,
            0.5024841569145648,
            0.5037717095719418,
            0.5044136819866721,
            0.504938391968382,
            0.5051664044915054,
            0.5058363530455335,
            0.5066370939534741,
            0.5080374455184626,
            0.5105861541735294,
            0.511859184343089,
            0.5121321638792756,
            0.5127190102888439,
            0.5138364188705655,
            0.5147291677866355,
            0.5151545394341389,
            0.5153362564378896,
            0.5179351980453744,
            0.5180353681203979,
            0.5191365808616013,
            0.519186798092373,
            0.5212264800264292,
            0.5223034114436822,
            0.5224082626758236,
            0.5225399526047981,
            0.5227867296762445,
            0.5234090735685246,
            0.5238117955984413,
            0.5321356183012447,
            0.5327099581985522,
            0.5329686413986103,
            0.5346051148040579,
            0.5353602114100435,
            0.5359845508739599,
            0.5360807026486999,
            0.5376142148935256,
            0.5385891025613145,
            0.5392783224004631,
            0.5405542113001618,
            0.5406087499901839,
            0.5426268830512584,
            0.545239707421885,
            0.5454642353500643,
            0.5514904970106834,
            0.5561045271664451,
            0.5583672810024487,
            0.5586810515507722,
            0.55986744148977,
            0.5637093311674242,
            0.5691020245286549,
            0.5697841535079022,
            0.578356938079353,
            0.5797906095865645,
            0.5851916915490631,
            0.5870126402876494,
            0.5917713429592778,
            0.5956706229116242,
            0.5958753971016034,
            0.5959720965859159,
            0.6010231479231888,
            0.6027680184116844,
            0.6097856554015606,
            0.6098133628804887,
            0.6203768270789904,
            0.6304399000481169,
            0.6370785422795638,
            0.6672843664190412,
        ],
        "kda": [
            1.116474405224405,
            1.2067063492063497,
            1.4732738095238096,
            1.6212021774521772,
            1.672120934620935,
            1.7789069264069266,
            1.7949486624486626,
            1.813912661412661,
            1.8610582010582013,
            1.9314694564694566,
            1.9379059502588913,
            1.9515239390239387,
            1.9861483886483886,
            2.066147186147186,
            2.075655270655271,
            2.0968403818403822,
            2.129119954119954,
            2.1467112299465243,
            2.1860124135124135,
            2.1958216783216784,
            2.2086219336219335,
            2.2309804084804084,
            2.253172013172013,
            2.3155435305435303,
            2.324722684722685,
            2.3370278795278794,
            2.3632725607725606,
            2.3634523809523813,
            2.3758802308802305,
            2.398143153143154,
            2.413015873015873,
            2.4355321530321534,
            2.4364682539682536,
            2.4516666666666667,
            2.463231120731121,
            2.4654037629037626,
            2.4760449735449734,
            2.5133730158730163,
            2.5185714285714287,
            2.5398075998076,
            2.5755026455026453,
            2.5761255411255415,
            2.579663147310206,
            2.5817243867243866,
            2.614300051800052,
            2.615026455026455,
            2.6377592359850426,
            2.663504088504089,
            2.6644961519961523,
            2.66462999963,
            2.674616402116402,
            2.6959656084656087,
            2.722498797498798,
            2.7391798941798937,
            2.744699097199097,
            2.7515923890923886,
            2.7535521885521885,
            2.7590841103341104,
            2.770383597883598,
            2.8169802419802417,
            2.8374494949494946,
            2.8434920634920635,
            2.8490719465719465,
            2.8677879527879524,
            2.8814033189033177,
            2.8949677174677175,
            2.8978943278943277,
            2.919153439153439,
            2.9220965962142436,
            2.9248148148148148,
            2.940557960557961,
            2.942993025493026,
            2.951907444407445,
            2.962430254930255,
            2.976508769008769,
            2.9917124542124536,
            2.9921031746031743,
            3.0135487660487654,
            3.0314057239057237,
            3.0317520442520443,
            3.057757936507936,
            3.0657814407814405,
            3.0951190476190478,
            3.1130844155844164,
            3.1191534391534392,
            3.119386724386724,
            3.1694943019943023,
            3.1796825396825397,
            3.1945726495726494,
            3.233661616161616,
            3.2432010582010586,
            3.2476462426462427,
            3.269154641654642,
            3.281760461760461,
            3.2938064713064708,
            3.294060846560846,
            3.349835257335257,
            3.3561519961519966,
            3.3722619047619053,
            3.3834319384319396,
            3.4108862433862437,
            3.4244047619047615,
            3.4359096459096463,
            3.439488936988937,
            3.4496174196174194,
            3.4672582972582964,
            3.4707804232804236,
            3.4741161616161613,
            3.495656565656566,
            3.498347763347764,
            3.5175264550264544,
            3.5198653198653194,
            3.5383980833980844,
            3.5391089466089474,
            3.611031746031746,
            3.622705535205535,
            3.633455988455989,
            3.637129629629629,
            3.6383730158730163,
            3.645927313427314,
            3.651547619047619,
            3.6661279461279457,
            3.721812169312169,
            3.730202939761763,
            3.7379232804232805,
            3.756111111111111,
            3.813528601028601,
            3.8252008177008174,
            3.860925925925926,
            3.8895238095238094,
            3.9607912457912464,
            3.9923821548821556,
            4.0071031746031744,
            4.030351222851222,
            4.039708994708995,
            4.07279341029341,
            4.078017075517075,
            4.13531746031746,
            4.139434269434269,
            4.1471693121693125,
            4.148253968253969,
            4.162288359788358,
            4.1754232804232805,
            4.176825396825397,
            4.178887686387686,
            4.280396825396825,
            4.287034632034632,
            4.30011544011544,
            4.306292226292226,
            4.315018037518038,
            4.3289935064935054,
            4.365158730158731,
            4.393278388278388,
            4.406178821178821,
            4.471136086136085,
            4.477777777777778,
            4.489822862322862,
            4.554098586598587,
            4.579841269841269,
            4.634660894660895,
            4.646521164021164,
            4.6839814814814815,
            4.6860185185185195,
            4.731685906685907,
            4.78024531024531,
            4.855542328042329,
            4.864814814814816,
            4.87838864838865,
            4.911115921115921,
            4.919467291967292,
            4.967020757020757,
            5.095753968253969,
            5.104931457431458,
            5.109576719576719,
            5.186507936507937,
            5.210714285714285,
            5.21111111111111,
            5.223201058201059,
            5.255467772967772,
            5.488280423280424,
            5.516468253968255,
            5.522089947089948,
            5.561468253968253,
            5.617592592592592,
            5.641881914381914,
            5.698888888888889,
            5.9935870610870605,
            6.008161375661376,
            6.055436507936507,
            6.464550264550266,
            6.740648148148148,
            6.990313852813852,
            7.0039417989418,
            7.0490079365079366,
            7.118452380952379,
            7.5054232804232806,
            7.578121693121692,
            7.633333333333334,
            7.656190476190476,
            10.987275132275133,
        ],
        "damage_per_min": [
            314.25131210296405,
            325.1157293730051,
            338.50555231819504,
            355.2328879754954,
            358.63682928277984,
            368.72053531903634,
            380.3011892757416,
            397.5000652267721,
            407.22889058186485,
            412.61725506298626,
            417.9681609622491,
            420.55095036027325,
            425.25556086862616,
            429.9861361836285,
            435.6731855005027,
            441.1501653326498,
            445.60417493910205,
            446.0852831926611,
            446.86021750999873,
            471.6820005244304,
            475.1806733666363,
            475.1864808621101,
            483.5930260955392,
            483.8894924344406,
            488.7929957998639,
            489.4726766156655,
            492.1286511222996,
            497.84104624665355,
            498.41027788283407,
            503.9447319557255,
            509.4798799035515,
            523.0096888506441,
            523.8500471567028,
            529.1797406188028,
            538.6639346584675,
            543.0264927091492,
            543.406125965719,
            543.5857393441904,
            547.2144683554761,
            549.4590613081782,
            562.4669125295807,
            564.1931876765683,
            566.9865595012548,
            570.1803381484118,
            578.2284660811082,
            578.2528114818675,
            582.6350810701404,
            586.845067552862,
            588.4344232296335,
            594.8980862940299,
            596.1785428710722,
            597.2912271358717,
            603.5397853454615,
            604.9352310542475,
            605.4696837466225,
            607.1917382514941,
            609.8155873577965,
            610.2376406433635,
            611.5519919785842,
            621.6459399898001,
            627.8471819408701,
            630.1981601293447,
            630.40729985104,
            632.5060577764156,
            632.6524213340343,
            633.3520818089049,
            633.7939354152993,
            635.0433064376925,
            636.5894871311923,
            637.4453360923084,
            640.7889347229865,
            640.872739966233,
            641.5432788613014,
            643.5141353240391,
            648.2087820977026,
            648.4914188780548,
            651.2690374113267,
            657.5938122137889,
            658.1341045474409,
            660.4459198405608,
            663.1344956102444,
            668.1321688651022,
            670.6862185437616,
            672.6602338609039,
            673.520128847659,
            673.9435688108787,
            679.7842699065353,
            679.8501479277422,
            681.1733104780398,
            681.75563500284,
            682.0335897432843,
            682.5679571570905,
            685.6392503663046,
            691.3673691757939,
            697.0137795175139,
            698.1740625294414,
            702.1095370195343,
            703.7684984138021,
            703.8076839695525,
            704.6863810089396,
            708.7406284461881,
            709.042650980601,
            713.4598527991883,
            719.9423453025291,
            720.4874462077988,
            722.4487609547784,
            723.756076987812,
            726.1477154998413,
            729.0586662089448,
            732.2305057864589,
            733.3644008100077,
            738.8880774559323,
            739.5654388143374,
            740.1088318630605,
            740.4184384941738,
            743.0865940970033,
            746.251842752063,
            751.7261025086099,
            753.5062921803417,
            753.6992966432149,
            762.6942302928727,
            763.0048974476792,
            763.5919001188938,
            763.8740338406063,
            765.5056744748385,
            766.8213452387096,
            770.4308704325699,
            773.752427488832,
            776.1036737016233,
            778.0663589490862,
            779.4410996167605,
            779.78484422402,
            783.2300234770217,
            784.3081178630532,
            785.2672146987377,
            787.2332902173658,
            788.6510007740261,
            794.6210217264071,
            796.2716681545779,
            806.8435539430349,
            807.0356175203437,
            809.9084880414016,
            810.7556358524292,
            812.7919914928008,
            813.7176290752252,
            818.7730134593799,
            818.7890110619121,
            824.7531040170672,
            829.6599948883523,
            831.9167826204044,
            836.4301197265453,
            839.639864376082,
            842.7971352939203,
            843.4804073085802,
            844.810758683438,
            848.8896298238828,
            853.5841410749533,
            854.015072900111,
            855.4270123563896,
            856.8406665699215,
            860.7216821475346,
            862.6669772485566,
            862.9354615278044,
            868.7398175100884,
            869.5804550073155,
            873.844968773912,
            875.1535910618821,
            875.2511772242228,
            880.4051183499481,
            881.3223409009665,
            883.3110151379457,
            883.7998170493316,
            886.078896310347,
            886.9641998558102,
            887.6425494429762,
            891.6599847318336,
            907.4905641515143,
            908.1741036015318,
            910.0661862840597,
            912.2612587191056,
            917.4464498440858,
            920.1012452671898,
            922.0447423355262,
            927.3869686540025,
            932.0413933976372,
            939.4522636086651,
            941.3138030002016,
            952.3265524620841,
            962.9711550213493,
            985.0977886406642,
            1012.6219961776775,
            1025.3655390258857,
            1035.0093257933347,
            1037.8564146404317,
            1042.2719637713603,
            1047.543590008384,
            1048.1004918419926,
            1067.6024979961373,
            1116.9344271262833,
            1176.8312807396353,
        ],
        "damage_share": [
            0.09693561731484247,
            0.09941490445417095,
            0.10538178660767436,
            0.1091224587760094,
            0.11008736734126981,
            0.11067358761871278,
            0.11516891849827281,
            0.11917924080162341,
            0.12025201967699534,
            0.12095927019328899,
            0.12259704161316898,
            0.12449024613503495,
            0.13439793168223937,
            0.1355566354955141,
            0.1386642818039115,
            0.1410462593320765,
            0.14171523027553415,
            0.1419381807198609,
            0.1422212632136496,
            0.1442261093221988,
            0.14453586576374688,
            0.14686902926232018,
            0.14971802226469497,
            0.15021409371495834,
            0.1503380877188077,
            0.1508683164428163,
            0.15220553053735353,
            0.15438756597777176,
            0.15464896868507946,
            0.15557086220499017,
            0.15566853833642577,
            0.15896969153075835,
            0.15977641006106272,
            0.16082348238207533,
            0.16091519142100816,
            0.16148978421984006,
            0.16378099176949007,
            0.1645819882127718,
            0.16590461774115287,
            0.16793645231638282,
            0.16793658800224298,
            0.16856535247801485,
            0.16903096894623115,
            0.16918137049350473,
            0.16980303801093294,
            0.1718089777313194,
            0.17265045656025613,
            0.17288352395464823,
            0.17440201507576553,
            0.1749015971877511,
            0.17637083297054093,
            0.17657445563496682,
            0.17803588521707966,
            0.1803239256872014,
            0.18055829390428976,
            0.18173383284184966,
            0.18309319667567603,
            0.18378613976090852,
            0.18455473795717972,
            0.18665218882063817,
            0.18688763548054405,
            0.18765676589531238,
            0.18850906325798827,
            0.18872963799464407,
            0.1898003396397912,
            0.1907501383208915,
            0.19199511213518586,
            0.19318200102883512,
            0.19326594025145677,
            0.19347682584447826,
            0.19412556240695025,
            0.19459400873257113,
            0.19470915632397245,
            0.1955580616682832,
            0.19706724825004363,
            0.197131157069247,
            0.1975962819693822,
            0.19789683985468196,
            0.19836570921108418,
            0.1987508944832125,
            0.19898673390179508,
            0.2000312057633365,
            0.20028816947516545,
            0.2004459927388637,
            0.20106777347662516,
            0.20244502317241367,
            0.20262034924792957,
            0.20267917446869138,
            0.2033021875053379,
            0.20362591718946393,
            0.20533313046018126,
            0.20764193149182594,
            0.2082768642776658,
            0.20930738465155196,
            0.2093302701921507,
            0.20934596716443557,
            0.21005203255036134,
            0.21115595664222067,
            0.2112130244512895,
            0.21133618425539932,
            0.2116222582900273,
            0.21203436821711483,
            0.21240518253860127,
            0.2131677975418575,
            0.2136059210288804,
            0.2137478846821751,
            0.2144222331613897,
            0.21454503431729147,
            0.21485311420522601,
            0.2150341273124459,
            0.2167722965580477,
            0.21677600195080673,
            0.2177311600264744,
            0.21811246824166214,
            0.21910025198297808,
            0.22082850280063926,
            0.2209959702400087,
            0.2230076117517844,
            0.22333979167309667,
            0.22336268361952624,
            0.22351649660115622,
            0.22394303785977868,
            0.22525179071572296,
            0.22572547501117582,
            0.22601227556053302,
            0.2266808934868282,
            0.22830117699303118,
            0.22835086841411856,
            0.22835515300637052,
            0.22874294505933862,
            0.22875182874775646,
            0.22946400527993305,
            0.23002350804864072,
            0.23041828789065638,
            0.23069929158799918,
            0.23148577164085327,
            0.23256594728534744,
            0.23291279095191172,
            0.2332960159952524,
            0.2335857750276289,
            0.2337947666358968,
            0.23418205054158028,
            0.23446982179172304,
            0.23741501248565305,
            0.23818302360530613,
            0.23838746770048763,
            0.23860661190324337,
            0.23864671508539675,
            0.23945009075310292,
            0.2403136991426879,
            0.24121641715156847,
            0.24168344563127322,
            0.2418347961568583,
            0.24254394925399494,
            0.24421356350738602,
            0.24437029565817758,
            0.24455283949543588,
            0.2450224518029194,
            0.24575010116467816,
            0.24724542995411294,
            0.2475948471006496,
            0.24793275636403456,
            0.24821952829885827,
            0.248693438724717,
            0.2494662450448164,
            0.25023717716068394,
            0.25038880740358427,
            0.25061885195063266,
            0.2555936051553696,
            0.2571674359030818,
            0.25792356599569455,
            0.25962759798064794,
            0.2607446254228296,
            0.260870158842951,
            0.2615913022427301,
            0.2622712985936522,
            0.26233647226318585,
            0.2631011706939803,
            0.2641793379414555,
            0.265248818184261,
            0.26606576091055406,
            0.2665132134424905,
            0.26743491642868283,
            0.2699071407967694,
            0.27173480737877326,
            0.2721150608985928,
            0.2731716800408532,
            0.27377887256187583,
            0.2748417140838812,
            0.27522095876358926,
            0.27856231779127566,
            0.28057372104028444,
            0.28165934617580474,
            0.28773322961148406,
            0.29046386231212457,
            0.29387945133279947,
            0.29633187009788925,
            0.29750025517832396,
            0.2976475712028788,
            0.3135800061554284,
        ],
        "double_kills": [
            0.03333333333333333,
            0.03333333333333333,
            0.06666666666666667,
            0.06666666666666667,
            0.1,
            0.1,
            0.1,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.2,
            0.2,
            0.2,
            0.2,
            0.2,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.26666666666666666,
            0.26666666666666666,
            0.26666666666666666,
            0.26666666666666666,
            0.26666666666666666,
            0.3,
            0.3,
            0.3,
            0.3,
            0.3,
            0.3,
            0.3,
            0.3,
            0.3,
            0.3,
            0.3,
            0.3,
            0.3,
            0.3,
            0.3333333333333333,
            0.3333333333333333,
            0.3333333333333333,
            0.3333333333333333,
            0.36666666666666664,
            0.36666666666666664,
            0.36666666666666664,
            0.36666666666666664,
            0.36666666666666664,
            0.36666666666666664,
            0.36666666666666664,
            0.36666666666666664,
            0.4,
            0.4,
            0.4,
            0.4,
            0.4,
            0.4,
            0.4,
            0.4,
            0.4,
            0.43333333333333335,
            0.43333333333333335,
            0.43333333333333335,
            0.43333333333333335,
            0.43333333333333335,
            0.43333333333333335,
            0.43333333333333335,
            0.4666666666666667,
            0.4666666666666667,
            0.4666666666666667,
            0.4666666666666667,
            0.4666666666666667,
            0.4666666666666667,
            0.4666666666666667,
            0.5,
            0.5,
            0.5,
            0.5,
            0.5,
            0.5,
            0.5,
            0.5,
            0.5,
            0.5333333333333333,
            0.5333333333333333,
            0.5333333333333333,
            0.5333333333333333,
            0.5333333333333333,
            0.5666666666666667,
            0.5666666666666667,
            0.5666666666666667,
            0.5666666666666667,
            0.5666666666666667,
            0.5666666666666667,
            0.5666666666666667,
            0.5666666666666667,
            0.5666666666666667,
            0.6,
            0.6,
            0.6,
            0.6,
            0.6,
            0.6,
            0.6,
            0.6,
            0.6333333333333333,
            0.6333333333333333,
            0.6333333333333333,
            0.6333333333333333,
            0.6333333333333333,
            0.6333333333333333,
            0.6333333333333333,
            0.6666666666666666,
            0.6666666666666666,
            0.6666666666666666,
            0.6666666666666666,
            0.6666666666666666,
            0.6666666666666666,
            0.7,
            0.7,
            0.7,
            0.7,
            0.7,
            0.7333333333333333,
            0.7333333333333333,
            0.7333333333333333,
            0.7333333333333333,
            0.7333333333333333,
            0.7333333333333333,
            0.7666666666666667,
            0.7666666666666667,
            0.7666666666666667,
            0.7666666666666667,
            0.7666666666666667,
            0.7666666666666667,
            0.8,
            0.8,
            0.8,
            0.8,
            0.8,
            0.8,
            0.8333333333333334,
            0.8333333333333334,
            0.8333333333333334,
            0.8333333333333334,
            0.8333333333333334,
            0.8666666666666667,
            0.8666666666666667,
            0.8666666666666667,
            0.8666666666666667,
            0.9,
            0.9,
            0.9333333333333333,
            0.9333333333333333,
            0.9333333333333333,
            0.9333333333333333,
            0.9666666666666667,
            0.9666666666666667,
            0.9666666666666667,
            1.0,
            1.0,
            1.0,
            1.0,
            1.0333333333333334,
            1.0333333333333334,
            1.0666666666666667,
            1.0666666666666667,
            1.0666666666666667,
            1.0666666666666667,
            1.0666666666666667,
            1.1,
            1.1,
            1.2,
            1.2,
            1.2333333333333334,
            1.2333333333333334,
            1.2666666666666666,
            1.3333333333333333,
            1.3666666666666667,
            1.4333333333333333,
            1.5,
            1.5,
            1.5,
            1.5666666666666667,
            1.6,
            1.6,
            1.9333333333333333,
        ],
        "triple_kills": [
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.2,
            0.2,
            0.2,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.26666666666666666,
            0.26666666666666666,
            0.26666666666666666,
            0.26666666666666666,
            0.26666666666666666,
            0.3,
            0.3,
            0.3,
            0.3333333333333333,
            0.3333333333333333,
            0.36666666666666664,
            0.36666666666666664,
            0.36666666666666664,
        ],
        "quadra_kills": [
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.1,
            0.1,
            0.1,
        ],
        "penta_kills": [
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.1,
        ],
        "first_blood_kill": [
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.2,
            0.2,
            0.2,
            0.2,
            0.2,
            0.2,
            0.2,
            0.2,
            0.2,
            0.2,
            0.2,
            0.2,
            0.2,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.26666666666666666,
            0.26666666666666666,
            0.26666666666666666,
            0.26666666666666666,
            0.26666666666666666,
            0.26666666666666666,
            0.3,
            0.3333333333333333,
            0.4,
        ],
        "first_blood_assist": [
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.03333333333333333,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.06666666666666667,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.1,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.13333333333333333,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.16666666666666666,
            0.2,
            0.2,
            0.2,
            0.2,
            0.2,
            0.2,
            0.2,
            0.2,
            0.2,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.23333333333333334,
            0.36666666666666664,
        ],
    },
    "team_contribution": {
        "kill_participation": [
            0.1969486093279247,
            0.2774818085123838,
            0.2995440123294645,
            0.313436843501546,
            0.31845280042851964,
            0.318607839571442,
            0.3224990218830479,
            0.3282838994551443,
            0.3353577218289778,
            0.337158172229984,
            0.34471717903981447,
            0.3469265426359487,
            0.35064364284046595,
            0.36462484240538134,
            0.3669992757382211,
            0.3672842805034401,
            0.36874738746192787,
            0.369913845376115,
            0.3701852225968662,
            0.3739250791972593,
            0.37753574028749065,
            0.379615651413058,
            0.38079677897195535,
            0.38123298242114084,
            0.3818659755056755,
            0.3833264196950938,
            0.38379590267204866,
            0.3874737929149619,
            0.3877575436806211,
            0.39074669410446833,
            0.39208716031178353,
            0.3944727849477151,
            0.4070100712904291,
            0.40779311681429803,
            0.4092586917963396,
            0.40987188145900927,
            0.4109741343458643,
            0.4114519083736396,
            0.4127790025058854,
            0.4129039628330616,
            0.4164643885877991,
            0.4173571207506232,
            0.4183814771242191,
            0.4202419954442069,
            0.42154715394657527,
            0.4217955838130217,
            0.4222257947234955,
            0.42225224515460114,
            0.42407359089741964,
            0.4241144963273898,
            0.4241566056279524,
            0.42547444120429134,
            0.4256022104261528,
            0.42652668379377817,
            0.4265270889833183,
            0.4299327761070992,
            0.4320259160583636,
            0.43237846852035317,
            0.43859800542268484,
            0.4395517422349982,
            0.4409859106086401,
            0.4415536764844707,
            0.442929722692382,
            0.4430680754080809,
            0.44328515020249776,
            0.44423516285811615,
            0.4449708601422407,
            0.4453310202947719,
            0.4467908984375721,
            0.44770679181247336,
            0.44851681296150947,
            0.4493253996694262,
            0.4496307768964231,
            0.44994899372585817,
            0.4503377788378216,
            0.45220809529869194,
            0.45269926297210233,
            0.4538583207722677,
            0.4551200145970929,
            0.4586094595398254,
            0.45968361152306597,
            0.4628015663175343,
            0.4628250839788912,
            0.46411154308024094,
            0.46490386645691084,
            0.4652685691525765,
            0.4652921610506836,
            0.46529333847783666,
            0.4659669781123333,
            0.46713308782652374,
            0.4680908388426006,
            0.46854914540367004,
            0.46875385637311123,
            0.4695519130962913,
            0.4698912278567893,
            0.47125542415005645,
            0.47141955715956846,
            0.4716348117143056,
            0.47286792951179174,
            0.47353902649172447,
            0.47364569679058427,
            0.47452941922812963,
            0.4757335508341053,
            0.4758714099225129,
            0.47619983694767815,
            0.4769421242548153,
            0.47696299831086864,
            0.47846233666565535,
            0.4786284826804351,
            0.48001030796341976,
            0.4821183518004515,
            0.4822071564193452,
            0.48264646112255954,
            0.4827835363157473,
            0.4830093391939423,
            0.4838853554943573,
            0.48517833184729275,
            0.4861403142788382,
            0.4861872048948692,
            0.4871654954518732,
            0.48795867968171674,
            0.48818777383174783,
            0.4902527558377103,
            0.4905651276341469,
            0.49173541404742843,
            0.4923902312834149,
            0.4934145678250712,
            0.49748399775918245,
            0.49872771668230076,
            0.49880693259024034,
            0.49893818033410287,
            0.49904277937416874,
            0.5010459759955294,
            0.5020678618082985,
            0.5024841569145648,
            0.5037717095719418,
            0.5044136819866721,
            0.504938391968382,
            0.5051664044915054,
            0.5058363530455335,
            0.5066370939534741,
            0.5080374455184626,
            0.5105861541735294,
            0.511859184343089,
            0.5121321638792756,
            0.5127190102888439,
            0.5138364188705655,
            0.5147291677866355,
            0.5151545394341389,
            0.5153362564378896,
            0.5179351980453744,
            0.5180353681203979,
            0.5191365808616013,
            0.519186798092373,
            0.5212264800264292,
            0.5223034114436822,
            0.5224082626758236,
            0.5225399526047981,
            0.5227867296762445,
            0.5234090735685246,
            0.5238117955984413,
            0.5321356183012447,
            0.5327099581985522,
            0.5329686413986103,
            0.5346051148040579,
            0.5353602114100435,
            0.5359845508739599,
            0.5360807026486999,
            0.5376142148935256,
            0.5385891025613145,
            0.5392783224004631,
            0.5405542113001618,
            0.5406087499901839,
            0.5426268830512584,
            0.545239707421885,
            0.5454642353500643,
            0.5514904970106834,
            0.5561045271664451,
            0.5583672810024487,
            0.5586810515507722,
            0.55986744148977,
            0.5637093311674242,
            0.5691020245286549,
            0.5697841535079022,
            0.578356938079353,
            0.5797906095865645,
            0.5851916915490631,
            0.5870126402876494,
            0.5917713429592778,
            0.5956706229116242,
            0.5958753971016034,
            0.5959720965859159,
            0.6010231479231888,
            0.6027680184116844,
            0.6097856554015606,
            0.6098133628804887,
            0.6203768270789904,
            0.6304399000481169,
            0.6370785422795638,
            0.6672843664190412,
        ],
        "heal_shield_per_min": [
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.2518610421836228,
            0.27435387673956263,
            0.29029905178701676,
            0.33812949640287765,
            0.4394395511058499,
            0.45949535192563074,
            0.5528255528255528,
            0.6969909027291813,
            0.7311960542540074,
            1.0881567230632236,
            1.1293441078610742,
            1.3386018237082067,
            1.3842394624312768,
            1.5018930642281656,
            1.816899189918992,
            1.8782561073405442,
            1.9212997804991068,
            1.9611246995569642,
            2.0089906791968013,
            2.0538208876227837,
            2.0635981132120813,
            2.099065600657878,
            2.1540155727460073,
            2.2262037702639352,
            2.280023432923257,
            2.2807308466978635,
            2.3570935841093825,
            2.4544745189634924,
            2.508186178876497,
            2.5777336817131427,
            2.775636723396239,
            2.788606731003505,
            2.825526673604271,
            2.9630237402878867,
            3.1165514619769987,
            3.194871664557477,
            3.2000064803644954,
            3.213317249698432,
            3.59125,
            4.01644245142003,
            4.102081549315082,
            4.258211594732174,
            4.521805273833672,
            4.536286177641062,
            4.66054688134657,
            4.755470314765949,
            4.761855040137779,
            4.827610466396616,
            4.909339501476891,
            4.95966369127339,
            5.228739816408104,
            5.540351776961201,
            5.618166383953881,
            5.763092124382447,
            5.959060806742926,
            5.995591433799006,
            6.0557512730058205,
            6.1584030907920155,
            6.63192389108249,
            6.782833028991868,
            6.794134975146307,
            6.981691814126664,
            7.2284918431654965,
            7.3791173311052765,
            7.601519887432037,
            7.616226769511269,
            7.822094511733739,
            8.534116312435353,
            8.640605313772877,
            8.76903445952085,
            9.101938386278741,
            9.215462186538371,
            9.239672461224425,
            9.287307663479787,
            9.646929824561404,
            9.697686879450382,
            10.366264886385924,
            10.62477073956185,
            10.628975632492121,
            10.824452864710336,
            10.936381712749112,
            11.036362418164963,
            11.442657778681923,
            11.85594843027062,
            11.975377493755555,
            12.71064044913924,
            12.846397331747092,
            12.956656493051282,
            12.990723562152134,
            13.21783313380891,
            13.34838095501527,
            13.943746531377816,
            14.072092934286031,
            14.856017516872987,
            14.914770214109407,
            15.148770233816958,
            15.160208374968006,
            15.16543182679504,
            15.476069711779276,
            15.969924812030076,
            17.609432904715035,
            17.86695545037474,
            18.215201391529785,
            18.94956152837119,
            19.171347782505983,
            19.376717224660798,
            19.534768545649566,
            19.735813967304757,
            19.799304580088815,
            22.025787372445397,
            22.564120646765367,
            22.728536548228753,
            23.37023531844352,
            24.282787195989304,
            24.326224675146065,
            25.187102652715783,
            25.88234796160446,
            25.987294727859815,
            26.300839092940993,
            26.511151691533463,
            30.338752868704805,
            31.281673754775902,
            31.3036159588537,
            31.529770830039105,
            33.60780419154864,
            33.630905068200434,
            33.694992599326724,
            34.147286035752586,
            35.52473684355936,
            37.05809013465765,
            39.75680637861026,
            40.19404426961977,
            41.01742683389757,
            41.86481033119605,
            42.22019633216707,
            42.263645003005294,
            43.616402023942555,
            43.65319796209939,
            47.017029361361224,
            49.18655284774814,
            53.389066041486224,
            53.77028575210827,
            53.84372473816787,
            56.14058943234481,
            56.68179660738843,
            63.3884403444009,
            63.79629933845923,
            66.84714034667442,
            80.51505630820978,
            81.82593095933865,
            81.94765600400507,
            82.1296730964648,
            82.20175670524732,
            86.51612835926797,
            87.88354785258039,
            92.18226034795369,
            102.02440451458138,
            106.86145650988495,
            109.59120841010983,
            112.44600579493466,
            115.58002765526615,
            120.74360724536335,
            124.76354996781014,
            132.2327890118562,
            139.13755541700078,
            164.2758747523349,
            168.79390047094975,
            172.90212814900946,
            179.2903463284183,
            199.8949120744719,
            209.15124154018812,
            210.02118355095075,
            231.092228886148,
            231.48667775951748,
            242.60345200224816,
            245.30862683824637,
            280.66448187308805,
            346.00379374247456,
        ],
        "objective_participation_ratio": [
            0.055224867724867725,
            0.05587301587301587,
            0.06314814814814815,
            0.06951058201058201,
            0.07277777777777779,
            0.07425925925925926,
            0.08526455026455025,
            0.08628306878306877,
            0.08915824915824917,
            0.0928919228919229,
            0.09493867243867243,
            0.09527777777777778,
            0.09633838383838383,
            0.09683862433862434,
            0.09775132275132274,
            0.10007529507529507,
            0.10023606023606023,
            0.10176286676286676,
            0.10203703703703702,
            0.10289682539682539,
            0.104512987012987,
            0.10674603174603174,
            0.10817580567580568,
            0.10841269841269843,
            0.11,
            0.11002886002886,
            0.11066137566137565,
            0.11087301587301586,
            0.1123160173160173,
            0.1130952380952381,
            0.11354497354497355,
            0.11484126984126984,
            0.1156746031746032,
            0.11822991822991824,
            0.12026168276168275,
            0.12298941798941798,
            0.12546296296296297,
            0.12694444444444444,
            0.13251322751322753,
            0.13392080142080143,
            0.1368939393939394,
            0.1372751322751323,
            0.13798534798534798,
            0.14105820105820105,
            0.14145502645502647,
            0.14203703703703704,
            0.14209235209235208,
            0.14373015873015874,
            0.14431697931697934,
            0.14484487734487733,
            0.1485978835978836,
            0.1493121693121693,
            0.14946368446368444,
            0.1495899470899471,
            0.15059523809523812,
            0.15243386243386245,
            0.1558597883597884,
            0.15662818662818662,
            0.1576058201058201,
            0.1603968253968254,
            0.16557239057239057,
            0.16560846560846562,
            0.16724386724386725,
            0.16869047619047622,
            0.1698447848447848,
            0.17152116402116402,
            0.1739356939356939,
            0.1755820105820106,
            0.17839345839345838,
            0.18090187590187592,
            0.18250000000000002,
            0.18345238095238098,
            0.1836820586820587,
            0.18395095645095644,
            0.18452020202020203,
            0.18588624338624343,
            0.1859259259259259,
            0.18628306878306883,
            0.18720298220298218,
            0.18726190476190477,
            0.18776048026048026,
            0.18875254375254383,
            0.188994708994709,
            0.19050264550264548,
            0.19060125060125058,
            0.1907443482443483,
            0.19128306878306878,
            0.19215959965959967,
            0.1926190476190476,
            0.19332251082251084,
            0.19336885336885334,
            0.19855820105820107,
            0.19896825396825396,
            0.19920634920634925,
            0.2019179894179894,
            0.20191919191919194,
            0.20240740740740742,
            0.2037301587301587,
            0.20387566137566135,
            0.20403439153439154,
            0.20584526584526583,
            0.20687830687830688,
            0.20784752284752284,
            0.2083910533910534,
            0.21054232804232806,
            0.21061447811447814,
            0.21209475709475709,
            0.21226190476190482,
            0.21269841269841272,
            0.21473905723905723,
            0.21547619047619052,
            0.21587301587301588,
            0.21720057720057717,
            0.2175036075036075,
            0.21814611314611312,
            0.21924603174603172,
            0.22053391053391053,
            0.22225783475783475,
            0.22564935064935063,
            0.22687830687830687,
            0.22856060606060605,
            0.2285978835978836,
            0.22865079365079363,
            0.2292063492063492,
            0.22991462241462246,
            0.23202861952861953,
            0.23464285714285718,
            0.237478354978355,
            0.23920634920634923,
            0.24104497354497353,
            0.24141774891774895,
            0.24257936507936506,
            0.24316017316017316,
            0.2457720057720058,
            0.24796536796536797,
            0.2502837902837903,
            0.25339947089947085,
            0.254047619047619,
            0.25412457912457914,
            0.2543831168831169,
            0.2567857142857143,
            0.2574460724460724,
            0.25795214045214043,
            0.25825396825396824,
            0.2603872053872054,
            0.26273809523809527,
            0.26273809523809527,
            0.26341269841269843,
            0.2634187109187109,
            0.2671693121693121,
            0.2672126022126022,
            0.26842592592592596,
            0.2692400192400192,
            0.27079365079365086,
            0.27246031746031746,
            0.27355820105820106,
            0.27412698412698416,
            0.27461640211640215,
            0.2771753246753247,
            0.2782178932178932,
            0.2787229437229437,
            0.281031746031746,
            0.282999037999038,
            0.28390211640211643,
            0.28523809523809524,
            0.2859259259259259,
            0.2862830687830688,
            0.2881349206349207,
            0.2882443482443483,
            0.2894179894179894,
            0.28992784992784987,
            0.2902164502164502,
            0.29072150072150077,
            0.2907551707551707,
            0.29099206349206347,
            0.29698653198653197,
            0.2999338624338624,
            0.3024148074148074,
            0.30250000000000005,
            0.305534188034188,
            0.3120502645502645,
            0.31547619047619047,
            0.31935185185185183,
            0.3208417508417509,
            0.3243650793650794,
            0.3247486772486773,
            0.33005291005291004,
            0.33341269841269844,
            0.339431216931217,
            0.34215608465608466,
            0.34746031746031747,
            0.3539417989417989,
            0.3549603174603175,
            0.3593253968253968,
            0.3601851851851851,
            0.36513227513227514,
            0.38390211640211647,
            0.4075925925925927,
            0.46562169312169305,
            0.4726587301587302,
        ],
        "roaming_ratio": [
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.004897255577974102,
            0.007887167928321417,
            0.008325328876507275,
            0.008687675880188261,
            0.010466162895359303,
            0.01065673459002876,
            0.01110046435621073,
            0.015774398064306087,
            0.016033982175145355,
            0.016650791279882664,
            0.02093257474951082,
            0.02278493214910109,
            0.024738179669593068,
            0.026641268657213456,
            0.027511013455057503,
            0.02756021476297354,
            0.028544888621412707,
            0.028675904007873846,
            0.030447366968793355,
            0.030525948212266344,
            0.030637887477695955,
            0.03402889687124853,
            0.03817922586631582,
            0.03838887313303691,
            0.03878381691951914,
            0.038977878661052665,
            0.04066227931336412,
            0.048101734500701605,
            0.050051720495709114,
            0.0505310100527097,
            0.05217222776380526,
            0.05226301919183518,
            0.05341790046347247,
            0.05498987570023992,
            0.05512931184835557,
            0.05966563064393519,
            0.05986551516966691,
            0.05990329510011732,
            0.06022855636672491,
            0.060441585532398166,
            0.060799431141496316,
            0.06081008170641301,
            0.06457055539542547,
            0.06510581112340241,
            0.06568324714714592,
            0.06775022463688436,
            0.07263925102684615,
            0.07288352831409509,
            0.08006761959199876,
            0.08097010296351653,
            0.08366951240743611,
            0.08577113243485268,
            0.08742924278980838,
            0.08915520824634494,
            0.08972521563587416,
            0.0906043679155947,
            0.09159509152712214,
            0.09234686167607946,
            0.0959604517382411,
            0.09791889718467646,
            0.0989534325211601,
            0.10106047618638334,
            0.10275889163695952,
            0.10460790298031311,
            0.10535023444333659,
            0.10644819647283013,
            0.10738582056386577,
            0.10760400456301412,
            0.10765212981400851,
            0.11015166460488028,
            0.11088603640255508,
            0.11120881276408556,
            0.11152603876208639,
            0.11323531199310079,
            0.11369184683979998,
            0.11406947312057068,
            0.11468949908189391,
            0.11519101131879685,
            0.11641274074671777,
            0.11774303293661652,
            0.11817344729132072,
            0.11866817155305166,
            0.11986701731438575,
            0.12313789672055946,
            0.1241202630623586,
            0.12621080761875061,
            0.12983135473575416,
            0.13141525204270263,
            0.13148404680376322,
            0.1323247502083977,
            0.13330779764451178,
            0.1334150702262148,
            0.13454440927909142,
            0.13545048395516315,
            0.13598752095155944,
            0.1382138224781763,
            0.13965764792396626,
            0.14257198464203136,
            0.14294594964659557,
            0.14749672771290193,
            0.14878924444198474,
            0.150014991693456,
            0.1530908332683575,
            0.15361602584616313,
            0.15388261288907223,
            0.16008780881449552,
            0.16048847876443942,
            0.16199223948942093,
            0.16437667362249261,
            0.16591428172528241,
            0.16830815582300676,
            0.17043981498628108,
            0.1709268510223665,
            0.17360962175655653,
            0.18155704686199634,
            0.18208157921088403,
            0.18804402877465634,
            0.1891158989375663,
            0.19013857061036918,
            0.19733037837877027,
            0.1985936867832495,
            0.19861127676723972,
            0.19999871580008352,
            0.20153674705822325,
            0.2017629357743291,
            0.2026645716464369,
            0.20531907057874219,
            0.20884693523820358,
            0.20886697995785933,
            0.20919037490879713,
            0.20927428678343393,
            0.20973996249497123,
            0.21074200543537483,
            0.21305908067863225,
            0.21456816686275965,
            0.21974209496708905,
            0.22580539297799795,
            0.22591409950177482,
            0.22635782985597366,
            0.22643960308298,
            0.22730175020252327,
            0.2340982828626645,
            0.23778519980710897,
            0.24129748116393113,
            0.24206132397211158,
            0.24972745178688824,
            0.2538716156039632,
            0.2545133371011941,
            0.25942821474364425,
            0.260610083465406,
            0.2609541366564395,
            0.2632022285646996,
            0.2746152583399307,
            0.28100953325631456,
            0.2894241095403803,
            0.29056990764961416,
            0.2922344713741466,
            0.29241629521654816,
            0.2962113861710174,
            0.2969091507676496,
            0.2972795980075123,
            0.29786581945825763,
            0.29838935205725103,
            0.298726763382019,
            0.30478734615086334,
            0.3066272678707679,
            0.30835320720773546,
            0.31143435314926865,
            0.31817242016711617,
            0.32077656757239137,
            0.3214551028371772,
            0.324180269033402,
            0.3325843913274805,
            0.3367940195043006,
            0.3395187288732142,
            0.3412367715722745,
            0.3473594590245742,
            0.3646450905542967,
            0.36732562734441215,
            0.37175633822409504,
            0.378977654970701,
            0.40659518944897177,
            0.4092300454225264,
            0.41848760983170596,
            0.4407348854429177,
            0.49609601197723135,
        ],
    },
    "resource_efficiency": {
        "gold_per_min": [
            279.9225847781396,
            287.3159504531663,
            287.8172087606276,
            289.9120630583129,
            290.35885654714923,
            290.8353598957605,
            291.58678746902183,
            292.00646698503,
            293.73902179491586,
            294.0855167225604,
            294.2098217349313,
            300.5910443009356,
            303.1902021304179,
            303.5944336641742,
            308.0185106302672,
            308.84318809507783,
            313.8019633080786,
            314.03577191357726,
            316.0160211608168,
            316.09598081835634,
            318.3120317924052,
            319.6253270959694,
            320.7023561146098,
            320.7058389947653,
            321.1301061058496,
            321.51826648214734,
            323.10069636290285,
            325.08732642209856,
            325.81555298654,
            329.4338643888206,
            330.8530475931871,
            332.89976083336427,
            332.90844774709944,
            333.4698273558431,
            333.5553222931391,
            335.120904745864,
            335.14257312477747,
            335.68158239392176,
            336.7802751982477,
            339.5050675845584,
            344.2370292043569,
            346.1079585285842,
            346.6305394866684,
            347.7945230146978,
            348.3371975110496,
            350.86330692649426,
            351.0020834401885,
            352.0224726546803,
            352.6633076142008,
            353.1383232201319,
            354.20456603517613,
            355.1033056416089,
            356.4007708955553,
            357.9102404441018,
            358.36356353322844,
            359.6276367065763,
            359.7865968476221,
            360.999311208636,
            363.22241840867827,
            365.18026856719223,
            365.5422276941834,
            366.08118833950994,
            366.87374482814903,
            367.4960925354264,
            368.47855782387853,
            369.8206505941147,
            369.92172425496915,
            370.4652233532458,
            371.34587931002017,
            371.49670932024753,
            372.50825834653403,
            372.80317121818564,
            374.3204990067162,
            375.6048456681709,
            375.8070381290407,
            375.90428855283244,
            378.3066650477177,
            378.3343067338292,
            379.0070172610187,
            379.2789785363168,
            379.4136755785649,
            379.96959156677974,
            380.76153512601627,
            381.08163249829994,
            381.34561891918366,
            381.44229503766104,
            382.05558469825155,
            383.433866296228,
            384.25575971564547,
            384.86558083030627,
            385.40551162712933,
            385.50432442883323,
            386.0944132993843,
            386.1931278089871,
            386.74491770391455,
            387.1551797741723,
            387.2847117596658,
            387.68582883958607,
            387.7729385787755,
            387.8038100968084,
            390.08914674078284,
            390.1721141128673,
            390.38989774611906,
            390.7715424686951,
            391.2483827268483,
            392.98167252617765,
            393.4889392028164,
            393.616197639611,
            394.02090560193443,
            394.6294657957426,
            395.09503157872865,
            395.4108182447521,
            396.31871103220413,
            397.1174073374195,
            397.9653323391989,
            398.3044106862544,
            401.4074218672657,
            401.5505272868202,
            402.47918667180255,
            403.5958127498114,
            404.68063526020944,
            404.77260461528294,
            406.3904892494295,
            407.6792425421506,
            408.0625606082868,
            410.2206462489118,
            412.0506705809117,
            412.3973417537111,
            413.92870140059864,
            414.45053290415416,
            414.76452551619667,
            415.0069998999991,
            415.1165327319404,
            416.11031969760376,
            416.1175427243845,
            417.18748760026676,
            417.5220830051498,
            417.6162276275282,
            417.9648977947989,
            418.3953997782039,
            420.8053567441513,
            421.886396746946,
            422.1589018134274,
            423.8169138880112,
            424.31022939572983,
            424.6335764410452,
            425.7536741057539,
            425.78732625689656,
            427.4649994892476,
            428.878058478068,
            429.6717909272711,
            429.85550014656735,
            431.54446331168293,
            431.76054247722925,
            431.9156349931323,
            432.9185899997618,
            433.93150313879386,
            434.7639224839833,
            435.5500500581771,
            436.8327272659392,
            438.93846257095163,
            439.0219396863808,
            439.95540238607094,
            440.331789297801,
            440.38482132016145,
            440.51304756355006,
            440.7228582519855,
            443.0214585121201,
            443.0647633090483,
            443.81854840149964,
            444.2498255143134,
            444.3417523330609,
            445.2665097803188,
            449.5433029978657,
            450.77899714706825,
            451.1496213542815,
            453.3648506939567,
            454.1565268180975,
            454.7624657780542,
            455.0189694144128,
            455.43251539591773,
            457.34338014779496,
            458.4426175508146,
            458.99221686368014,
            459.85865487922825,
            460.10017170482973,
            460.17822141724224,
            463.08208789337516,
            466.0421296417988,
            466.2473200272507,
            470.9266550877123,
            472.96188481604366,
            485.6309374311542,
            490.01946716742935,
            492.38873748083097,
            493.49247808703507,
            495.2918851944679,
            499.86990492658697,
            500.77617839324364,
            505.95597606581384,
        ],
        "cs_per_min": [
            0.9722645228173907,
            1.22543091946514,
            1.3914697913485818,
            1.4682974737977206,
            1.583002787121171,
            1.5866051195697695,
            1.6048364560324162,
            1.6314263765210562,
            1.6677741217925315,
            1.8957471530648702,
            1.929325158156412,
            1.9424956028854412,
            2.083826619024063,
            2.1692993201533244,
            2.2612529921798683,
            2.341848516384767,
            2.3684110849404703,
            2.4468903862426994,
            2.601180518893279,
            2.722980641911513,
            2.7995615702655807,
            2.9366066768255235,
            2.9474613209158727,
            3.0341648519404814,
            3.0366109612589107,
            3.2715711763576345,
            3.34299895955923,
            3.3742647818488085,
            3.3788177771498344,
            3.5112866764630124,
            3.5226679044157168,
            3.5427664118842848,
            3.5863156995979497,
            3.620771176811074,
            3.6766878523706974,
            3.826816314771955,
            3.882196957834735,
            3.8912966782228207,
            3.980820104883594,
            4.014957202768762,
            4.025140647952208,
            4.029558161013923,
            4.037787708002051,
            4.154683483327187,
            4.220995935556377,
            4.250373603094236,
            4.376438558809772,
            4.4593826205016285,
            4.521680102883766,
            4.595114774363017,
            4.651964388978906,
            4.652244109825786,
            4.671196382443459,
            4.7303145034760465,
            4.740517402752979,
            4.802078378766018,
            4.82414665769747,
            4.826264811938469,
            4.834276397813332,
            4.909772510393891,
            4.962762390264702,
            5.0295808787417275,
            5.073604226329021,
            5.1236768419584555,
            5.172230175695726,
            5.196337576376423,
            5.2006793642812985,
            5.263195044909829,
            5.27907605686965,
            5.284173720074887,
            5.309515981149445,
            5.323005612651364,
            5.373584266928329,
            5.40381872275286,
            5.4209086737085075,
            5.469462302308691,
            5.514154039343522,
            5.528574996608272,
            5.529331619985131,
            5.612749892944972,
            5.623235203092481,
            5.64678311224807,
            5.668419342438202,
            5.689615945372961,
            5.7220666024317275,
            5.817008903309473,
            5.817514625175151,
            5.836339978628729,
            5.8654961132619565,
            5.878030705962502,
            5.9102564269063125,
            5.926888360423064,
            5.94338583567115,
            5.952120040561424,
            5.956931947122834,
            5.9718472635352855,
            6.0485536475214525,
            6.078908699940239,
            6.113239426117184,
            6.155065399587509,
            6.173012709999414,
            6.261517334657992,
            6.283283102866097,
            6.293036004825568,
            6.294310417789584,
            6.377414698586957,
            6.412729803903502,
            6.44020684686431,
            6.471027936112092,
            6.483662760956347,
            6.4940198778999285,
            6.566116613789881,
            6.570018272539425,
            6.614600991138214,
            6.651973356257558,
            6.700414638221852,
            6.704229669986223,
            6.704336350564862,
            6.759028709118598,
            6.795771821153526,
            6.801854569674521,
            6.811689094144573,
            6.837785900481189,
            6.8725665476250075,
            6.882229246652691,
            6.9151719594635885,
            6.931188200510852,
            6.942874621690487,
            6.96376674422873,
            6.972349828081226,
            6.982633038538854,
            7.020015887333253,
            7.054857144178439,
            7.079885643030909,
            7.102194840302133,
            7.146926615240674,
            7.161148485399918,
            7.163340882820862,
            7.163632012427147,
            7.171460494224552,
            7.175482257456912,
            7.177025322352678,
            7.18668572534888,
            7.1907641050560756,
            7.21554624359796,
            7.303372258146087,
            7.31723248603699,
            7.35834339286012,
            7.372404974910014,
            7.403019147992145,
            7.427267672936722,
            7.4277844458763544,
            7.462340625170924,
            7.479483145862831,
            7.500425748856027,
            7.515313961956379,
            7.5192802076877125,
            7.624447489898237,
            7.647580129498865,
            7.657857494402657,
            7.661448881167071,
            7.699323038816298,
            7.719360302733451,
            7.7375103597975,
            7.761929049113647,
            7.763995015075193,
            7.801945071159375,
            7.832433297301632,
            7.884126271258924,
            7.889543805774568,
            7.938280177314058,
            7.980563137321988,
            8.014524918159378,
            8.125879206299121,
            8.183073225378376,
            8.221254263511916,
            8.309916594849087,
            8.324203827667068,
            8.448347265636123,
            8.7399819945968,
            8.741349315629895,
            8.913624004453764,
            9.198243458543953,
            9.287307550908885,
            9.29996545959167,
            9.532017591421164,
            9.545547252821345,
            9.687629872910845,
            9.741745013484316,
            9.792307758562323,
            9.805110374638115,
            9.930125430446614,
            10.62859375650377,
            10.809324627713814,
            10.846532692650948,
            11.02195178273074,
            11.114056191369455,
            11.443526008389902,
            11.70360161780069,
            12.783230468839019,
        ],
    },
    "vision_awareness": {
        "vision_score_per_min": [
            0.1578860230665481,
            0.18294645360617665,
            0.27756944956164037,
            0.2975743148514958,
            0.3300490828440335,
            0.36467079783604167,
            0.36901017620877946,
            0.38533170764140345,
            0.4005463745939756,
            0.40118632284782146,
            0.4015705762197079,
            0.4020354394629531,
            0.4106750999743702,
            0.4310821542144784,
            0.43137709377858957,
            0.4316796371349584,
            0.43899263016136947,
            0.4389955770351147,
            0.4400929454283886,
            0.4492284770913215,
            0.4524025472290966,
            0.45843524381134537,
            0.46181390145345363,
            0.46298451372309624,
            0.4632959275699882,
            0.46692394578419977,
            0.47100515104980173,
            0.4746817648014332,
            0.4859429682684923,
            0.49406932247126445,
            0.4949931865022587,
            0.49849214571648687,
            0.4987664901313052,
            0.4991481137143493,
            0.5022711378266927,
            0.503826233196857,
            0.5040506208358626,
            0.5066032077673491,
            0.5076886585428168,
            0.5159476124166928,
            0.5247807601565865,
            0.5257139353312573,
            0.5281825377908634,
            0.5303715784621391,
            0.5315941722030196,
            0.531657808849954,
            0.5377647616291179,
            0.5413116062572185,
            0.5436574934401828,
            0.5510689871102008,
            0.5528401456447736,
            0.5553582677115697,
            0.5591180933100717,
            0.5604333866766799,
            0.5614001794007367,
            0.56142665035016,
            0.5625055309365377,
            0.5656552272408476,
            0.5660291929536773,
            0.5686456570793514,
            0.569029705963503,
            0.5708388520883075,
            0.5726448679589847,
            0.5730289410712096,
            0.5791802020840829,
            0.5793514382243762,
            0.5799329651644489,
            0.5799498384584661,
            0.5852609141389201,
            0.5864517493828004,
            0.5918770082580852,
            0.6045234729248558,
            0.6049128344361521,
            0.6072492867885211,
            0.6093376440052899,
            0.60933971073654,
            0.6111871603520908,
            0.6138169844355504,
            0.618096288208576,
            0.622047376852234,
            0.6256268186855835,
            0.6263234840616421,
            0.6329865490376858,
            0.6404923882787663,
            0.6438931467057228,
            0.6446641973158339,
            0.6489205605621652,
            0.6562274788113036,
            0.662418892347808,
            0.6626255039943404,
            0.6627256407379769,
            0.6671137179454895,
            0.667394972465929,
            0.6684086117222687,
            0.6698066868486414,
            0.6699472127087029,
            0.6728870376322597,
            0.6743520079293436,
            0.6743719880154357,
            0.6759124992244264,
            0.6813035511553482,
            0.6872422811777131,
            0.6882085061029412,
            0.695208384232406,
            0.6979256115840575,
            0.6985470447767519,
            0.7021818092679696,
            0.7055271821099579,
            0.7067118158393402,
            0.7075197643958524,
            0.7077754929369762,
            0.7083438262498312,
            0.7094024994777406,
            0.7114352926629082,
            0.7132419596117713,
            0.71827947379104,
            0.7218847691337503,
            0.7272124387968636,
            0.7273207684361176,
            0.7295711648059592,
            0.7319934287838072,
            0.7320163772554801,
            0.7341867619749256,
            0.7355446860040262,
            0.7381757328134579,
            0.7426601869664115,
            0.7526012522565844,
            0.7593255818124454,
            0.7617112196079044,
            0.772543430502872,
            0.7810113613637716,
            0.7914080968411115,
            0.7929959492765195,
            0.8061311979709999,
            0.8326357097566505,
            0.8439495530878952,
            0.8534103677499346,
            0.8688170657342488,
            0.8745134274986662,
            0.8829998238394122,
            0.8965182021430849,
            0.8971046828046002,
            0.9061756429836968,
            0.9072242258387899,
            0.9291716285948464,
            0.9306619306124406,
            0.9343928369100604,
            0.9405908551517089,
            0.947662866620904,
            0.9487084248123823,
            0.963220106534867,
            0.9892137542982007,
            1.0076834511797925,
            1.0096045256544701,
            1.010236799402493,
            1.01554825491029,
            1.0159559587728137,
            1.0327056328445923,
            1.0353358792284557,
            1.0439131552877405,
            1.054319807347616,
            1.0797581241212713,
            1.0863930311664207,
            1.0899462151321637,
            1.113135773995,
            1.119997816542417,
            1.1358494175023803,
            1.1523794481206835,
            1.1612568827375078,
            1.1666641567995486,
            1.1754481303951005,
            1.185934897363986,
            1.193166623343577,
            1.228657903125938,
            1.2329068895083872,
            1.295261423709494,
            1.3164651663251206,
            1.332969857627532,
            1.3508502475121071,
            1.365731113698609,
            1.3755861738335295,
            1.3809014566251396,
            1.4268045064985477,
            1.4457863685339178,
            1.4680197392470085,
            1.4686877845452078,
            1.5289323900745382,
            1.6038062292050321,
            1.6702828099098312,
            1.7016237991749406,
            1.7342509351091617,
            1.773687134802011,
            1.8342822286704716,
            1.894840196361003,
            1.9540612849365382,
            2.145442241517209,
            2.2354473384689455,
            2.2613942437779073,
            2.271220853824999,
            2.3630421046772048,
        ]
    },
    "consistency": {
        "combat": [
            0.03447761381981131,
            0.09870493765745517,
            0.11097103627862574,
            0.1132362627830145,
            0.12750660072227385,
            0.12836387149102205,
            0.1338041507065291,
            0.1360107882071457,
            0.13632021028111213,
            0.1506890385144925,
            0.160116033379559,
            0.1624015004838577,
            0.16371725880954566,
            0.163835806602984,
            0.165455220676154,
            0.17080510317584385,
            0.1717522803376895,
            0.17264696143565428,
            0.1742775209130016,
            0.1752190764135449,
            0.17667292419189418,
            0.17688108142931536,
            0.18053235527627606,
            0.18126713424609078,
            0.18321015036373398,
            0.19629232324260418,
            0.19774612246534204,
            0.19930071100646482,
            0.20070701441271016,
            0.2019198125112417,
            0.21149977104534487,
            0.21278432757889054,
            0.21411763447543464,
            0.21485340618977564,
            0.21536084261405025,
            0.22154626911098063,
            0.22344296849817416,
            0.22396946481489388,
            0.22667958269226973,
            0.2309548300086236,
            0.23319267402753455,
            0.23607427022631508,
            0.23829927700034598,
            0.23932143486576163,
            0.2445455248209144,
            0.246981798525499,
            0.24726457644535121,
            0.24874946040496004,
            0.2516056437544441,
            0.25810308459257153,
            0.2639357190251489,
            0.2649956034354621,
            0.26585214355886755,
            0.2658838495322638,
            0.26623200862928065,
            0.26625040098026165,
            0.2694132906359162,
            0.27052704610949657,
            0.2729834677660133,
            0.2736499312619211,
            0.2785322401412244,
            0.28423713241743087,
            0.28503317953346285,
            0.28971115903572386,
            0.2915755746166354,
            0.29307314568876375,
            0.2932694571176334,
            0.2948228453843372,
            0.29490745342078234,
            0.29535953170853885,
            0.30048627513664045,
            0.3020436054531164,
            0.3030536917178961,
            0.3035604119177351,
            0.30471184159594566,
            0.3050387440551702,
            0.3062206981956935,
            0.3068372194871854,
            0.3102038672943211,
            0.31028842263563366,
            0.31079265967111747,
            0.3119187756366728,
            0.31268305823774806,
            0.3143373179700967,
            0.31541221359902605,
            0.3206873529861522,
            0.32179787209975863,
            0.32190595099393154,
            0.32271579665754274,
            0.32389325525147133,
            0.32550282581709655,
            0.3271519498688837,
            0.32719047180591243,
            0.3278817596678367,
            0.33131309940143455,
            0.3355099814817492,
            0.33620991477437934,
            0.338022125455833,
            0.33930253074667865,
            0.33965712246886837,
            0.3419649135930668,
            0.34255976021819146,
            0.34341138294738655,
            0.3439822602054864,
            0.34626019311993406,
            0.34923941125614805,
            0.35319507606603195,
            0.353859030306317,
            0.35735465548114154,
            0.35763173298138917,
            0.3594809699263847,
            0.359499878222103,
            0.36130769887117176,
            0.3620427172148639,
            0.36351705345947366,
            0.36632811540699345,
            0.3671901945649242,
            0.3680220436936044,
            0.37028550228598545,
            0.3757663286373489,
            0.3762283809157205,
            0.3763830544137755,
            0.37809228428203334,
            0.37859927688183287,
            0.37907228314340113,
            0.3797325570036227,
            0.38528782094355435,
            0.38702000226713223,
            0.388737871818873,
            0.3913509288704906,
            0.39228887735782153,
            0.3945653196450754,
            0.39664747191308347,
            0.3968531545016341,
            0.4018081062942056,
            0.40377205109140085,
            0.4073986828965719,
            0.40756968080218736,
            0.4096050554885223,
            0.41251179757626516,
            0.41272154436276764,
            0.417473876059684,
            0.41972525365280455,
            0.42150058565921056,
            0.42661264973979646,
            0.42729305114645655,
            0.43082643474042076,
            0.4311717315121103,
            0.4363869807803598,
            0.4412998661601044,
            0.4467258216770404,
            0.45600629615635585,
            0.45734788493835843,
            0.4583277238805265,
            0.4641839584773722,
            0.4693976179264568,
            0.4731322424435214,
            0.480648795928256,
            0.48533470388617733,
            0.48541997720964497,
            0.4867131438680326,
            0.48772364010096464,
            0.4889945055341452,
            0.4893032217633694,
            0.49007683996831947,
            0.4902049062883985,
            0.49175173783119724,
            0.49556789846182075,
            0.4979148421930144,
            0.5062853067231804,
            0.5068084673476613,
            0.508542481279185,
            0.5088237564574262,
            0.5135749471938685,
            0.5141273404120131,
            0.5165612438370204,
            0.5206013183677225,
            0.5264178202157279,
            0.530143194977037,
            0.5360948259941258,
            0.5392215968039182,
            0.5421725499065911,
            0.5432824212419964,
            0.5516597929038026,
            0.5615358821363824,
            0.5751922800149055,
            0.5760333369971578,
            0.5782787256522595,
            0.5783369121659878,
            0.5864013703990653,
            0.5924021123745321,
            0.6127168962724794,
            0.6127694151990986,
            0.6538518140524083,
            0.6679099363806769,
            0.6712181615602641,
            0.73759843421615,
            0.7521879693785855,
            0.7566421005976283,
            0.8350256207254352,
        ],
        "team_contribution": [
            0.0013843983163537603,
            0.005672791436107151,
            0.006166220917506233,
            0.00785352691720512,
            0.015166096049817833,
            0.022109526863533117,
            0.02213480017899164,
            0.02392025067986951,
            0.0289540036633235,
            0.03051435644923734,
            0.030972181444809517,
            0.03384179487490655,
            0.035318510003053895,
            0.03560263548754341,
            0.03568230874038065,
            0.03880885733389632,
            0.039882410123494104,
            0.039959502809867334,
            0.043409915500966144,
            0.0438405802656007,
            0.044140252007996615,
            0.04506200623145484,
            0.04593707872003755,
            0.04642398006744233,
            0.04729667499450168,
            0.04849661953620397,
            0.04957547272357499,
            0.05038671497743136,
            0.05111457544536335,
            0.05193376044462937,
            0.05197189632134443,
            0.05444777035764095,
            0.05496089178697399,
            0.055197948649615296,
            0.05610533468296605,
            0.05698254302523229,
            0.057701658878975444,
            0.05836062795487259,
            0.06101075862502216,
            0.06257335889165902,
            0.06300612893695079,
            0.06334631588021065,
            0.0635033865405777,
            0.0679824286632369,
            0.06953576368805696,
            0.07045961539540115,
            0.07204175803381736,
            0.07253899070537494,
            0.07292971711263059,
            0.07327344001330667,
            0.0736348391049801,
            0.07369869685659784,
            0.07374819494435078,
            0.07460193127337927,
            0.07565294193285654,
            0.07782721511084935,
            0.07842137049818938,
            0.07844696534296636,
            0.0793197924354743,
            0.08021405183008137,
            0.08080347547636839,
            0.08337276250927517,
            0.08451634014905732,
            0.08460625560369746,
            0.08649058980880807,
            0.08656744262147549,
            0.08674519410427729,
            0.08702631491254707,
            0.08904023160356116,
            0.08919511781332735,
            0.09093648152073316,
            0.09238129822459004,
            0.09517049122649573,
            0.09544079110862404,
            0.09551459001621347,
            0.09555259422606485,
            0.09565930222351518,
            0.09740960545544629,
            0.09742053618304437,
            0.09829120626434551,
            0.1020987830353178,
            0.10263163287915816,
            0.10300807986724067,
            0.10330758503153596,
            0.10735749797916033,
            0.10867572303273157,
            0.10889168828021571,
            0.10943862998864626,
            0.11314545605648098,
            0.1139056912770697,
            0.11647344576761084,
            0.11768245557710974,
            0.11770711112228592,
            0.11930113728638563,
            0.11958343679821937,
            0.12227106940204376,
            0.12319693404879488,
            0.12444770163329906,
            0.12454782882789538,
            0.12661106866849314,
            0.12871017873312593,
            0.13013730815394237,
            0.13326958553600213,
            0.1357353504435474,
            0.13867169804762688,
            0.13937938680824274,
            0.13970271620572228,
            0.1400847954633724,
            0.14042244870233625,
            0.14160902740841105,
            0.14221436253811387,
            0.14264143425827377,
            0.14685360956242283,
            0.1471310235046999,
            0.1471512789573615,
            0.14868832184101485,
            0.14885635580096093,
            0.14919119887707094,
            0.14962498878143876,
            0.14993284501997178,
            0.15562403904789063,
            0.15731161090005447,
            0.15843742324538515,
            0.16021235781630525,
            0.1610672551115353,
            0.16125079381808083,
            0.16262218825557348,
            0.16565261743704476,
            0.16570216801020343,
            0.16685339510727543,
            0.16951114741343956,
            0.16996146668948414,
            0.17253252224584942,
            0.1743285267749977,
            0.17568932082953814,
            0.17820025993646657,
            0.17821188845101812,
            0.1790303890595179,
            0.180738807420851,
            0.18944505062245576,
            0.1902604554982816,
            0.19055768809686632,
            0.19440592403006224,
            0.20862059042334438,
            0.2099962030677307,
            0.21086853534078817,
            0.21210864371183183,
            0.21312941320281142,
            0.21509656612451464,
            0.21661348207132972,
            0.217313888785504,
            0.21969791924394902,
            0.2234598490085011,
            0.22555221922756344,
            0.23303563841591154,
            0.23410266714355318,
            0.23575994391411842,
            0.23743306092774127,
            0.2374453880875633,
            0.23807489890073152,
            0.23954451274340163,
            0.2396266193798805,
            0.2435967258966733,
            0.24396119927415613,
            0.2491240919252743,
            0.2510517094739149,
            0.2511850444905185,
            0.25278251347531977,
            0.2549914517405642,
            0.257493868553231,
            0.25938473371590437,
            0.26032800222036595,
            0.2641792000267543,
            0.2691399420819302,
            0.2730298542413482,
            0.27307834537353093,
            0.2744586526321773,
            0.27986415021843075,
            0.2855147497339866,
            0.29696616533137726,
            0.31407911851997006,
            0.31853608714834986,
            0.3244377386553559,
            0.3508409368846547,
            0.35697584299726126,
            0.36598457892360287,
            0.36850370764884566,
            0.3850598746747692,
            0.3930064859096401,
            0.3937547258990873,
            0.40316849850790115,
            0.41476071361518074,
            0.41998888494688186,
            0.42127081508998093,
            0.4522306279547622,
            0.48940891073217696,
            0.5050080920397074,
            0.508070931533007,
            0.512865638487539,
            0.5553892218042954,
        ],
        "resource_efficiency": [
            4.1711000285866437e-07,
            5.323173884738914e-07,
            6.9372239425981815e-06,
            3.1356246604196186e-05,
            3.409379667971222e-05,
            4.304003778105158e-05,
            6.16216149883069e-05,
            9.670287265147429e-05,
            0.0001140348307628852,
            0.0001510465463933277,
            0.00019264672775640027,
            0.00019375570009518822,
            0.00020154357195207174,
            0.00022302708487439158,
            0.00028332135958734173,
            0.0003268249356788899,
            0.0003380595094403753,
            0.0003769342902271979,
            0.0004096205204633577,
            0.0005483514995794383,
            0.0005920260744423737,
            0.0007850028863779561,
            0.0008172701132748864,
            0.001206780878258526,
            0.001433077726626098,
            0.0018415646826563854,
            0.0019287466183582906,
            0.003071711719985627,
            0.0037455751446414183,
            0.004068307631153515,
            0.004100008442802146,
            0.004110880879407831,
            0.004179849048007906,
            0.004628751964044621,
            0.005467343176488768,
            0.005859137132133977,
            0.007192343379636477,
            0.007249229919118335,
            0.007581495765323337,
            0.007621025942010136,
            0.008036985848778277,
            0.008271119555998699,
            0.008521404047436645,
            0.009985899172377265,
            0.010353540850938783,
            0.011212173574556806,
            0.011618616253435105,
            0.012083965288758235,
            0.012796656588618113,
            0.01487099167947096,
            0.014917660506234562,
            0.017389556445567066,
            0.01783663613285108,
            0.01961369342919243,
            0.02124276233624671,
            0.025043147564081998,
            0.025876102945292037,
            0.028271901328729806,
            0.029012284929540725,
            0.03539849779413555,
            0.038636588158857246,
            0.04106719218182209,
            0.043641561820832064,
            0.04386319784551446,
            0.049793424275703185,
            0.05176218226553814,
            0.052428579595874825,
            0.05394625091773972,
            0.054955694283875825,
            0.05503144450586824,
            0.05949991634330405,
            0.0596694229501523,
            0.062118887495472924,
            0.062365735764028866,
            0.06330400301099112,
            0.06619923420922773,
            0.06645950471384186,
            0.06713035647794864,
            0.0684325113368971,
            0.07350030454708843,
            0.08074161770912174,
            0.08351938271180417,
            0.08464005903174227,
            0.08545034488948165,
            0.08741990827388983,
            0.09609300001403465,
            0.09708798034901035,
            0.09973381418407244,
            0.10200578596708444,
            0.1045242171170167,
            0.10638483490813191,
            0.10825160629442028,
            0.1119604604294091,
            0.11544457037308982,
            0.11961261441343976,
            0.12472926407924144,
            0.12895395048908032,
            0.13112571556630984,
            0.13164697165560632,
            0.1340885202051693,
            0.13785717694982497,
            0.14318900984836436,
            0.14385778310338565,
            0.14516387228084032,
            0.15012764343333546,
            0.1525255528825903,
            0.15498456060760324,
            0.15873381572187284,
            0.15918447111573716,
            0.16292986572611642,
            0.1662064952047588,
            0.166322662458967,
            0.17485322247001073,
            0.18754129774389272,
            0.18845749546212917,
            0.18999078947709427,
            0.20692975700188337,
            0.20779759421630895,
            0.23293965988541945,
            0.2436331316781554,
            0.24669883740331838,
            0.2484178122260344,
            0.24881532802245931,
            0.25619262832623463,
            0.2586602571589605,
            0.26383467513554276,
            0.27094051977116057,
            0.27106776650568265,
            0.2713647107455514,
            0.27673355725586846,
            0.2794487831312768,
            0.2852775372526654,
            0.29539955607698504,
            0.30386651423300676,
            0.30757990103247734,
            0.31896431045826645,
            0.33106834692414056,
            0.3327668972364851,
            0.3335989766201778,
            0.3512452172789269,
            0.36149244163860156,
            0.38390972216629693,
            0.3872123089953693,
            0.39086725033409486,
            0.3979813315139594,
            0.3990290341053365,
            0.4030770470175547,
            0.41474404816133686,
            0.4212482507968345,
            0.4277410278074381,
            0.4312798211712471,
            0.43327379157798707,
            0.433698494932938,
            0.43714500625287045,
            0.43835904789405034,
            0.43996018815369986,
            0.4424030538841602,
            0.45053664077166794,
            0.4510606779823492,
            0.4528419801779087,
            0.4548982652669399,
            0.46383419749368054,
            0.4705502475019518,
            0.4807133825497878,
            0.4827293889657837,
            0.4889285827392937,
            0.48918508350251205,
            0.5017773807918618,
            0.5039837306456674,
            0.506937191926347,
            0.5145425308704044,
            0.5191746083738038,
            0.5230772648121482,
            0.5245483572809257,
            0.5272617877426304,
            0.527383049468018,
            0.5379073594106354,
            0.5452378994810936,
            0.5523456504292582,
            0.5548770036547774,
            0.5685066748665806,
            0.576588851519255,
            0.5847962505847036,
            0.5861382425159468,
            0.5872677546229866,
            0.5928366401090199,
            0.6200143441978389,
            0.628094830209969,
            0.6351597012885246,
            0.6417672250295501,
            0.6451163708856664,
            0.6469349270181882,
            0.6594886296231288,
            0.6836694233599905,
            0.6910286766370652,
            0.7226675596076718,
            0.730202252920861,
            0.7587857884449425,
            0.775643340119311,
            0.8940012480329796,
        ],
        "vision_awareness": [
            6.426367519432109e-10,
            6.7055709775709085e-06,
            3.047590684724673e-05,
            7.62434812352999e-05,
            0.0003423819107344189,
            0.0003562679595902117,
            0.0003605036572689125,
            0.0006240032457874999,
            0.000624374496111567,
            0.0007765383461637538,
            0.0008055468171423748,
            0.0009468451306611697,
            0.0009839286094718662,
            0.0010194634406393488,
            0.0010538794783326011,
            0.0011240846347605553,
            0.0012075228788566443,
            0.001245806887357481,
            0.0017000876498425154,
            0.001717189141009674,
            0.001850462535061148,
            0.001884239125077757,
            0.001896946983017704,
            0.002909151871115845,
            0.0030670264511463564,
            0.00329336253866822,
            0.003400328963141843,
            0.004014419223026898,
            0.0041568930775461276,
            0.004255797479174435,
            0.004444159776422134,
            0.004458279099192935,
            0.004635668393714454,
            0.004852950356900245,
            0.005160107114422788,
            0.007510520144858501,
            0.007936094657878782,
            0.008179444530799947,
            0.008812347689394897,
            0.008823300886162986,
            0.009248915277650715,
            0.009339760019693168,
            0.010165649276916341,
            0.010192054248397435,
            0.010719887586737158,
            0.010722576229309002,
            0.011336336355051976,
            0.011718783509403584,
            0.011882016282045754,
            0.012172065687092318,
            0.012980815429855687,
            0.014243918140673422,
            0.014509121771832664,
            0.01527558119784815,
            0.017724971552338247,
            0.018556328208565585,
            0.018798618720748075,
            0.020345514487163253,
            0.021262508474047575,
            0.023706004488964372,
            0.02412247150794512,
            0.025111257383038176,
            0.02542194666765305,
            0.02572542017083305,
            0.02632409444587362,
            0.0279608184875342,
            0.02818375537862076,
            0.028196211221380205,
            0.029765196413234603,
            0.03107253067642439,
            0.03163221974739618,
            0.032171627765784,
            0.03305500060245608,
            0.033424248947908915,
            0.035322445777275606,
            0.03885987064421207,
            0.040515493837995915,
            0.041757528544784664,
            0.04175815422697666,
            0.04230253881894638,
            0.04383054023839007,
            0.04552455083932571,
            0.050382130945232036,
            0.05133034806138083,
            0.052492876541553934,
            0.053620887430506664,
            0.05559973534782697,
            0.05716779867303521,
            0.0584641208546985,
            0.05948501128932489,
            0.062013848266837375,
            0.06756895869932066,
            0.06958097484869474,
            0.06975064665396362,
            0.07035219483367557,
            0.07309964097265564,
            0.07566035289017778,
            0.07599837651108657,
            0.08087193677380015,
            0.0819556340612544,
            0.08435111159528064,
            0.08664104187177837,
            0.08813296080005784,
            0.08955822209627026,
            0.08981598415229963,
            0.09381074144996761,
            0.09840278171831827,
            0.09844929839897446,
            0.10230469432819154,
            0.10244101255864899,
            0.10250594859196409,
            0.10531384977303591,
            0.10703130512902288,
            0.1073120995985256,
            0.10955670919548728,
            0.11454706373803646,
            0.11463492661772429,
            0.11719101597666053,
            0.11797298935432438,
            0.12171327245716611,
            0.12809401061596,
            0.13061067202537793,
            0.13129869958322854,
            0.13637404131636774,
            0.13683173246377778,
            0.14138555903084882,
            0.14357768711949365,
            0.14910410336858335,
            0.1551395396401163,
            0.15694568379298127,
            0.15747672271892532,
            0.15766517890632317,
            0.1591901124056926,
            0.16058643277163842,
            0.16704500948744574,
            0.1670644559379343,
            0.1761075743119585,
            0.17679838463354247,
            0.19222589444294894,
            0.19263680111614642,
            0.19718278678391218,
            0.19809273648783163,
            0.2125612805906545,
            0.22476993448955032,
            0.22562489809447211,
            0.2308077777771462,
            0.24643060414981052,
            0.24723027056785687,
            0.2577081203283695,
            0.25918920540325807,
            0.2596427631234855,
            0.2724480016650563,
            0.27310129424035706,
            0.28187612936769146,
            0.28276059704104156,
            0.28940720271476916,
            0.3011424330417399,
            0.30157764728031405,
            0.3130492211633441,
            0.31544573775161666,
            0.3160809538289406,
            0.32282507300511365,
            0.3254582645966823,
            0.3330209433428669,
            0.33358872907904813,
            0.34112525040673125,
            0.34124337035768176,
            0.34293232996625256,
            0.3506637081840291,
            0.35313723078560627,
            0.3826536879122987,
            0.3926656830988693,
            0.39459903374595234,
            0.39661696501361166,
            0.4062924290112854,
            0.4210020919737743,
            0.4320371667969611,
            0.4480750729853473,
            0.4559595028632697,
            0.45870666231710755,
            0.465048133894361,
            0.4784423060973487,
            0.48796711787476665,
            0.5205869905122623,
            0.5208797491736754,
            0.5500205387107914,
            0.5524631486829696,
            0.5550051961067185,
            0.5641309071982585,
            0.5685079893657966,
            0.5710400304805268,
            0.5870205335824952,
            0.591086736195567,
            0.654320127521817,
            0.6590013881924426,
            0.69362179349266,
            0.6975215488155957,
            0.7257691181878888,
            0.8257468593796012,
            0.8959838180408748,
        ],
    },
}

#### Metric Weights

In [ ]:
combat_weighting = {
    "kill_participation": 0.28,
    "kda": 0.21,
    "damage_per_min": 0.14,
    "damage_share": 0.07,
    "double_kills": 0.03,
    "triple_kills": 0.045,
    "quadra_kills": 0.06,
    "penta_kills": 0.09,
    "first_blood_kill": 0.06,
    "first_blood_assist": 0.015,
}

team_contribution_weighting = {
    "kill_participation": 0.3,
    "heal_shield_per_min": 0.2,
    "objective_participation_ratio": 0.4,
    "roaming_ratio": 0.1,
}

resource_efficiency_weighting = {"gold_per_min": 0.8, "cs_per_min": 0.2}
vision_awareness_weighting = {"vision_score_per_min": 1}
consistency_weighting = {
    "combat": 0.3,
    "team_contribution": 0.3,
    "resource_efficiency": 0.3,
    "vision_awareness": 0.1,
}
print(sum(combat_weighting.values()))
print(sum(team_contribution_weighting.values()))
print(sum(resource_efficiency_weighting.values()))
print(sum(vision_awareness_weighting.values()))
print(sum(consistency_weighting.values()))

#### Compute Ionia Metric Scores

In [ ]:
def normalize_raw_metric(value, metric_name, category):
    # print(category, metric_name, value)
    sample_array = sample_baselines[category][metric_name]
    return ranked_cdf_score(value, sample_array)

#### Info

In [ ]:
archetype_means = [
    {
        "archetype": 0,
        "combat_effectiveness": 0.5239931174620329,
        "team_contribution": 0.5780096332198227,
        "resource_efficiency": 0.30694336762159846,
        "vision_awareness": 0.774767163064712,
        "consistency": 0.6516261089220058
    },
    {
        "archetype": 1,
        "combat_effectiveness": 0.6933594668287115,
        "team_contribution": 0.5392140473188785,
        "resource_efficiency": 0.8016641514008339,
        "vision_awareness": 0.5038720136384401,
        "consistency": 0.9014789823333402
    },
    {
        "archetype": 2,
        "combat_effectiveness": 0.38263965764800645,
        "team_contribution": 0.3418902972482486,
        "resource_efficiency": 0.34210270899667966,
        "vision_awareness": 0.35959583574442683,
        "consistency": 0.3369011057452526
    },
    {
        "archetype": 3,
        "combat_effectiveness": 0.5696781252482486,
        "team_contribution": 0.43406597785233386,
        "resource_efficiency": 0.7303831094507716,
        "vision_awareness": 0.4341589641763218,
        "consistency": 0.734013357445227
    },
    {
        "archetype": 4,
        "combat_effectiveness": 0.5017346754630886,
        "team_contribution": 0.41000935077012235,
        "resource_efficiency": 0.572489480057141,
        "vision_awareness": 0.4242749648593833,
        "consistency": 0.5067579073677854
    }
]

In [ ]:
def determine_archetype(player_scores):
    def cosine_similarity(a, b):
        return np.dot(a, b) / (norm(a) * norm(b))

    archetype_means_vectors = np.array(
        [
            [
                archetype["combat_effectiveness"],
                archetype["team_contribution"],
                archetype["resource_efficiency"],
                archetype["vision_awareness"],
                archetype["consistency"],
            ]
            for archetype in archetype_means
        ]
    )
    player_vector = np.array(
        [
            player_scores["combat_effectiveness"],
            player_scores["team_contribution"],
            player_scores["resource_efficiency"],
            player_scores["vision_awareness"],
            player_scores["consistency"],
        ]
    )

    similarities = [cosine_similarity(player_vector, a) for a in archetype_means_vectors]
    assigned = np.argmax(similarities).item()
    return {
        "archetype": assigned,
        "similarity_score": similarities[assigned].item(),
    }

In [ ]:
def info_from_raw_ionia_metrics(raw_metrics: List[Dict[str, Any]]):
    if not raw_metrics:
        return {
            "combat_effectiveness": 0.0,
            "team_contribution": 0.0,
            "resource_efficiency": 0.0,
            "vision_awareness": 0.0,
            "consistency": 0.0,
        }
    combat_scores = []
    team_contribution_scores = []
    resource_efficiency_scores = []
    vision_awareness_scores = []
    for m in raw_metrics:
        combat_scores.append(sum([normalize_raw_metric(m["combat_metrics"][metric], metric, "combat") * weight for metric, weight in combat_weighting.items()]))
        team_contribution_scores.append(
            sum([normalize_raw_metric(m["team_contribution_metrics"][metric], metric, "team_contribution") * weight for metric, weight in team_contribution_weighting.items()])
        )
        resource_efficiency_scores.append(
            sum([normalize_raw_metric(m["resource_efficiency_metrics"][metric], metric, "resource_efficiency") * weight for metric, weight in resource_efficiency_weighting.items()])
        )
        vision_awareness_scores.append(sum([normalize_raw_metric(m["vision_awareness_metrics"][metric], metric, "vision_awareness") * weight for metric, weight in vision_awareness_weighting.items()]))

    statistics = {
        "combat": {
            "avg": np.mean(combat_scores).item(),
            "std": np.std(combat_scores).item(),
        },
        "team_contribution": {
            "avg": np.mean(team_contribution_scores).item(),
            "std": np.std(team_contribution_scores).item(),
        },
        "resource_efficiency": {
            "avg": np.mean(resource_efficiency_scores).item(),
            "std": np.std(resource_efficiency_scores).item(),
        },
        "vision_awareness": {
            "avg": np.mean(vision_awareness_scores).item(),
            "std": np.std(vision_awareness_scores).item(),
        },
    }

    consistency_score = sum(
        [normalize_raw_metric(consistency_score_logistic(statistics[metric]["avg"], statistics[metric]["std"]), metric, "consistency") * weight for metric, weight in consistency_weighting.items()]
    )

    archetype_info = determine_archetype({
        "combat_effectiveness": statistics["combat"]["avg"],
        "team_contribution": statistics["team_contribution"]["avg"],
        "resource_efficiency": statistics["resource_efficiency"]["avg"],
        "vision_awareness": statistics["vision_awareness"]["avg"],
        "consistency": consistency_score,
    })
    
    return {
        "combat_effectiveness": statistics["combat"]["avg"],
        "team_contribution": statistics["team_contribution"]["avg"],
        "resource_efficiency": statistics["resource_efficiency"]["avg"],
        "vision_awareness": statistics["vision_awareness"]["avg"],
        "consistency": consistency_score,
        "archetype_id": archetype_info["archetype"],
        "archetype_similarity": archetype_info["similarity_score"],
    }

#### K Means Clustering (Experimenting)

In [ ]:
k_means_records = []

for rank in player_matches:
    for player in player_matches[rank]:
        matches = player_matches[rank][player]
        puuid = player

        raw_ionia_metrics_sr = []
        for match_id in matches:
            try:
                with open(f"Player_Data/{rank}/data/processed/match_data/{match_id}_processed.json", "r", encoding="utf-8") as f:
                    match_data_json = json.load(f)
                    match_data = MatchData.model_validate(match_data_json)
                with open(f"Player_Data/{rank}/data/processed/timeline_data/{match_id}_timeline_processed.json", "r", encoding="utf-8") as f:
                    timeline_data_json = json.load(f)
                    timeline_data = MatchTimelineData.model_validate(timeline_data_json)
                raw_ionia_metrics_sr.append(get_raw_ionia_metrics_for_match(puuid, match_data, timeline_data))
                ionia_info_sr = info_from_raw_ionia_metrics(raw_ionia_metrics_sr)
                k_means_records.append(
                    {
                        "combat_effectiveness": ionia_info_sr["combat_effectiveness"],
                        "team_contribution": ionia_info_sr["team_contribution"],
                        "resource_efficiency": ionia_info_sr["resource_efficiency"],
                        "vision_awareness": ionia_info_sr["vision_awareness"],
                        "consistency": ionia_info_sr["consistency"],
                    }
                )
            except Exception as e:
                print(f"Error processing match {match_id}: {e}")
                continue

In [ ]:
from sklearn.preprocessing import StandardScaler

df = pd.DataFrame(k_means_records)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), cmap="coolwarm", center=0)
plt.title("Feature Correlations")
plt.show()

In [ ]:
from sklearn.cluster import KMeans

# Try a few cluster counts to see what fits
inertia = []
for k in range(2, 10):
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(X_scaled)
    inertia.append(km.inertia_)

plt.plot(range(2, 10), inertia, marker="o")
plt.title("Elbow Method for K")
plt.xlabel("Number of clusters")
plt.ylabel("Inertia")
plt.show()


In [ ]:
kmeans = KMeans(n_clusters=5, random_state=42)
df["archetype"] = kmeans.fit_predict(X_scaled)


# Average stats per archetype
archetype_summary = df.groupby("archetype").mean()
print(archetype_summary)


In [ ]:
with open("k_means_archetypes_sr.json", "w", encoding="utf-8") as f:
    json.dump(archetype_summary.reset_index().to_dict(orient="records"), f, indent=4)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
coords = pca.fit_transform(X_scaled)

plt.figure(figsize=(8, 6))
sns.scatterplot(x=coords[:, 0], y=coords[:, 1], hue=df["archetype"], palette="tab10")
plt.title("Player Archetypes (PCA)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.show()


In [ ]:
archetype_means = [
    {
        "archetype": 0,
        "combat_effectiveness": 0.5239931174620329,
        "team_contribution": 0.5780096332198227,
        "resource_efficiency": 0.30694336762159846,
        "vision_awareness": 0.774767163064712,
        "consistency": 0.6516261089220058
    },
    {
        "archetype": 1,
        "combat_effectiveness": 0.6933594668287115,
        "team_contribution": 0.5392140473188785,
        "resource_efficiency": 0.8016641514008339,
        "vision_awareness": 0.5038720136384401,
        "consistency": 0.9014789823333402
    },
    {
        "archetype": 2,
        "combat_effectiveness": 0.38263965764800645,
        "team_contribution": 0.3418902972482486,
        "resource_efficiency": 0.34210270899667966,
        "vision_awareness": 0.35959583574442683,
        "consistency": 0.3369011057452526
    },
    {
        "archetype": 3,
        "combat_effectiveness": 0.5696781252482486,
        "team_contribution": 0.43406597785233386,
        "resource_efficiency": 0.7303831094507716,
        "vision_awareness": 0.4341589641763218,
        "consistency": 0.734013357445227
    },
    {
        "archetype": 4,
        "combat_effectiveness": 0.5017346754630886,
        "team_contribution": 0.41000935077012235,
        "resource_efficiency": 0.572489480057141,
        "vision_awareness": 0.4242749648593833,
        "consistency": 0.5067579073677854
    }
]

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

archetype_means_vectors = np.array([
    [
        archetype["combat_effectiveness"],
        archetype["team_contribution"],
        archetype["resource_efficiency"],
        archetype["vision_awareness"],
        archetype["consistency"],
    ]
    for archetype in archetype_means
])

player = np.array([0.60, 0.55, 0.65, 0.40, 0.70])

similarities = [cosine_similarity(player, a) for a in archetype_means_vectors]
assigned = np.argmax(similarities)

print(f"Closest archetype: {assigned}, similarity: {similarities[assigned]:.3f}")


## Demacia

### Raw

In [ ]:
def get_raw_demacia_metrics_for_match(puuid: str, match_data: MatchData, timeline_data: MatchTimelineData):
    team_id = match_data.players[next(i for i, p in enumerate(match_data.players) if p.identity.puuid == puuid)].identity.teamId
    teammates = [
        {
            "puuid": player.identity.puuid,
            "riotIdGameName": player.identity.riotIdGameName,
            "riotIdTagline": player.identity.riotIdTagline,
        }
        for player in match_data.players
        if player.identity.puuid != puuid and player.identity.teamId == team_id
    ]
    win = match_data.game.teams[next(i for i, t in enumerate(match_data.game.teams) if t.teamId == team_id)].win
    return {
        "teammates": teammates,
        "win": win,
        "gameMode": match_data.game.gameMode,
    }

### Info

In [ ]:
def info_from_raw_demacia_metrics(raw_metrics: List[Dict[str, Any]]):
    # find top 10 teammates and winrate
    teammates = {}
    for m in raw_metrics:
        for teammate in m["teammates"]:
            if teammate["puuid"] not in teammates:
                teammates[teammate["puuid"]] = {
                    "puuid": teammate["puuid"],
                    "matches": 0,
                    "wins": 0,
                    "riotIdGameName": teammate["riotIdGameName"],
                    "riotIdTagline": teammate["riotIdTagline"],
                }
            teammates[teammate["puuid"]]["matches"] += 1
            if m["win"]:
                teammates[teammate["puuid"]]["wins"] += 1
    most_common_teammates = sorted(teammates.items(), key=lambda x: x[1]["matches"], reverse=True)[:10]
    return {
        "most_common_teammates": [
            {
                "puuid": data["puuid"],
                "riotIdGameName": data["riotIdGameName"],
                "riotIdTagline": data["riotIdTagline"],
                "matches": data["matches"],
                "wins": data["wins"],
            }
            for teammate, data in most_common_teammates
        ],
    }

## Targon

### Raw

In [ ]:
def get_raw_targon_metrics_for_match(puuid: str, match_data: MatchData, timeline_data: MatchTimelineData):
    player = match_data.players[next(i for i, p in enumerate(match_data.players) if p.identity.puuid == puuid)]
    champion = player.identity.championName
    q_ability_casts = player.abilityUsage.spell1Casts
    w_ability_casts = player.abilityUsage.spell2Casts
    e_ability_casts = player.abilityUsage.spell3Casts
    r_ability_casts = player.abilityUsage.spell4Casts

    play_time = player.combat.timePlayed / 60
    game_count = 1
    win = match_data.game.teams[next(i for i, t in enumerate(match_data.game.teams) if t.teamId == player.identity.teamId)].win

    kills = player.combat.kills
    deaths = player.combat.deaths
    assists = player.combat.assists
    kda = (kills + assists) / max(1, deaths)

    gold_earned = player.economy.goldEarned
    jungle_minions_killed = player.farming.neutralMinionsKilled + player.farming.totalAllyJungleMinionsKilled + player.farming.totalEnemyJungleMinionsKilled
    minions_killed = player.farming.totalMinionsKilled - jungle_minions_killed

    items = [player.economy.item0, player.economy.item1, player.economy.item2, player.economy.item3, player.economy.item4, player.economy.item5, player.economy.item6]
    keystone_rune = player.abilityUsage.perks.styles[0].selections[0].perk if player.abilityUsage.perks.styles else None
    summoner_spells_1 = player.abilityUsage.summoner1Id
    summoner_spells_2 = player.abilityUsage.summoner2Id
    summoner_spells_1_casts = player.abilityUsage.summoner1Casts
    summoner_spells_2_casts = player.abilityUsage.summoner2Casts

    first_blood = player.combat.firstBloodKill
    double_kills = player.combat.doubleKills
    triple_kills = player.combat.tripleKills
    quadra_kills = player.combat.quadraKills
    penta_kills = player.combat.pentaKills
    solo_kills = kills - double_kills - triple_kills - quadra_kills - penta_kills

    return {
        "champion": champion,
        "q_ability_casts": q_ability_casts,
        "w_ability_casts": w_ability_casts,
        "e_ability_casts": e_ability_casts,
        "r_ability_casts": r_ability_casts,
        "play_time": play_time,
        "game_count": game_count,
        "win": win,
        "kills": kills,
        "deaths": deaths,
        "assists": assists,
        "kda": kda,
        "gold_earned": gold_earned,
        "jungle_minions_killed": jungle_minions_killed,
        "minions_killed": minions_killed,
        "items": items,
        "keystone_rune": keystone_rune,
        "summoner_spells_1": summoner_spells_1,
        "summoner_spells_2": summoner_spells_2,
        "summoner_spells_1_casts": summoner_spells_1_casts,
        "summoner_spells_2_casts": summoner_spells_2_casts,
        "first_blood": first_blood,
        "double_kills": double_kills,
        "triple_kills": triple_kills,
        "quadra_kills": quadra_kills,
        "penta_kills": penta_kills,
        "solo_kills": solo_kills,
    }

Mutlikills - Left Arm
Solo Kills
Doubles
Trips
Quads
Penta

Superlatives - Weapon
Highest winstreak
highest KDA

### Info

In [ ]:
def info_from_raw_targon_metrics(raw_metrics: List[Dict[str, Any]]):
    champion_counts = {}
    for metric in raw_metrics:
        champion = metric["champion"]
        if champion not in champion_counts:
            champion_counts[champion] = 0
        champion_counts[champion] += 1
    if champion_counts:
        most_played_champion = sorted(champion_counts.items(), key=lambda x: x[1], reverse=True)[0]
        relevant_champion_metrics = [m for m in raw_metrics if m["champion"] == most_played_champion[0]]
    else:
        most_played_champion = (None, 0)
        relevant_champion_metrics = []

    q_ability_casts = 0
    w_ability_casts = 0
    e_ability_casts = 0
    r_ability_casts = 0

    play_time = 0
    game_count = 0
    win = 0

    kills = 0
    deaths = 0
    assists = 0

    gold_earned = 0
    jungle_minions_killed = 0
    minions_killed = 0

    item_counter = {}
    keystone_rune_counter = {}
    summoner_spells_cast_counter = {}

    first_blood = 0
    double_kills = 0
    triple_kills = 0
    quadra_kills = 0
    penta_kills = 0
    solo_kills = 0

    highest_kda = 0
    highest_winstreak = 0
    current_winstreak = 0

    for m in relevant_champion_metrics:
        q_ability_casts += m["q_ability_casts"]
        w_ability_casts += m["w_ability_casts"]
        e_ability_casts += m["e_ability_casts"]
        r_ability_casts += m["r_ability_casts"]
        play_time += m["play_time"]
        game_count += m["game_count"]
        win += m["win"]
        kills += m["kills"]
        deaths += m["deaths"]
        assists += m["assists"]
        gold_earned += m["gold_earned"]
        jungle_minions_killed += m["jungle_minions_killed"]
        minions_killed += m["minions_killed"]
        first_blood += 1 if m["first_blood"] else 0
        double_kills += m["double_kills"]
        triple_kills += m["triple_kills"]
        quadra_kills += m["quadra_kills"]
        penta_kills += m["penta_kills"]
        solo_kills += m["solo_kills"]
        for item in m["items"]:
            if item not in item_counter:
                item_counter[item] = 0
            item_counter[item] += 1
        keystone_rune = m["keystone_rune"]
        if keystone_rune not in keystone_rune_counter:
            keystone_rune_counter[keystone_rune] = 0
        keystone_rune_counter[keystone_rune] += 1
        summoner_spells_1 = m["summoner_spells_1"]
        summoner_spells_2 = m["summoner_spells_2"]
        if summoner_spells_1 not in summoner_spells_cast_counter:
            summoner_spells_cast_counter[summoner_spells_1] = 0
        summoner_spells_cast_counter[summoner_spells_1] += m["summoner_spells_1_casts"]
        if summoner_spells_2 not in summoner_spells_cast_counter:
            summoner_spells_cast_counter[summoner_spells_2] = 0
        summoner_spells_cast_counter[summoner_spells_2] += m["summoner_spells_2_casts"]

        highest_kda = max(highest_kda, m["kda"])
        if m["win"]:
            current_winstreak += 1
            highest_winstreak = max(highest_winstreak, current_winstreak)
        else:
            current_winstreak = 0

    # no item?
    if 0 in item_counter:
        del item_counter[0]

    # no keystone rune?
    if None in keystone_rune_counter:
        del keystone_rune_counter[None]


    top_3_items = sorted(item_counter.items(), key=lambda x: x[1], reverse=True)[:3] if item_counter else []
    most_used_keystone_rune = sorted(keystone_rune_counter.items(), key=lambda x: x[1], reverse=True)[0] if keystone_rune_counter else (None, 0)
    top_2_summoner_spells = sorted(summoner_spells_cast_counter.items(), key=lambda x: x[1], reverse=True)[:2] if summoner_spells_cast_counter else []

    return {
        "most_played_champion": most_played_champion,
        "q_ability_casts": q_ability_casts,
        "w_ability_casts": w_ability_casts,
        "e_ability_casts": e_ability_casts,
        "r_ability_casts": r_ability_casts,
        "play_time": play_time,
        "game_count": game_count,
        "win": win,
        "kills": kills,
        "deaths": deaths,
        "assists": assists,
        "kda": (kills + assists) / max(1, deaths),
        "gold_earned": gold_earned,
        "jungle_minions_killed": jungle_minions_killed,
        "minions_killed": minions_killed,
        "top_3_items": top_3_items,
        "most_used_keystone_rune": most_used_keystone_rune,
        "top_2_summoner_spells": top_2_summoner_spells,
        "first_blood": first_blood,
        "double_kills": double_kills,
        "triple_kills": triple_kills,
        "quadra_kills": quadra_kills,
        "penta_kills": penta_kills,
        "solo_kills": solo_kills,
        "highest_kda": highest_kda,
        "highest_winstreak": highest_winstreak,
    }

## Piltover

### Raw

In [ ]:
def get_raw_piltover_metrics_for_match(puuid: str, match_data: MatchData, timeline_data: MatchTimelineData):
    
        
    return None

### Info

In [ ]:
def info_from_raw_piltover_metrics(raw_metrics: List[Dict[str, Any]]):
    return {}

## Zaun

### Raw

In [ ]:
def get_raw_zaun_metrics_for_match(puuid: str, match_data: MatchData, timeline_data: MatchTimelineData):
    return None

### Info

In [ ]:
def info_from_raw_zaun_metrics(raw_metrics: List[Dict[str, Any]]):
    return {}

## Bilgewater

### Raw

In [ ]:
def get_raw_bilgewater_metrics_for_match(puuid: str, match_data: MatchData, timeline_data: MatchTimelineData):
    player = match_data.players[next(i for i, p in enumerate(match_data.players) if p.identity.puuid == puuid)]
    play_time = player.combat.timePlayed / 60
    gold_earned = player.economy.goldEarned
    jungle_minions_killed = player.farming.neutralMinionsKilled + player.farming.totalAllyJungleMinionsKilled + player.farming.totalEnemyJungleMinionsKilled
    minions_killed = player.farming.totalMinionsKilled - jungle_minions_killed
    items = [player.economy.item0, player.economy.item1, player.economy.item2, player.economy.item3, player.economy.item4, player.economy.item5, player.economy.item6]

    highest_shutdown_bounty = 0
    total_shutdown_bounty = 0
    for frame in timeline_data.frames:
        for event in frame.events:
            if event.type == "CHAMPION_KILL":
                if event.killerId == player.identity.participantId:
                    total_shutdown_bounty += event.shutdownBounty
                    highest_shutdown_bounty = max(highest_shutdown_bounty, event.shutdownBounty)
    return {
        "play_time": play_time,
        "gold_earned": gold_earned,
        "jungle_minions_killed": jungle_minions_killed,
        "minions_killed": minions_killed,
        "items": items,
        "highest_shutdown_bounty": highest_shutdown_bounty,
        "total_shutdown_bounty": total_shutdown_bounty,
    }

### Info

In [ ]:
def info_from_raw_bilgewater_metrics(raw_metrics: List[Dict[str, Any]]):
    play_time = 0
    total_gold_earned = 0
    total_jungle_minions_killed = 0
    total_minions_killed = 0
    item_counter = {}
    highest_shutdown_bounty = 0
    total_shutdown_bounty = 0

    most_gold_in_single_game = 0
    most_cs_in_single_game = 0

    for m in raw_metrics:
        play_time += m["play_time"]
        total_gold_earned += m["gold_earned"]
        total_jungle_minions_killed += m["jungle_minions_killed"]
        total_minions_killed += m["minions_killed"]
        highest_shutdown_bounty = max(highest_shutdown_bounty, m["highest_shutdown_bounty"])
        total_shutdown_bounty += m["total_shutdown_bounty"]
        for item in m["items"]:
            if item not in item_counter:
                item_counter[item] = 0
            item_counter[item] += 1
        most_gold_in_single_game = max(most_gold_in_single_game, m["gold_earned"])
        most_cs_in_single_game = max(most_cs_in_single_game, m["jungle_minions_killed"] + m["minions_killed"])

    # no item?
    if 0 in item_counter:
        del item_counter[0]

    top_3_items = sorted(item_counter.items(), key=lambda x: x[1], reverse=True)[:3]

    avg_cs_per_min = (total_jungle_minions_killed + total_minions_killed) / max(1, play_time)

    return {
        "total_gold_earned": total_gold_earned,
        "avg_cs_per_min": avg_cs_per_min,
        "top_3_items": top_3_items,
        "highest_shutdown_bounty": highest_shutdown_bounty,
        "total_shutdown_bounty": total_shutdown_bounty,
        "most_gold_in_single_game": most_gold_in_single_game,
        "most_cs_in_single_game": most_cs_in_single_game,
    }

## Shadow Isles

### Raw

In [ ]:
def get_raw_shadow_isles_metrics_for_match(puuid: str, match_data: MatchData, timeline_data: MatchTimelineData):
    player = match_data.players[next(i for i, p in enumerate(match_data.players) if p.identity.puuid == puuid)]
    play_time = player.combat.timePlayed / 60
    gameMode = match_data.game.gameMode
    deaths = player.combat.deaths
    time_spent_dead = player.combat.totalTimeSpentDead / 60
    death_positions = []
    revenge_kills = 0
    latest_death_for_revenge = None
    for frame in timeline_data.frames:
        for event in frame.events:
            if event.type == "CHAMPION_KILL":
                if event.victimId == player.identity.participantId:
                    death_positions.append((event.position.x, event.position.y))
                    latest_death_for_revenge = {
                        "time": event.timestamp,
                        "killerId": event.killerId,
                    }
                if (
                    latest_death_for_revenge
                    and event.victimId == latest_death_for_revenge["killerId"]
                    and event.killerId == player.identity.participantId
                    # within 3 minutes
                    and (event.timestamp - latest_death_for_revenge["time"] <= 3 * 60 * 1000)
                ):
                    revenge_kills += 1
                    latest_death_for_revenge = None

    game_ended_in_surrender = match_data.game.gameEndedInSurrender
    return {
        "play_time": play_time,
        "game_mode": gameMode,
        "deaths": deaths,
        "time_spent_dead": time_spent_dead,
        "death_positions": death_positions,
        "revenge_kills": revenge_kills,
        "game_ended_in_surrender": game_ended_in_surrender,
    }

### Info

In [ ]:
def info_from_raw_shadow_isles_metrics(raw_metrics: List[Dict[str, Any]]):
    total_play_time = 0
    total_deaths = 0
    total_time_spent_dead = 0
    total_revenge_kills = 0
    surrender_count = 0
    death_position_counter = set()
    max_deaths_in_a_game = 0

    for m in raw_metrics:
        total_play_time += m["play_time"]
        total_deaths += m["deaths"]
        total_time_spent_dead += m["time_spent_dead"]
        total_revenge_kills += m["revenge_kills"]
        if m["game_ended_in_surrender"]:
            surrender_count += 1
        for pos in m["death_positions"]:
            death_position_counter.add(pos)

        max_deaths_in_a_game = max(max_deaths_in_a_game, m["deaths"])

    death_ratio = total_time_spent_dead / max(1, total_play_time)

    return {
        "total_deaths": total_deaths,
        "total_time_spent_dead": total_time_spent_dead,
        "max_deaths_in_a_game": max_deaths_in_a_game,
        "death_ratio": death_ratio,
        "total_revenge_kills": total_revenge_kills,
        "surrender_count": surrender_count,
        "death_positions": list(death_position_counter),
    }

## Shurima

### Raw

In [ ]:
def get_raw_shurima_metrics_for_match(puuid: str, match_data: MatchData, timeline_data: MatchTimelineData):
    player = match_data.players[next(i for i, p in enumerate(match_data.players) if p.identity.puuid == puuid)]
    play_time = player.combat.timePlayed / 60
    team = set([p.identity.participantId for p in match_data.players if p.identity.teamId == player.identity.teamId])
    turret_kills = player.objectives.turretKills
    dragon_kills = player.objectives.dragonKills
    baron_kills = player.objectives.baronKills
    herald_kills = 0
    atakhan_kills = 0
    dragon_killed = False
    first_dragon_kill = False
    baron_killed = False
    first_baron_kill = False
    rift_herald_killed = False
    first_rift_herald_kill = False

    total_team_objectives_structures = 0
    total_team_objectives_jungle_bosses = 0
    total_player_objectives_structures_participation = 0
    total_player_objectives_jungle_bosses_participation = 0

    for frame in timeline_data.frames:
        for event in frame.events:
            if event.type == "ELITE_MONSTER_KILL":
                if event.monsterType == "RIFTHERALD":
                    if event.killerId in team:
                        total_team_objectives_jungle_bosses += 1
                        if event.killerId == player.identity.participantId or event.assistingParticipantIds and player.identity.participantId in event.assistingParticipantIds:
                            total_player_objectives_jungle_bosses_participation += 1
                    if event.killerId == player.identity.participantId:
                        herald_kills += 1
                        if rift_herald_killed is False:
                            first_rift_herald_kill = True
                    rift_herald_killed = True
                elif event.monsterType == "ATAKHAN":
                    if event.killerId in team:
                        total_team_objectives_jungle_bosses += 1
                        if event.killerId == player.identity.participantId or event.assistingParticipantIds and player.identity.participantId in event.assistingParticipantIds:
                            total_player_objectives_jungle_bosses_participation += 1
                    if event.killerId == player.identity.participantId:
                        atakhan_kills += 1
                elif event.monsterType == "DRAGON":
                    if event.killerId in team:
                        total_team_objectives_jungle_bosses += 1
                        if event.killerId == player.identity.participantId or event.assistingParticipantIds and player.identity.participantId in event.assistingParticipantIds:
                            total_player_objectives_jungle_bosses_participation += 1
                    if dragon_killed is False:
                        if event.killerId == player.identity.participantId:
                            first_dragon_kill = True
                    dragon_killed = True
                elif event.monsterType == "BARON_NASHOR":
                    if event.killerId in team:
                        total_team_objectives_jungle_bosses += 1
                        if event.killerId == player.identity.participantId or event.assistingParticipantIds and player.identity.participantId in event.assistingParticipantIds:
                            total_player_objectives_jungle_bosses_participation += 1
                    if baron_killed is False:
                        if event.killerId == player.identity.participantId:
                            first_baron_kill = True
                    baron_killed = True
            elif event.type == "BUILDING_KILL":
                if event.killerId in team:
                    total_team_objectives_structures += 1
                    if event.killerId == player.identity.participantId or event.assistingParticipantIds and player.identity.participantId in event.assistingParticipantIds:
                        total_player_objectives_structures_participation += 1

    objective_participation_ratio = (total_player_objectives_structures_participation + total_player_objectives_jungle_bosses_participation) / max(
        1, (total_team_objectives_structures + total_team_objectives_jungle_bosses)
    )
    first_tower_kill = player.objectives.firstTowerKill
    damage_dealt_to_objectives = player.objectives.damageDealtToObjectives
    damage_dealt_to_structures = player.objectives.damageDealtToBuildings + player.objectives.damageDealtToTurrets

    return {
        "play_time": play_time,
        "turret_kills": turret_kills,
        "dragon_kills": dragon_kills,
        "baron_kills": baron_kills,
        "herald_kills": herald_kills,
        "atakhan_kills": atakhan_kills,
        "first_tower_kill": first_tower_kill,
        "first_dragon_kill": first_dragon_kill,
        "first_baron_kill": first_baron_kill,
        "first_rift_herald_kill": first_rift_herald_kill,
        "objective_participation_ratio": objective_participation_ratio,
        "damage_dealt_to_objectives": damage_dealt_to_objectives,
        "damage_dealt_to_structures": damage_dealt_to_structures,
    }

### Info

In [ ]:
def info_from_raw_shurima_metrics(raw_metrics: List[Dict[str, Any]]):
    total_turret_kills = 0
    total_dragon_kills = 0
    total_baron_kills = 0
    total_herald_kills = 0
    total_atakhan_kills = 0
    total_first_tower_kill = 0
    total_first_dragon_kill = 0
    total_first_baron_kill = 0
    total_first_rift_herald_kill = 0
    total_objective_participation_ratio = 0
    total_damage_dealt_to_objectives = 0
    total_damage_dealt_to_structures = 0

    for m in raw_metrics:
        total_turret_kills += m["turret_kills"]
        total_dragon_kills += m["dragon_kills"]
        total_baron_kills += m["baron_kills"]
        total_herald_kills += m["herald_kills"]
        total_atakhan_kills += m["atakhan_kills"]
        total_first_tower_kill += 1 if m["first_tower_kill"] else 0
        total_first_dragon_kill += 1 if m["first_dragon_kill"] else 0
        total_first_baron_kill += 1 if m["first_baron_kill"] else 0
        total_first_rift_herald_kill += 1 if m["first_rift_herald_kill"] else 0
        total_objective_participation_ratio += m["objective_participation_ratio"]
        total_damage_dealt_to_objectives += m["damage_dealt_to_objectives"]
        total_damage_dealt_to_structures += m["damage_dealt_to_structures"]

    avg_objective_participation_ratio = total_objective_participation_ratio / max(1, len(raw_metrics))

    return {
        "total_turret_kills": total_turret_kills,
        "total_dragon_kills": total_dragon_kills,
        "total_baron_kills": total_baron_kills,
        "total_herald_kills": total_herald_kills,
        "total_atakhan_kills": total_atakhan_kills,
        "total_first_tower_kill": total_first_tower_kill,
        "total_first_dragon_kill": total_first_dragon_kill,
        "total_first_baron_kill": total_first_baron_kill,
        "total_first_rift_herald_kill": total_first_rift_herald_kill,
        "avg_objective_participation_ratio": avg_objective_participation_ratio,
        "total_damage_dealt_to_objectives": total_damage_dealt_to_objectives,
        "total_damage_dealt_to_structures": total_damage_dealt_to_structures,
    }

## Ixtal

### Raw

In [ ]:
def get_raw_ixtal_metrics_for_match(puuid: str, match_data: MatchData, timeline_data: MatchTimelineData):
    player = match_data.players[next(i for i, p in enumerate(match_data.players) if p.identity.puuid == puuid)]
    play_time = player.combat.timePlayed / 60

    vision_score_per_min = player.vision.visionScore / max(1, play_time)
    allInPings = player.pings.allInPings
    assistMePings = player.pings.assistMePings
    basicPings = player.pings.basicPings
    commandPings = player.pings.commandPings
    dangerPings = player.pings.dangerPings
    getBackPings = player.pings.getBackPings
    holdPings = player.pings.holdPings
    onMyWayPings = player.pings.onMyWayPings
    pushPings = player.pings.pushPings
    retreatPings = player.pings.retreatPings
    needVisionPings = player.pings.needVisionPings
    enemyMissingPings = player.pings.enemyMissingPings
    enemyVisionPings = player.pings.enemyVisionPings
    visionClearedPings = player.pings.visionClearedPings

    objectives_stolen = player.objectives.objectivesStolen

    avg_jungle_share_when_jungle = "N/A"
    jungle_role = player.identity.lane == "JUNGLE"
    # TODO: maybe tailor for invade instead
    if jungle_role:
        other_jungle = next((p for p in match_data.players if p.identity.lane == "JUNGLE" and p.identity.participantId != player.identity.participantId), None)
        if other_jungle is not None:
            player_jungle_minions = player.farming.neutralMinionsKilled + player.farming.totalAllyJungleMinionsKilled + player.farming.totalEnemyJungleMinionsKilled
            other_total_jungle_minions = other_jungle.farming.neutralMinionsKilled + other_jungle.farming.totalAllyJungleMinionsKilled + other_jungle.farming.totalEnemyJungleMinionsKilled
            avg_jungle_share_when_jungle = player_jungle_minions / max(1, player_jungle_minions + other_total_jungle_minions)
        else:
            avg_jungle_share_when_jungle = 1.0

    total_wards_placed = player.vision.wardsPlaced
    total_wards_killed = player.vision.wardsKilled

    return {
        "play_time": play_time,
        "vision_score_per_min": vision_score_per_min,
        "allInPings": allInPings,
        "assistMePings": assistMePings,
        "basicPings": basicPings,
        "commandPings": commandPings,
        "dangerPings": dangerPings,
        "getBackPings": getBackPings,
        "holdPings": holdPings,
        "onMyWayPings": onMyWayPings,
        "pushPings": pushPings,
        "retreatPings": retreatPings,
        "needVisionPings": needVisionPings,
        "enemyMissingPings": enemyMissingPings,
        "enemyVisionPings": enemyVisionPings,
        "visionClearedPings": visionClearedPings,
        "objectives_stolen": objectives_stolen,
        "jungle_role": jungle_role,
        "avg_jungle_share_when_jungle": avg_jungle_share_when_jungle,
        "total_wards_placed": total_wards_placed,
        "total_wards_killed": total_wards_killed,
    }

### Info

In [ ]:
def info_from_raw_ixtal_metrics(raw_metrics: List[Dict[str, Any]]):
    total_play_time = 0
    total_vision_score_per_min = 0
    total_allInPings = 0
    total_assistMePings = 0
    total_basicPings = 0
    total_commandPings = 0
    total_dangerPings = 0
    total_getBackPings = 0
    total_holdPings = 0
    total_onMyWayPings = 0
    total_pushPings = 0
    total_retreatPings = 0
    total_needVisionPings = 0
    total_enemyMissingPings = 0
    total_enemyVisionPings = 0
    total_visionClearedPings = 0
    total_objectives_stolen = 0
    total_jungle_games = 0
    total_avg_jungle_share_when_jungle = 0
    total_wards_placed = 0
    total_wards_killed = 0

    for m in raw_metrics:
        total_play_time += m["play_time"]
        total_vision_score_per_min += m["vision_score_per_min"]
        total_allInPings += m["allInPings"]
        total_assistMePings += m["assistMePings"]
        total_basicPings += m["basicPings"]
        total_commandPings += m["commandPings"]
        total_dangerPings += m["dangerPings"]
        total_getBackPings += m["getBackPings"]
        total_holdPings += m["holdPings"]
        total_onMyWayPings += m["onMyWayPings"]
        total_pushPings += m["pushPings"]
        total_retreatPings += m["retreatPings"]
        total_needVisionPings += m["needVisionPings"]
        total_enemyMissingPings += m["enemyMissingPings"]
        total_enemyVisionPings += m["enemyVisionPings"]
        total_visionClearedPings += m["visionClearedPings"]
        total_objectives_stolen += m["objectives_stolen"]
        if m["jungle_role"]:
            total_jungle_games += 1
            total_avg_jungle_share_when_jungle += m["avg_jungle_share_when_jungle"]
        total_wards_placed += m["total_wards_placed"]
        total_wards_killed += m["total_wards_killed"]
    avg_vision_score_per_min = total_vision_score_per_min / max(1, len(raw_metrics))
    avg_jungle_share_when_jungle = total_avg_jungle_share_when_jungle / max(1, total_jungle_games)

    return {
        "total_allInPings": total_allInPings,
        "total_assistMePings": total_assistMePings,
        "total_basicPings": total_basicPings,
        "total_commandPings": total_commandPings,
        "total_dangerPings": total_dangerPings,
        "total_getBackPings": total_getBackPings,
        "total_holdPings": total_holdPings,
        "total_onMyWayPings": total_onMyWayPings,
        "total_pushPings": total_pushPings,
        "total_retreatPings": total_retreatPings,
        "total_needVisionPings": total_needVisionPings,
        "total_enemyMissingPings": total_enemyMissingPings,
        "total_enemyVisionPings": total_enemyVisionPings,
        "total_visionClearedPings": total_visionClearedPings,
        "total_objectives_stolen": total_objectives_stolen,
        "avg_vision_score_per_min": avg_vision_score_per_min,
        "avg_jungle_share_when_jungle": avg_jungle_share_when_jungle,
        "total_total_wards_placed": total_wards_placed,
        "total_total_wards_killed": total_wards_killed,
    }

## Noxus

### Raw

In [ ]:
def get_raw_noxus_metrics_for_match(puuid: str, match_data: MatchData, timeline_data: MatchTimelineData):
    player = match_data.players[next(i for i, p in enumerate(match_data.players) if p.identity.puuid == puuid)]
    physical_damage_dealt_to_champions = player.combat.physicalDamageDealtToChampions
    magic_damage_dealt_to_champions = player.combat.magicDamageDealtToChampions
    true_damage_dealt_to_champions = player.combat.trueDamageDealtToChampions

    kills = player.combat.kills
    assists = player.combat.assists
    deaths = player.combat.deaths
    kda = (kills + assists) / max(1, deaths)
    first_blood = player.combat.firstBloodKill

    double_kills = player.combat.doubleKills
    triple_kills = player.combat.tripleKills
    quadra_kills = player.combat.quadraKills
    penta_kills = player.combat.pentaKills

    largest_killing_spree = player.combat.largestKillingSpree

    total_team_damage_to_champions = 0
    for p in match_data.players:
        if p.identity.teamId == player.identity.teamId:
            total_team_damage_to_champions += p.combat.totalDamageDealtToChampions
    team_damage_share = (physical_damage_dealt_to_champions + magic_damage_dealt_to_champions + true_damage_dealt_to_champions) / max(1, total_team_damage_to_champions)
    return {
        "physical_damage_dealt_to_champions": physical_damage_dealt_to_champions,
        "magic_damage_dealt_to_champions": magic_damage_dealt_to_champions,
        "true_damage_dealt_to_champions": true_damage_dealt_to_champions,
        "kills": kills,
        "assists": assists,
        "kda": kda,
        "first_blood": first_blood,
        "double_kills": double_kills,
        "triple_kills": triple_kills,
        "quadra_kills": quadra_kills,
        "penta_kills": penta_kills,
        "largest_killing_spree": largest_killing_spree,
        "team_damage_share": team_damage_share,
    }

### Info

In [ ]:
def info_from_raw_noxus_metrics(raw_metrics: List[Dict[str, Any]]):
    total_physical_damage_dealt_to_champions = 0
    total_magic_damage_dealt_to_champions = 0
    total_true_damage_dealt_to_champions = 0

    total_kills = 0
    total_assists = 0
    total_kda = 0
    total_first_bloods = 0

    total_double_kills = 0
    total_triple_kills = 0
    total_quadra_kills = 0
    total_penta_kills = 0

    highest_largest_killing_spree = 0
    total_team_damage_share = 0

    for m in raw_metrics:
        total_physical_damage_dealt_to_champions += m["physical_damage_dealt_to_champions"]
        total_magic_damage_dealt_to_champions += m["magic_damage_dealt_to_champions"]
        total_true_damage_dealt_to_champions += m["true_damage_dealt_to_champions"]
        total_kills += m["kills"]
        total_assists += m["assists"]
        total_kda += m["kda"]
        total_first_bloods += 1 if m["first_blood"] else 0
        total_double_kills += m["double_kills"]
        total_triple_kills += m["triple_kills"]
        total_quadra_kills += m["quadra_kills"]
        total_penta_kills += m["penta_kills"]
        highest_largest_killing_spree = max(highest_largest_killing_spree, m["largest_killing_spree"])
        total_team_damage_share += m["team_damage_share"]
    avg_kda = total_kda / max(1, len(raw_metrics))
    avg_team_damage_share = total_team_damage_share / max(1, len(raw_metrics))
    return {
        "total_physical_damage_dealt_to_champions": total_physical_damage_dealt_to_champions,
        "total_magic_damage_dealt_to_champions": total_magic_damage_dealt_to_champions,
        "total_true_damage_dealt_to_champions": total_true_damage_dealt_to_champions,
        "total_kills": total_kills,
        "total_assists": total_assists,
        "avg_kda": avg_kda,
        "total_first_bloods": total_first_bloods,
        "total_double_kills": total_double_kills,
        "total_triple_kills": total_triple_kills,
        "total_quadra_kills": total_quadra_kills,
        "total_penta_kills": total_penta_kills,
        "highest_largest_killing_spree": highest_largest_killing_spree,
        "avg_team_damage_share": avg_team_damage_share,
    }

## Freljord

### Raw

In [ ]:
def get_raw_freljord_metrics_for_match(puuid: str, match_data: MatchData, timeline_data: MatchTimelineData):
    player = match_data.players[next(i for i, p in enumerate(match_data.players) if p.identity.puuid == puuid)]
    time_played = player.combat.timePlayed / 60
    physical_damage_taken = player.combat.physicalDamageTaken
    magic_damage_taken = player.combat.magicDamageTaken
    true_damage_taken = player.combat.trueDamageTaken
    damage_self_mitigated = player.combat.damageSelfMitigated
    total_heal = player.combat.totalHeal
    total_heals_on_teammates = player.combat.totalHealsOnTeammates
    survival_ratio = (player.combat.timePlayed - player.combat.totalTimeSpentDead) / max(1, player.combat.timePlayed)
    snowball_throws = 0
    if player.abilityUsage.summoner1Id == 32:
        snowball_throws = player.abilityUsage.summoner1Casts
    elif player.abilityUsage.summoner2Id == 32:
        snowball_throws = player.abilityUsage.summoner2Casts
    return {
        "time_played": time_played,
        "physical_damage_taken": physical_damage_taken,
        "magic_damage_taken": magic_damage_taken,
        "true_damage_taken": true_damage_taken,
        "damage_self_mitigated": damage_self_mitigated,
        "total_heal": total_heal,
        "total_heals_on_teammates": total_heals_on_teammates,
        "survival_ratio": survival_ratio,
        "snowball_throws": snowball_throws,
    }

### Info

In [ ]:
def info_from_raw_freljord_metrics(raw_metrics: List[Dict[str, Any]]):
    total_play_time = 0
    total_physical_damage_taken = 0
    total_magic_damage_taken = 0
    total_true_damage_taken = 0
    total_damage_self_mitigated = 0
    total_heal = 0
    total_heals_on_teammates = 0
    total_survival_ratio = 0
    total_snowball_throws = 0
    for m in raw_metrics:
        total_play_time += m["time_played"]
        total_physical_damage_taken += m["physical_damage_taken"]
        total_magic_damage_taken += m["magic_damage_taken"]
        total_true_damage_taken += m["true_damage_taken"]
        total_damage_self_mitigated += m["damage_self_mitigated"]
        total_heal += m["total_heal"]
        total_heals_on_teammates += m["total_heals_on_teammates"]
        total_survival_ratio += m["survival_ratio"]
        total_snowball_throws += m["snowball_throws"]
    avg_survival_ratio = total_survival_ratio / max(1, len(raw_metrics))
    return {
        "total_play_time": total_play_time,
        "total_physical_damage_taken": total_physical_damage_taken,
        "total_magic_damage_taken": total_magic_damage_taken,
        "total_true_damage_taken": total_true_damage_taken,
        "total_damage_self_mitigated": total_damage_self_mitigated,
        "total_heal": total_heal,
        "total_heals_on_teammates": total_heals_on_teammates,
        "avg_survival_ratio": avg_survival_ratio,
        "total_snowball_throws": total_snowball_throws,
    }

## Runner

In [ ]:
def process_player_stats(puuid: str):
    with open(f"{bucket_root}/players/{puuid}/info.json", "r", encoding="utf-8") as f:
        account_data = json.load(f)
        matches = account_data["matches"]
    puuid = account_data["puuid"]
    # matches = matches[:5]  # limit to 5 matches for now
    matches_counted_sr = 0
    raw_ionia_metrics_sr = []
    raw_demacia_metrics_sr = []
    raw_targon_metrics_sr = []
    raw_piltover_metrics_sr = []
    raw_zaun_metrics_sr = []
    raw_bilgewater_metrics_sr = []
    raw_shadow_isles_metrics_sr = []
    raw_shurima_metrics_sr = []
    raw_ixtal_metrics_sr = []
    raw_noxus_metrics_sr = []
    raw_freljord_metrics_sr = []

    matches_counted_aram = 0
    # raw_ionia_metrics_aram = []
    raw_demacia_metrics_aram = []
    # raw_targon_metrics_aram = []
    raw_piltover_metrics_aram = []
    raw_zaun_metrics_aram = []
    raw_bilgewater_metrics_aram = []
    raw_shadow_isles_metrics_aram = []
    # raw_shurima_metrics_aram = []
    # raw_ixtal_metrics_aram = []
    raw_noxus_metrics_aram = []
    raw_freljord_metrics_aram = []

    for match_id in matches:
        # try:
        # except Exception as e:
        #     print(f"Error processing match {match_id} for player {puuid}: {e}")
        #     continue

        with open(f"{bucket_root}/processed/{match_id}/{match_id}_processed.json", "r", encoding="utf-8") as f:
            match_data = MatchData.model_validate_json(f.read())
        with open(f"{bucket_root}/processed/{match_id}/{match_id}_timeline_processed.json", "r", encoding="utf-8") as f:
            timeline_data = MatchTimelineData.model_validate_json(f.read())
        gameMode = match_data.game.gameMode
        if gameMode == "CLASSIC" or gameMode == "SWIFTPLAY":
            raw_ionia_metrics_sr.append(get_raw_ionia_metrics_for_match(puuid, match_data, timeline_data))
            raw_demacia_metrics_sr.append(get_raw_demacia_metrics_for_match(puuid, match_data, timeline_data))
            raw_targon_metrics_sr.append(get_raw_targon_metrics_for_match(puuid, match_data, timeline_data))
            raw_piltover_metrics_sr.append(get_raw_piltover_metrics_for_match(puuid, match_data, timeline_data))
            raw_zaun_metrics_sr.append(get_raw_zaun_metrics_for_match(puuid, match_data, timeline_data))
            raw_bilgewater_metrics_sr.append(get_raw_bilgewater_metrics_for_match(puuid, match_data, timeline_data))
            raw_shadow_isles_metrics_sr.append(get_raw_shadow_isles_metrics_for_match(puuid, match_data, timeline_data))
            raw_shurima_metrics_sr.append(get_raw_shurima_metrics_for_match(puuid, match_data, timeline_data))
            raw_ixtal_metrics_sr.append(get_raw_ixtal_metrics_for_match(puuid, match_data, timeline_data))
            raw_noxus_metrics_sr.append(get_raw_noxus_metrics_for_match(puuid, match_data, timeline_data))
            raw_freljord_metrics_sr.append(get_raw_freljord_metrics_for_match(puuid, match_data, timeline_data))
            matches_counted_sr += 1
        elif gameMode == "ARAM":
            # raw_ionia_metrics_aram.append(get_raw_ionia_metrics_for_match(puuid, match_data, timeline_data))
            raw_demacia_metrics_aram.append(get_raw_demacia_metrics_for_match(puuid, match_data, timeline_data))
            # raw_targon_metrics_aram.append(get_raw_targon_metrics_for_match(puuid, match_data, timeline_data))
            raw_piltover_metrics_aram.append(get_raw_piltover_metrics_for_match(puuid, match_data, timeline_data))
            raw_zaun_metrics_aram.append(get_raw_zaun_metrics_for_match(puuid, match_data, timeline_data))
            raw_bilgewater_metrics_aram.append(get_raw_bilgewater_metrics_for_match(puuid, match_data, timeline_data))
            raw_shadow_isles_metrics_aram.append(get_raw_shadow_isles_metrics_for_match(puuid, match_data, timeline_data))
            # raw_shurima_metrics_aram.append(get_raw_shurima_metrics_for_match(puuid, match_data, timeline_data))
            # raw_ixtal_metrics_aram.append(get_raw_ixtal_metrics_for_match(puuid, match_data, timeline_data))
            raw_noxus_metrics_aram.append(get_raw_noxus_metrics_for_match(puuid, match_data, timeline_data))
            raw_freljord_metrics_aram.append(get_raw_freljord_metrics_for_match(puuid, match_data, timeline_data))
            matches_counted_aram += 1

    # summarize SR
    ionia_info_sr = info_from_raw_ionia_metrics(raw_ionia_metrics_sr)
    demacia_info_sr = info_from_raw_demacia_metrics(raw_demacia_metrics_sr)
    targon_info_sr = info_from_raw_targon_metrics(raw_targon_metrics_sr)
    piltover_info_sr = info_from_raw_piltover_metrics(raw_piltover_metrics_sr)
    zaun_info_sr = info_from_raw_zaun_metrics(raw_zaun_metrics_sr)
    bilgewater_info_sr = info_from_raw_bilgewater_metrics(raw_bilgewater_metrics_sr)
    shadow_isles_info_sr = info_from_raw_shadow_isles_metrics(raw_shadow_isles_metrics_sr)
    shurima_info_sr = info_from_raw_shurima_metrics(raw_shurima_metrics_sr)
    ixtal_info_sr = info_from_raw_ixtal_metrics(raw_ixtal_metrics_sr)
    noxus_info_sr = info_from_raw_noxus_metrics(raw_noxus_metrics_sr)
    freljord_info_sr = info_from_raw_freljord_metrics(raw_freljord_metrics_sr)
    # summarize ARAM
    # ionia_info_aram = info_from_raw_ionia_metrics(raw_ionia_metrics_aram)
    demacia_info_aram = info_from_raw_demacia_metrics(raw_demacia_metrics_aram)
    # targon_info_aram = info_from_raw_targon_metrics(raw_targon_metrics_aram)
    piltover_info_aram = info_from_raw_piltover_metrics(raw_piltover_metrics_aram)
    zaun_info_aram = info_from_raw_zaun_metrics(raw_zaun_metrics_aram)
    bilgewater_info_aram = info_from_raw_bilgewater_metrics(raw_bilgewater_metrics_aram)
    shadow_isles_info_aram = info_from_raw_shadow_isles_metrics(raw_shadow_isles_metrics_aram)
    # shurima_info_aram = info_from_raw_shurima_metrics(raw_shurima_metrics_aram)
    # ixtal_info_aram = info_from_raw_ixtal_metrics(raw_ixtal_metrics_aram)
    noxus_info_aram = info_from_raw_noxus_metrics(raw_noxus_metrics_aram)
    freljord_info_aram = info_from_raw_freljord_metrics(raw_freljord_metrics_aram)
    return {
        "sr": {
            "ionia_info": ionia_info_sr,
            "demacia_info": demacia_info_sr,
            "targon_info": targon_info_sr,
            "piltover_info": piltover_info_sr,
            "zaun_info": zaun_info_sr,
            "bilgewater_info": bilgewater_info_sr,
            "shadow_isles_info": shadow_isles_info_sr,
            "shurima_info": shurima_info_sr,
            "ixtal_info": ixtal_info_sr,
            "noxus_info": noxus_info_sr,
            "freljord_info": freljord_info_sr,
        },
        "aram": {
            # "ionia_info": ionia_info_aram,
            "demacia_info": demacia_info_aram,
            # "targon_info": targon_info_aram,
            "piltover_info": piltover_info_aram,
            "zaun_info": zaun_info_aram,
            "bilgewater_info": bilgewater_info_aram,
            "shadow_isles_info": shadow_isles_info_aram,
            # "shurima_info": shurima_info_aram,
            # "ixtal_info": ixtal_info_aram,
            "noxus_info": noxus_info_aram,
            "freljord_info": freljord_info_aram,
        }
    }

In [ ]:
process_player_stats(user_puuid)

In [ ]:
player_daaaata = process_player_stats(user_puuid)
with open(f"{bucket_root}/players/{user_puuid}/detailed_stats.json", "w", encoding="utf-8") as f:
    json.dump(player_daaaata, f, indent=4)

In [ ]:
def process_player_stats_tessst(puuid: str):
    with open(f"{bucket_root}/players/{puuid}/info.json", "r", encoding="utf-8") as f:
        account_data = json.load(f)
        matches = account_data["matches"]
    puuid = account_data["puuid"]
    matches = []
    # matches = matches[:5]  # limit to 5 matches for now
    matches_counted_sr = 0
    raw_ionia_metrics_sr = []
    raw_demacia_metrics_sr = []
    raw_targon_metrics_sr = []
    raw_piltover_metrics_sr = []
    raw_zaun_metrics_sr = []
    raw_bilgewater_metrics_sr = []
    raw_shadow_isles_metrics_sr = []
    raw_shurima_metrics_sr = []
    raw_ixtal_metrics_sr = []
    raw_noxus_metrics_sr = []
    raw_freljord_metrics_sr = []

    matches_counted_aram = 0
    # raw_ionia_metrics_aram = []
    raw_demacia_metrics_aram = []
    # raw_targon_metrics_aram = []
    raw_piltover_metrics_aram = []
    raw_zaun_metrics_aram = []
    raw_bilgewater_metrics_aram = []
    raw_shadow_isles_metrics_aram = []
    # raw_shurima_metrics_aram = []
    # raw_ixtal_metrics_aram = []
    raw_noxus_metrics_aram = []
    raw_freljord_metrics_aram = []

    for match_id in matches:
        # try:
        # except Exception as e:
        #     print(f"Error processing match {match_id} for player {puuid}: {e}")
        #     continue

        with open(f"{bucket_root}/processed/{match_id}/{match_id}_processed.json", "r", encoding="utf-8") as f:
            match_data = MatchData.model_validate_json(f.read())
        with open(f"{bucket_root}/processed/{match_id}/{match_id}_timeline_processed.json", "r", encoding="utf-8") as f:
            timeline_data = MatchTimelineData.model_validate_json(f.read())
        gameMode = match_data.game.gameMode
        if gameMode == "CLASSIC" or gameMode == "SWIFTPLAY":
            raw_ionia_metrics_sr.append(get_raw_ionia_metrics_for_match(puuid, match_data, timeline_data))
            raw_demacia_metrics_sr.append(get_raw_demacia_metrics_for_match(puuid, match_data, timeline_data))
            raw_targon_metrics_sr.append(get_raw_targon_metrics_for_match(puuid, match_data, timeline_data))
            raw_piltover_metrics_sr.append(get_raw_piltover_metrics_for_match(puuid, match_data, timeline_data))
            raw_zaun_metrics_sr.append(get_raw_zaun_metrics_for_match(puuid, match_data, timeline_data))
            raw_bilgewater_metrics_sr.append(get_raw_bilgewater_metrics_for_match(puuid, match_data, timeline_data))
            raw_shadow_isles_metrics_sr.append(get_raw_shadow_isles_metrics_for_match(puuid, match_data, timeline_data))
            raw_shurima_metrics_sr.append(get_raw_shurima_metrics_for_match(puuid, match_data, timeline_data))
            raw_ixtal_metrics_sr.append(get_raw_ixtal_metrics_for_match(puuid, match_data, timeline_data))
            raw_noxus_metrics_sr.append(get_raw_noxus_metrics_for_match(puuid, match_data, timeline_data))
            raw_freljord_metrics_sr.append(get_raw_freljord_metrics_for_match(puuid, match_data, timeline_data))
            matches_counted_sr += 1
        elif gameMode == "ARAM":
            # raw_ionia_metrics_aram.append(get_raw_ionia_metrics_for_match(puuid, match_data, timeline_data))
            raw_demacia_metrics_aram.append(get_raw_demacia_metrics_for_match(puuid, match_data, timeline_data))
            # raw_targon_metrics_aram.append(get_raw_targon_metrics_for_match(puuid, match_data, timeline_data))
            raw_piltover_metrics_aram.append(get_raw_piltover_metrics_for_match(puuid, match_data, timeline_data))
            raw_zaun_metrics_aram.append(get_raw_zaun_metrics_for_match(puuid, match_data, timeline_data))
            raw_bilgewater_metrics_aram.append(get_raw_bilgewater_metrics_for_match(puuid, match_data, timeline_data))
            raw_shadow_isles_metrics_aram.append(get_raw_shadow_isles_metrics_for_match(puuid, match_data, timeline_data))
            # raw_shurima_metrics_aram.append(get_raw_shurima_metrics_for_match(puuid, match_data, timeline_data))
            # raw_ixtal_metrics_aram.append(get_raw_ixtal_metrics_for_match(puuid, match_data, timeline_data))
            raw_noxus_metrics_aram.append(get_raw_noxus_metrics_for_match(puuid, match_data, timeline_data))
            raw_freljord_metrics_aram.append(get_raw_freljord_metrics_for_match(puuid, match_data, timeline_data))
            matches_counted_aram += 1

    # summarize SR
    ionia_info_sr = info_from_raw_ionia_metrics(raw_ionia_metrics_sr)
    demacia_info_sr = info_from_raw_demacia_metrics(raw_demacia_metrics_sr)
    targon_info_sr = info_from_raw_targon_metrics(raw_targon_metrics_sr)
    piltover_info_sr = info_from_raw_piltover_metrics(raw_piltover_metrics_sr)
    zaun_info_sr = info_from_raw_zaun_metrics(raw_zaun_metrics_sr)
    bilgewater_info_sr = info_from_raw_bilgewater_metrics(raw_bilgewater_metrics_sr)
    shadow_isles_info_sr = info_from_raw_shadow_isles_metrics(raw_shadow_isles_metrics_sr)
    shurima_info_sr = info_from_raw_shurima_metrics(raw_shurima_metrics_sr)
    ixtal_info_sr = info_from_raw_ixtal_metrics(raw_ixtal_metrics_sr)
    noxus_info_sr = info_from_raw_noxus_metrics(raw_noxus_metrics_sr)
    freljord_info_sr = info_from_raw_freljord_metrics(raw_freljord_metrics_sr)
    # summarize ARAM
    # ionia_info_aram = info_from_raw_ionia_metrics(raw_ionia_metrics_aram)
    demacia_info_aram = info_from_raw_demacia_metrics(raw_demacia_metrics_aram)
    # targon_info_aram = info_from_raw_targon_metrics(raw_targon_metrics_aram)
    piltover_info_aram = info_from_raw_piltover_metrics(raw_piltover_metrics_aram)
    zaun_info_aram = info_from_raw_zaun_metrics(raw_zaun_metrics_aram)
    bilgewater_info_aram = info_from_raw_bilgewater_metrics(raw_bilgewater_metrics_aram)
    shadow_isles_info_aram = info_from_raw_shadow_isles_metrics(raw_shadow_isles_metrics_aram)
    # shurima_info_aram = info_from_raw_shurima_metrics(raw_shurima_metrics_aram)
    # ixtal_info_aram = info_from_raw_ixtal_metrics(raw_ixtal_metrics_aram)
    noxus_info_aram = info_from_raw_noxus_metrics(raw_noxus_metrics_aram)
    freljord_info_aram = info_from_raw_freljord_metrics(raw_freljord_metrics_aram)
    return {
        "sr": {
            "ionia_info": ionia_info_sr,
            "demacia_info": demacia_info_sr,
            "targon_info": targon_info_sr,
            "piltover_info": piltover_info_sr,
            "zaun_info": zaun_info_sr,
            "bilgewater_info": bilgewater_info_sr,
            "shadow_isles_info": shadow_isles_info_sr,
            "shurima_info": shurima_info_sr,
            "ixtal_info": ixtal_info_sr,
            "noxus_info": noxus_info_sr,
            "freljord_info": freljord_info_sr,
        },
        "aram": {
            # "ionia_info": ionia_info_aram,
            "demacia_info": demacia_info_aram,
            # "targon_info": targon_info_aram,
            "piltover_info": piltover_info_aram,
            "zaun_info": zaun_info_aram,
            "bilgewater_info": bilgewater_info_aram,
            "shadow_isles_info": shadow_isles_info_aram,
            # "shurima_info": shurima_info_aram,
            # "ixtal_info": ixtal_info_aram,
            "noxus_info": noxus_info_aram,
            "freljord_info": freljord_info_aram,
        }
    }
process_player_stats_tessst(user_puuid)